<a href="https://colab.research.google.com/github/fadelmuli/covid-faq-bert/blob/main/COVID_FAQ_Retrieval_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IR Final Project: Question and Answering from Health FAQ/Q&A Data.
**Group 99**
1. Christian Wisnu - 2006506981
2. Fadel Muhammad - 2006560863
3. Mirza Alim - 2006609651
4. M. Noor Dwi Eldianto - 2006507044

**Disclaimers:**
- This work is not Q&A using dataset format like *SQuAD* or *Natural Question*.
- Making a fine-tuning dataset in format like SQuAD takes a lot of time.
- We never learned machile learning & deep learning before.
- This work is just an exploration.
- This is the first time we explore BERT.
- The dataset used in this work was harvested from Health Q&A website (https://www.alodokter.com/), contains long-question and long-answer pairs.

# A. Load the Data

## Data
Don't use all data. It will blow your ~mind~ RAM or will take so long processing the data. Please, run sample in testbed.

In [ ]:
import requests
import json

url = "https://raw.githubusercontent.com/mirza-alim-m/miscellaneous/main/alodokter_qa_covid19_only_1895_clean.json"
filename = url.rsplit('/', 1)[-1]
try:
  file = open(filename)
except:
  r = requests.get(url)
  with open(filename,'wb') as file: file.write(r.content)
  file = open(filename)

data = json.load(file)
data = data[0:950]
data[0]

{'_id': {'$oid': '5fcc86ba4d310cb2309bdbec'},
 'answer': 'Alo Inyong,\nTerimakasih atas pertanyaannya.\n\nMerebaknya infeksi Coronavirus yang bisa mematikan akhir-akhir ini memang kerap menyebabkan kekhwatiran tersendiri. Mengingat virus ini berasal dari negara China, maka tidak sedikit orang yang menjadi takut berlebihan, termasuk saat memberi barang dari negara tersebut. Lantas pertanyaannya, apakah kontak dengan barang yang dikirim dari China bisa meningkatkan Anda terinfeksi Coronavirus?\nCoronavirus bisa menular dari manusia ke manusia biasanya melalui terhirup atau tertelannya droplet saluran napas dari penderita, seperti saat ia bersin, batuk, meludah, atau berbicara dalam jarak dekat. Bisa juga, penularan terjadi secara tidak langsung, yakni melalui kontak dengan benda-benda yang terkontaminasi droplet saluran napas penderita, contohnya bila Anda bertukar peralatan makan, lap tangan, atau berjabat tangan.\nSejauh ini, penularan Coronavirus lewat benda-benda mati yang dikirimkan

In [ ]:
# There are some variations we can try. It searchs for the question, the answer, or both. Here we only take the questions to proceed.
sentences = [item["question"] for item in data]
sentences[0]

'Halo Dokter, saya ingin bertanya. apakah virus Corona dapat menyebar melalui barang-barang yang dibeli dari tiongkok? saya sering berbelanja online dan khawatir jika harus berbelanja barang pribadi seperti kosmetik dari Cina. mohon bantuannya. terima kasih'

## Testbed

###Sample

In [ ]:
# These are indices of dataset to sample.
sampleIndices = [13, 16, 23, 57, 63, 72, 75, 95, 109, 114, 137, 146, 150, 165, 173, 184, 202, 216, 219, 223, 
             227, 234, 243, 249, 253, 276, 281, 283, 289, 295, 296, 297, 332, 336, 339, 355, 369, 394, 398, 410, 
             421, 429, 432, 446, 467, 469, 476, 478, 486, 488, 491, 502, 518, 527, 533, 536, 549, 568, 575, 585, 
             594, 600, 607, 615, 621, 623, 630, 645, 650, 657, 677, 679, 686, 690, 691, 703, 930, 734, 738, 739, 
             762, 779, 780, 787, 790, 791, 793, 805, 815, 847, 868, 870, 883, 884, 885, 888, 897, 904, 927, 941]
sampleData = [data[index] for index in sampleIndices]
data = sampleData
data[0]

{'_id': {'$oid': '5fcc86ba4d310cb2309bdbfa'},
 'answer': 'Alo Nevi, terimakasih sudah bertanya ke Alodokter.\nPerkenalkan saya dr Rio akan mencoba menjawab pertanyaan anda.\nVirus corona adalah virus yang menyebabkan penyakit infeksi saluran pernafasan yang dikenal dengan COVID 19. Penyakit ini bisa menimbulkan gejala yang ringan hingga berat bahkan sampai kematian. Tanda dan gejala infeksi virus corona ini memang awalnya seperti penyakit flu biasanya. Namun khasnya, penyakit ini memiliki gejala seperti:\n\nDemam tinggi diatas 38 derajat celcius.\nBatuk.\nSesak nafas.\n\nPenyakit ini bisa menyerang semua kalangan usia, mulai dari anak-anak, remaja, dewasa, dan juga lansia. Berat dan ringannya gejala yang ditimbulkan juga berpengaruh dengan penyakit penyerta yang diderita sebelumnya seperti penyakit diabetes mellitus, penyakit jantung, paru-paru, kanker, HIV/AIDS, dll.\nBerdasarkan keluhan yang anda rasakan, seperti merupakan gejala flu pada umumnya. Apabila anda tidak memiliki riwayat 

In [ ]:
# There are some variations we can try. It searchs for the question, the answer, or both. Here we only take the questions to proceed.
sentences = [item["question"] for item in data]
sentences[0]

'dokk. saya udah dua hari ini sakit kepala dan juga pilek. pilek nya agak buntu di hidung si apakah itu gejala korona dok?'

###Ground Truth (Gold Standard)

In [ ]:
# "q" is query/question.
# "gt1" & "gt2" is indices of relevant documents of sample.
testingData = [
  {
    "q":"gejala corona", 
    "gt1":[0,1,3,8,10,13,15,16,18,23,26,38,39,42,433,44,46,50,51,57,59,63,65,66,67,73,74,75,80,86,88,95,98],
    "gt2":[0,1,3,8,10,13,15,16,18,23,26,38,39,42,43,44,45,50,51,52,57,59,63,65,66,67,70,73,74,75,80,86,88,95,98]
  },
  {
    "q":"penularan corona", 
    "gt1":[36,54,64],
    "gt2":[36,54,64]
  },
  {
    "q":"pencegahan corona", 
    "gt1":[31,34,37,77,89],
    "gt2":[20,29,34,37,77,89]
  },
  {
    "q":"isolasi mandiri", 
    "gt1":[91],
    "gt2":[91]
  },
  {
    "q":"olah raga", 
    "gt1":[30],
    "gt2":[30,]
  },
  {
    "q":"Apa gejala corona?", 
    "gt1":[0,1,18,25,26,42,44,46,51,57,73,80],
    "gt2":[0,1,3,8,10,13,15,16,18,25,26,42,46,51,52,59,63,80,88,95,98]
  },
  {
    "q":"Apa saya tertular corona?", 
    "gt1":[35,36,38,71,96],
    "gt2":[35,36,38,54,64,71]
  },
  {
    "q":"Apa yang dapat mencegah corona?", 
    "gt1":[3],
    "gt2":[3,20,29,34,37,77,89]
  },
  {
    "q":"Perlukah isolasi mandiri di rumah?", 
    "gt1":[91],
    "gt2":[91]
  },
  {
    "q":"Apakah berolahraga renang aman?", 
    "gt1":[30],
    "gt2":[30]
  },
  {
    "q":"Pilek, sakit tenggorokan. Apa itu gejala corona?", 
    "gt1":[0,1,2,8,13,15,16,38,42,51,52,57,59,66,68,95],
    "gt2":[0,1,2,3,8,13,15,26,38,42,51,52,57,59,66,68,88,95]
  },
  {
    "q":"Saya deman, dan sakit tenggorokan. Apakah saya tertular corona?", 
    "gt1":[1,2,8,10,13,15,16,18,24,36,39,41,47,55,65,86,88],
    "gt2":[1,2,8,10,13,15,16,18,24,36,39,41,47,86,88]
  },
  {
    "q":"Apakah vaksin dapat mencegah corona? setidaknya mengurangi kerusakan akibat corona?", 
    "gt1":[20,69],
    "gt2":[20,69,34]
  },
  {
    "q":"Perlukah isolasi mandiri di rumah? memperhatikan kebersihan diri?", 
    "gt1":[91],
    "gt2":[91]
  },
  {
    "q":"Apakah berolahraga renang aman? dengan penyebaran virus corona saat ini?", 
    "gt1":[30],
    "gt2":[30]
  }
]


### Cohen's Kappa

In [ ]:
sampleSet = set(range(100)) # Sample indices 0-99
for index, item in enumerate(testingData):
  # Same Relevant
  rel1Set = set(item["gt1"])
  rel2Set = set(item["gt2"])
  sameRelSet = rel1Set.intersection(rel2Set)
  # Same Irrelevant
  irrel1Set = sampleSet - rel1Set
  irrel2Set = sampleSet - rel2Set
  sameIrrelSet = irrel1Set.intersection(irrel2Set)
  # Relevant-Irrelevant
  rel1Irrel2Set = None
  # Irrelevant-Relevant
  irrel1Rel2Set = None
  # Cohen's Kappa
  total = len(sampleSet)
  PA = (len(sameRelSet)+len(sameIrrelSet))/total
  P1 = len(rel1Set)/total # (Relevant docs according to assessor 1)
  P2 = len(rel2Set)/total # (Relevant docs according to assessor 2)
  P_rel = P1 * P2 # (Probability both say relevant by chance)
  P_irel = (1-P1) * (1-P2) # (Probability both say irrelevant by chance)
  PE = P_rel + P_irel # (Probability both agree at random)
  kappa = (PA-PE)/(1-PE)

  print("Cohen's Kappa of Query %d: %.2f" % (index+1, kappa))

Cohen's Kappa of Query 1: 0.89
Cohen's Kappa of Query 2: 1.00
Cohen's Kappa of Query 3: 0.71
Cohen's Kappa of Query 4: 1.00
Cohen's Kappa of Query 5: 1.00
Cohen's Kappa of Query 6: 0.46
Cohen's Kappa of Query 7: 0.71
Cohen's Kappa of Query 8: 0.24
Cohen's Kappa of Query 9: 1.00
Cohen's Kappa of Query 10: 1.00
Cohen's Kappa of Query 11: 0.86
Cohen's Kappa of Query 12: 0.93
Cohen's Kappa of Query 13: 0.80
Cohen's Kappa of Query 14: 1.00
Cohen's Kappa of Query 15: 1.00


## Preprocessing

Preprocessing? Skip it. Let's see how good our models are without preprocessing.

BERT deals with stopwords & upper/lower case well. Don't worry about it.

Leave all of the things to the model and its tokenizer.

# B. Sentence-BERT (Using XLM-RoBERTa and IndoBERT Pre-Trained)
https://www.sbert.net

##-1. Background

Why Sentence-BERT?

BERT embeds tokens/words, but (sometimes) we need sentence (or maybe document) vector/embedding instead of word embedding. It's not a simple thing to do inferring sentence vector from word vectors in BERT. There are multiple approaches to get sentence vector in BERT. Sentence-BERT make it simpler to get sentence embedding from BERT.

##0. Installation

In [ ]:
!pip install -U sentence-transformers
!pip install sentencepiece

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.4.0)


## 1. Load the Pre-Trained Model

There are many pretrained models you can choose, based on:
- BERT variant,
- task,
- training dataset,
- and how the sentence vector is inferred from its word vectors.

See: https://www.sbert.net/docs/pretrained_models.html

Some multilingual pretrained models that (may) contain Indonesian:
1. distiluse-base-multilingual-cased-v2
2. distilbert-multilingual-nli-stsb-quora-ranking
3. xlm-r-distilroberta-base-paraphrase-v1
4. xlm-r-bert-base-nli-stsb-mean-tokens
5. xlm-r-100langs-bert-base-nli-stsb-mean-tokens
6. xlm-r-100langs-bert-base-nli-mean-tokens

Here we use "xlm-r-100langs-bert-base-nli-stsb-mean-tokens" for our trial. It's 1 GB.

Alternatively, we'll also try to load IndoBERT. It will produce warnings like these:

- Exception when trying to download https://sbert.net/models/indobenchmark/indobert-base-p2.zip. Response 404
- WARNING:root:SentenceTransformer-Model https://sbert.net/models/indobenchmark/indobert-base-p2.zip not found. Try to create it from scratch
- WARNING:root:Try to create Transformer Model indobenchmark/indobert-base-p2 with mean pooling

Then it will download something. We don't know where it comes from. Our prediction it comes from Huggingface.

In [ ]:
from sentence_transformers import SentenceTransformer, util
# pretrainedModelName = "xlm-r-100langs-bert-base-nli-stsb-mean-tokens"
pretrainedModelName = "bert-base-multilingual-cased"
# pretrainedModelName = "indobenchmark/indobert-base-p2"
# pretrainedModelName = "indolem/indobert-base-uncased"
# pretrainedModelName = "indobenchmark/indobert-lite-base-p2" # It produces error! Not suitable for AutoModel.
model = SentenceTransformer(pretrainedModelName, device="cuda")
model.max_seq_length = 500

Exception when trying to download https://sbert.net/models/bert-base-multilingual-cased.zip. Response 404


##2. Embed the Data (Convert Sentences to be Vectors)

In [ ]:
################################################
# BE CAREFUL! It's so slow! Use it only once. #
##############################################
import datetime

startTime = datetime.datetime.now()
print ("Started at", startTime.strftime("%Y-%m-%d %H:%M:%S"))

# Sentences are encoded by calling model.encode()
sentenceVectors = model.encode(sentences)

endTime = datetime.datetime.now()
print ("Finished at", endTime.strftime("%Y-%m-%d %H:%M:%S"))
print ("Executed in", (endTime-startTime).seconds/60, "minutes")

Started at 2020-12-24 09:33:09
Finished at 2020-12-24 09:33:24
Executed in 0.25 minutes


In [ ]:
# Print the embeddings. Take only 1 example. They are so many.
print("Sentence:", sentences[0])
print("Embedding:", sentenceVectors[0])

Sentence: dokk. saya udah dua hari ini sakit kepala dan juga pilek. pilek nya agak buntu di hidung si apakah itu gejala korona dok?
Embedding: [ 1.57828093e-01 -4.78938013e-01  2.06422344e-01  3.26807022e-01
  3.75199467e-01  3.41097534e-01 -5.04719794e-01  1.90881371e-01
 -8.95681828e-02  3.32380056e-01 -2.88608074e-01  1.76725805e-01
  7.60586560e-01  4.30287540e-01  2.25699730e-02  2.04017133e-01
  5.72448015e-01  2.00058922e-01  7.13353276e-01  2.39734128e-01
 -5.31742088e-02  2.17750981e-01  8.53926912e-02  4.83467393e-02
  8.05123210e-01  7.38813162e-01 -3.71061325e-01 -2.33761668e-02
 -4.00080174e-01  2.21980169e-01 -5.13286218e-02  5.80615282e-01
 -1.48287565e-01  7.19610929e-01 -2.35543683e-01 -1.76467180e-01
 -5.10779798e-01  3.71541053e-01 -8.55108947e-02 -2.34349504e-01
 -4.97527242e-01  3.84303540e-01 -2.72147246e-02  4.52946909e-02
  2.53149390e-01 -1.76852077e-01  2.24383384e-01  1.97680756e-01
 -6.11931443e-01 -2.26076335e-01 -3.87323767e-01 -8.86696056e-02
 -6.81372061

##3. Search the Q&A (Using Semantic Search/Similarity)

###3.1 Using Highest Cosine Similarity

In [ ]:
import torch

print("Hi! What Q&A are you looking for?")
query = input()
queryVector = model.encode(query)

# Compute similarity using Cosine Similarity. Alternatively, we can also use Euclidean Distance.
cosineSimilarities = util.pytorch_cos_sim(queryVector, sentenceVectors)
cosineSimilarities = cosineSimilarities[0]

# Print the result.
topN = 1
topResults = torch.topk(cosineSimilarities, k = topN)
print(f"Top-{topN} most related result:")
for score, index in zip(topResults[0], topResults[1]):
  item = data[index]
  print()
  print(f"# Q&A ke: {index+1}")
  print(f"# SCORE: {score}")
  print()
  print("# PERTANYAAN:")
  print(item["question"])
  print()
  print("# JAWABAN:")
  print(item["answer"])
  print("##################################################")

# # Index the cosine similarities before we sort it.
# indexedCosSimList = []
# maxCosSim = 0
# maxIndex = 0
# for index, cosSim in enumerate(cosineSimilarities):
#   if cosSim > maxCosSim:
#     maxIndex = index
#     maxCosSim = cosSim
#   indexedCosSimList.append([cosSim, index])
# print(maxCosSim)

# # Sort the cosine similarities descendingly.
# indexedCosSimList = sorted(indexedCosSimList, key=lambda x: x[0], reverse=True)

# # Print the result.
# topN = 1
# print(f"Top-{topN} most related results:")
# for score, index in indexedCosSimList[0:topN]:
#   item = data[index]
#   print()
#   print(f"# Q&A ke: {index+1}")
#   print(f"# SCORE: {score}")
#   print()
#   print("# PERTANYAAN:")
#   print(item["question"])
#   print()
#   print("# JAWABAN:")
#   print(item["answer"])
#   print("##################################################")

Hi! What Q&A are you looking for?
Apakah aman berenang?
Top-1 most related result:

# Q&A ke: 297
# SCORE: 0.6712941527366638

# PERTANYAAN:
Apakah berolahraga renang "aman" dikaitkan dengan penyebaran virus corona saat ini??

# JAWABAN:
Halo Kalvin,
Mengingat saat ini ada himbauan pemerintah untuk tetap di rumah dan membatasi kegiatan yang menyebabkan Anda berkunjung ke tempat umum, sebaiknya Anda menghindari untuk pergi ke kolam renang umum maupun tempat umum lain yang memungkinkan Anda berinteraksi dan berkontak dengan orang yang terinfeksi virus corona .
Kolam renang umum biasanya diberikan bahan kimia berupa klorin ataupun bahan kimia lain yang dapat membuat kolam menjadi tetap bersih dan aman dari kuman termasuk virus dan bakteri. Menurut penelitian, virus flu maupun SARS mati pada air yang diberi klorin dalam konsentrasi tertentu. Virus corona juga tidak bisa menular melalui air. Meski begitu, tidak ada jaminan Anda tidak berkontak dengan orang lain yang terinfeksi di luar kolam

Alternatively,
Instead of implementing semantic search by your self, you can use this instant function:

**util.semantic_search(query_embeddings: torch.Tensor, corpus_embeddings: torch.Tensor, query_chunk_size: int = 100, corpus_chunk_size: int = 100000, top_k: int = 10)**

https://www.sbert.net/examples/applications/semantic-search/README.html#sentence_transformers.util.semantic_search

###3.2 Using Smallest Euclidean Distance

In [ ]:
print("Hi! What Q&A are you looking for?")
query = input()
queryVector = model.encode(query)

# Compute Euclidean Distance between query & sentence vector.
pDist = torch.nn.PairwiseDistance(p=2)
indexedPDistList = []
for index, sentenceVector in enumerate(sentenceVectors):
  distance = pDist(torch.tensor([queryVector]), torch.tensor([sentenceVector]))
  indexedPDistList.append([distance, index])

# Sort the distances ascendingly.
indexedPDistList = sorted(indexedPDistList, key=lambda x: x[0], reverse=False)

# Print the result.
topN = 1
print(f"Top-{topN} most related results:")
for distance, index in indexedPDistList[0:topN]:
  item = data[index]
  print()
  print(f"# Q&A ke: {index+1}")
  print(f"# SMALLEST DISTANCE: {distance}")
  print()
  print("# PERTANYAAN:")
  print(item["question"])
  print()
  print("# JAWABAN:")
  print(item["answer"])
  print("##################################################")

Hi! What Q&A are you looking for?
Apakah aman berenang?
Top-1 most related results:

# Q&A ke: 31
# SMALLEST DISTANCE: tensor([15.4280])

# PERTANYAAN:
Apakah berolahraga renang "aman" dikaitkan dengan penyebaran virus corona saat ini??

# JAWABAN:
Halo Kalvin,
Mengingat saat ini ada himbauan pemerintah untuk tetap di rumah dan membatasi kegiatan yang menyebabkan Anda berkunjung ke tempat umum, sebaiknya Anda menghindari untuk pergi ke kolam renang umum maupun tempat umum lain yang memungkinkan Anda berinteraksi dan berkontak dengan orang yang terinfeksi virus corona .
Kolam renang umum biasanya diberikan bahan kimia berupa klorin ataupun bahan kimia lain yang dapat membuat kolam menjadi tetap bersih dan aman dari kuman termasuk virus dan bakteri. Menurut penelitian, virus flu maupun SARS mati pada air yang diberi klorin dalam konsentrasi tertentu. Virus corona juga tidak bisa menular melalui air. Meski begitu, tidak ada jaminan Anda tidak berkontak dengan orang lain yang terinfeksi di

## 4. Let's try Different Approach (Using Cross-Encoder)

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

# Now, all the models come from https://huggingface.co/models only.
# As model name, you can pass any model or path that is compatible with Huggingface AutoModel class.
model = CrossEncoder("indobenchmark/indobert-base-p2", device="cuda") # It produces some warning.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np

print("Hi! What Q&A are you looking for?")
query = input()

# Make query-sentence pairs to predict.
qsPairs = []
for sentence in sentences:
  splits = sentence.split(" ")
  splitLength = len(splits)
  if splitLength > 300:
    s = int((splitLength-300)/2)
    sentence = ""
    for i in range(s, s+300): sentence += splits[i] + " "
    
  qsPairs.append([query, sentence.strip()])

# Compute prediction.
crossScores = model.predict(qsPairs)

# Sort the scores descendingly.
sortedIndex = reversed(np.argsort(crossScores))

# Print the result.
topN = 1
print(f"Top-{topN} most related result:")
for count, dataIndex in enumerate(sortedIndex):
  item = data[dataIndex]
  print()
  print(f"# Q&A ke: {count+1}")
  print(f"# SCORE: {crossScores[dataIndex]}")
  print()
  print("# PERTANYAAN:")
  print(item["question"])
  print()
  print("# JAWABAN:")
  print(item["answer"])
  print("##################################################")
  if count+1 >= topN: break

Hi! What Q&A are you looking for?
Apakah aman berenang?
Top-1 most related result:

# Q&A ke: 1
# SCORE: 0.5173470377922058

# PERTANYAAN:
Seberapa akurat buat rapid test corona?

# JAWABAN:
Halo Ali, terimakasih sudah bertanya di Alodokter.
Rapid test Corona merupakan pemeriksaan yang berfungsi sebagai skrining untuk melihat antibodi yang terbentuk di tubuh sebagai respon dalam melawan virus Corona. Jika hasil yang didapatkan positif menandakan seseorang pernah terinfeksi virus Corona. Sedangkan jika hasil pemeriksaan adalah negatif belum tentu seseorang tidak terkena infeksi virus Corona, bisa saja hal tersebut terjadi karena antibodi dalam tubuh belum terbentuk dan pemeriksaan harus diulang dalam 7-10 hari setelah itu.
Oleh karena berfungsi sebagai pemeriksaan skrining/penyaring, rapid test tidak digunakan untuk menegakkan diagnosis COVID-19. Pemeriksaan yang digunakan dalam menegakkan diagnosis COVID-19 adalah polymerase chain reaction (PCR).
Sehingga dapat disimpulkan bahwa jika s

Feel so slow? Yes, because Cross-Encoder is slow, and it compares the query with all data. Alternatively, we can combine Semantic Search and Cross Encoder.
1. Use Semantic Search or [Elastic Searh](https://www.elastic.co/elasticsearch/) to search top 50-100 documents. (It should be faster to compare query with all documents)
2. Re-rank the top 50 documents with Cross-Encoder.

For more information about Semantic Search vs Cross-Encoder, see:
- https://www.sbert.net/examples/applications/information-retrieval/README.html
- https://www.sbert.net/examples/applications/cross-encoder/README.html

Besides Sentence-BERT, to make our life easier, we can use https://github.com/hanxiao/bert-as-service. Using BERT model as a sentence encoding service, i.e. mapping a variable-length sentence to a fixed-length vector.

# C. BERT/ALBERT (Using mBERT and IndoBERT Pre-Trained)

## -1. Background

Let's try something more chalenging. We'll try to infer sentence embedding from it's word embeddings without Sentence-BERT.

## 0. Install Tranformer

In [ ]:
!pip install transformers

## 1. Load the Pretrained Model

There are some options that contain Indonesian language. You can search here: https://huggingface.co/models?filter=bert

Here we'll use some:
1. bert-base-multilingual-cased (BERT; Most downloaded mBERT)
2. indobenchmark/indobert-base-p2 (BERT trained using 23GB dataset)
3. indobenchmark/indobert-lite-base-p2 (ALBERT trained using 23GB dataset)
4. indolem/indobert-base-uncased (BERT trained using over 220M words)
5. sarahlintang/IndoBERT (BERT with 32.000 vocabulary)
6. cahya/bert-base-indonesian-1.5G (BERT)

In [ ]:
import torch
from transformers import BertTokenizer, AutoTokenizer, AutoModel

# BERT-Base (by Google).
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
# model = AutoModel.from_pretrained("bert-base-multilingual-cased")

# BERT-Base (by Indobenchmark Team)
tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p2")

# ALBERT-Base (by Indobenchmark Team)
# tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-lite-base-p2")
# model = AutoModel.from_pretrained("indobenchmark/indobert-lite-base-p2")

# BERT-Base (by Fajri Koto)
# tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
# model = AutoModel.from_pretrained("indolem/indobert-base-uncased")

# BERT-Base (by Sarah Lintang)
# tokenizer = AutoTokenizer.from_pretrained("sarahlintang/IndoBERT")
# model = AutoModel.from_pretrained("sarahlintang/IndoBERT")

# BERT-Base (by Cahya Wirawan)
# tokenizer = AutoTokenizer.from_pretrained("cahya/bert-base-indonesian-1.5G")
# model = AutoModel.from_pretrained("cahya/bert-base-indonesian-1.5G")


## 2. Preparing Model Input Data.

In [ ]:
import datetime

startTime = datetime.datetime.now()
print ("Started at", startTime.strftime("%Y-%m-%d %H:%M:%S"))

# First, convert words to be Vocabulary IDs and other things needed by BERT as the input.
# tokenizer.encode_plus() is deprecated. Use tokenizer.__call__() instead as a shortcut to get things needed:
# input_ids, token_type_ids, and attention_mask.
modelInputs = tokenizer(sentences, padding=True, truncation=True, max_length=500, return_tensors='pt')

endTime = datetime.datetime.now()
print ("Finished at", endTime.strftime("%Y-%m-%d %H:%M:%S"))
print ("Executed in", (endTime-startTime).seconds/60, "minutes")

# Print the vocab IDs. Take only 1 example. They are so many.
print("Sentence:", sentences[0])
print("Vocab IDs", modelInputs["input_ids"][0])
print("Full Input:")
print(modelInputs)

Started at 2020-12-24 03:58:33
Finished at 2020-12-24 03:58:33
Executed in 0.0 minutes
Sentence: dokk. saya udah dua hari ini sakit kepala dan juga pilek. pilek nya agak buntu di hidung si apakah itu gejala korona dok?
Vocab IDs tensor([    2,  1197, 30361, 30470,   209,  2137,   662,   406,    92,  1252,
         1179,    41,   186, 17622, 30470, 17622,  1107,  2839, 17080,    26,
         6558,   356,   937,   137,  2962,   904,  2524,  1197, 30477,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0

Notice vocab ID 101 & 102 (in mBERT), 2 & 3 (in IndoBenchmark/IndoBERT), or 3 & 4 (in IndoLEM/IndoBERT). These are vocab IDs of special tokens [CLS] and [SEP]. BERT needs them as parts of input data.

For more details, please take a look of long explanation by Chris McCormick in the following links:
- https://colab.research.google.com/drive/1yFphU6PW9Uo6lmDly_ud9a6c4RCYlwdX
- http://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

## 3. Embed the Data

In [ ]:
################################################
# BE CAREFUL! It's so slow! Use it only once. #
##############################################
startTime = datetime.datetime.now()
print ("Started at", startTime.strftime("%Y-%m-%d %H:%M:%S"))

with torch.no_grad():
  modelOutput = model(**modelInputs)

endTime = datetime.datetime.now()
print ("Finished at", endTime.strftime("%Y-%m-%d %H:%M:%S"))
print ("Executed in", (endTime-startTime).seconds/60, "minutes")

Started at 2020-12-24 03:58:33
Finished at 2020-12-24 03:59:48
Executed in 1.25 minutes


###3.1 Alternative 1 (Using [CLS] Special Token Embedding)

The simplest way to get sentence embedding is getting vector of [CLS] Special Token in last_hidden_state. How reliable? Who knows. But for sentence classification, people say it is reliable.

The model output contains at least "last_hidden_state" and "pooler_output". You can also request complete "hidden_states" as the 3rd return element.

**last_hidden_state** length is the same as the sentence batch length. If you have 100 sentences, then last_hidden_state length is 100.

last_hidden_state[i] length is the same as the sentence length. If the sentence have 20 tokens, then last_hidden_state[i] length is 20.

last_hidden_state[i][j] length is the same as the vector dimension/hidden unit/feature number length (768).

The [CLS] vector position is in last_hidden_state[i][0].

In [ ]:
def clsVector(modelOutput):
  # lastHiddenStates = modelOutput.last_hidden_state
  lastHiddenStates = modelOutput[0]
  # Collect [CLS] vector of each sentence. It represent sentence vector.
  return [lastHiddenStates[index][0] for index in range(len(lastHiddenStates))]

In [ ]:
sentenceVectors = clsVector(modelOutput)

###3.2 Alternative 2 (Using Mean Pooling)

The second way, may be the more reliable approach, is by averaging the token vectors.

https://www.sbert.net/examples/applications/computing-embeddings/README.html#sentence-embeddings-with-transformers

In [ ]:
# Mean Pooling - Take attention mask into account for correct averaging
def meanPooling(modelOutput, attentionMask):
    # lastHiddenStates = modelOutput.last_hidden_state
    lastHiddenStates = modelOutput[0]
    inputMaskExpanded = attentionMask.unsqueeze(-1).expand(lastHiddenStates.size()).float()
    sumEmbeddings = torch.sum(lastHiddenStates * inputMaskExpanded, 1)
    sumMask = torch.clamp(inputMaskExpanded.sum(1), min=1e-9)
    return sumEmbeddings / sumMask

In [ ]:
# Perform pooling. In this case, mean pooling
sentenceVectors = meanPooling(modelOutput, modelInputs['attention_mask'])

###3.3 Other Alternatives

We can try Max Pooling or other approaches to calculate.

There are also some strategies where in the hidden states the vectors come from.
For more details, please take a look the explanation below:
- https://colab.research.google.com/drive/1yFphU6PW9Uo6lmDly_ud9a6c4RCYlwdX?authuser=1#scrollTo=orjhWUJgmxo5
- https://github.com/hanxiao/bert-as-service#q-why-not-the-last-hidden-layer-why-second-to-last

##4. Search the Q&A (Using Semantic Search/Similarity)

###4.1 Using Highest Cosine Similarity

In [ ]:
# Copied from Sentence BERT source code:
# https://github.com/UKPLab/sentence-transformers/blob/94f1e47e0cd5d8e8f4e1e91f75fe76f1bbbecc11/sentence_transformers/util.py
#
# Or you can install Sentence-BERT and import:
# from sentence_transformers import util
from torch import Tensor

def pytorch_cos_sim(a: Tensor, b: Tensor):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    This function can be used as a faster replacement for 1-scipy.spatial.distance.cdist(a,b)
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1))

In [ ]:
import torch

print("Hi! What Q&A are you looking for?")
query = input()

# Embed the query
queryModelInput = tokenizer(query, padding = "max_length", return_tensors='pt')
with torch.no_grad(): queryModelOutput = model(**queryModelInput)

# Choose one of these alternatives:
# queryVectors = clsVector(queryModelOutput) # Alternative 1
queryVector = meanPooling(queryModelOutput, queryModelInput['attention_mask']) # Alternative 2

# Compute similarity using Cosine Similarity. Alternatively, we can also use Euclidean Distance.
cosineSimilarities = pytorch_cos_sim(queryVector, sentenceVectors)
cosineSimilarities = cosineSimilarities[0]

# Print the result.
topN = 1
topResults = torch.topk(cosineSimilarities, k = topN)
print(f"Top-{topN} most related result:")
for score, index in zip(topResults[0], topResults[1]):
  item = data[index]
  print()
  print(f"# Q&A ke: {index+1}")
  print(f"# SCORE: {score}")
  print()
  print("# PERTANYAAN:")
  print(item["question"])
  print()
  print("# JAWABAN:")
  print(item["answer"])
  print("##################################################")

Hi! What Q&A are you looking for?
Apakah aman berenang?


Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


Top-1 most related result:

# Q&A ke: 31
# SCORE: 0.671294093132019

# PERTANYAAN:
Apakah berolahraga renang "aman" dikaitkan dengan penyebaran virus corona saat ini??

# JAWABAN:
Halo Kalvin,
Mengingat saat ini ada himbauan pemerintah untuk tetap di rumah dan membatasi kegiatan yang menyebabkan Anda berkunjung ke tempat umum, sebaiknya Anda menghindari untuk pergi ke kolam renang umum maupun tempat umum lain yang memungkinkan Anda berinteraksi dan berkontak dengan orang yang terinfeksi virus corona .
Kolam renang umum biasanya diberikan bahan kimia berupa klorin ataupun bahan kimia lain yang dapat membuat kolam menjadi tetap bersih dan aman dari kuman termasuk virus dan bakteri. Menurut penelitian, virus flu maupun SARS mati pada air yang diberi klorin dalam konsentrasi tertentu. Virus corona juga tidak bisa menular melalui air. Meski begitu, tidak ada jaminan Anda tidak berkontak dengan orang lain yang terinfeksi di luar kolam renang umum tersebut, baik dalam perjalanan, saat sebelum

###4.2 Using Smallest Euclidean Distance

In [ ]:
print("Hi! What Q&A are you looking for?")
query = input()

# Embed the query
queryModelInput = tokenizer(query, padding = True, return_tensors='pt')
with torch.no_grad(): queryModelOutput = model(**queryModelInput)

# Choose one of these alternatives:
# queryVectors = clsVector(queryModelOutput) # Alternative 1
queryVector = meanPooling(queryModelOutput, queryModelInput['attention_mask']) # Alternative 2

# Compute Euclidean Distance between query & sentence vector.
pDist = torch.nn.PairwiseDistance(p=2)
indexedPDistList = []
for index, sentenceVector in enumerate(sentenceVectors):
  distance = pDist(torch.tensor(queryVector), torch.tensor(sentenceVector))
  indexedPDistList.append([distance, index])

# Sort the distances ascendingly.
indexedPDistList = sorted(indexedPDistList, key=lambda x: x[0], reverse=False)

# Print the result.
topN = 1
print(f"Top-{topN} most related results:")
for distance, index in indexedPDistList[0:topN]:
  item = data[index]
  print()
  print(f"# Q&A ke: {index+1}")
  print(f"# SMALLEST DISTANCE: {distance}")
  print()
  print("# PERTANYAAN:")
  print(item["question"])
  print()
  print("# JAWABAN:")
  print(item["answer"])
  print("##################################################")

Hi! What Q&A are you looking for?
Apakah aman berenang?
Top-1 most related results:

# Q&A ke: 31
# SMALLEST DISTANCE: tensor([15.4280])

# PERTANYAAN:
Apakah berolahraga renang "aman" dikaitkan dengan penyebaran virus corona saat ini??

# JAWABAN:
Halo Kalvin,
Mengingat saat ini ada himbauan pemerintah untuk tetap di rumah dan membatasi kegiatan yang menyebabkan Anda berkunjung ke tempat umum, sebaiknya Anda menghindari untuk pergi ke kolam renang umum maupun tempat umum lain yang memungkinkan Anda berinteraksi dan berkontak dengan orang yang terinfeksi virus corona .
Kolam renang umum biasanya diberikan bahan kimia berupa klorin ataupun bahan kimia lain yang dapat membuat kolam menjadi tetap bersih dan aman dari kuman termasuk virus dan bakteri. Menurut penelitian, virus flu maupun SARS mati pada air yang diberi klorin dalam konsentrasi tertentu. Virus corona juga tidak bisa menular melalui air. Meski begitu, tidak ada jaminan Anda tidak berkontak dengan orang lain yang terinfeksi di

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


# D. System Evaluation

## 0. Testing

###Use this to chek Pretrained Model's Vocabulary

In [ ]:
# ##############
# # for SBERT #
# ############

# for item in testingData:
#   tokens = model.tokenizer.tokenize(item["q"])
#   print(tokens)

# # If the word is broken down to be some subwords, it means Out of Vocabulary (OOV).

['gejala', 'cor', '##ona']
['penularan', 'cor', '##ona']
['pencegahan', 'cor', '##ona']
['isolasi', 'mandiri']
['olah', 'raga']
['apa', 'gejala', 'cor', '##ona', '?']
['apa', 'saya', 'tertular', 'cor', '##ona', '?']
['apa', 'yang', 'dapat', 'mencegah', 'cor', '##ona', '?']
['perlu', '##kah', 'isolasi', 'mandiri', 'di', 'rumah', '?']
['apakah', 'berolahraga', 'renang', 'aman', '?']
['pil', '##ek', ',', 'sakit', 'tenggorokan', '.', 'apa', 'itu', 'gejala', 'cor', '##ona', '?']
['saya', 'dem', '##an', ',', 'dan', 'sakit', 'tenggorokan', '.', 'apakah', 'saya', 'tertular', 'cor', '##ona', '?']
['apakah', 'vaksin', 'dapat', 'mencegah', 'cor', '##ona', '?', 'setidaknya', 'mengurangi', 'kerusakan', 'akibat', 'cor', '##ona', '?']
['perlu', '##kah', 'isolasi', 'mandiri', 'di', 'rumah', '?', 'memperhatikan', 'kebersihan', 'diri', '?']
['apakah', 'berolahraga', 'renang', 'aman', '?', 'dengan', 'penyebaran', 'virus', 'cor', '##ona', 'saat', 'ini', '?']


In [ ]:
# ##############################################
# # for BERT/ALBERT from Huggingface library. #
# ############################################
# for item in testingData:
#   tokens = tokenizer.tokenize(item["q"])
#   print(tokens)

# # If the word is broken down to be some subwords, it means Out of Vocabulary (OOV).

['gejala', 'cor', '##ona']
['penularan', 'cor', '##ona']
['pencegahan', 'cor', '##ona']
['isolasi', 'mandiri']
['olah', 'raga']
['apa', 'gejala', 'cor', '##ona', '?']
['apa', 'saya', 'tertular', 'cor', '##ona', '?']
['apa', 'yang', 'dapat', 'mencegah', 'cor', '##ona', '?']
['perlu', '##kah', 'isolasi', 'mandiri', 'di', 'rumah', '?']
['apakah', 'berolahraga', 'renang', 'aman', '?']
['pil', '##ek', ',', 'sakit', 'tenggorokan', '.', 'apa', 'itu', 'gejala', 'cor', '##ona', '?']
['saya', 'dem', '##an', ',', 'dan', 'sakit', 'tenggorokan', '.', 'apakah', 'saya', 'tertular', 'cor', '##ona', '?']
['apakah', 'vaksin', 'dapat', 'mencegah', 'cor', '##ona', '?', 'setidaknya', 'mengurangi', 'kerusakan', 'akibat', 'cor', '##ona', '?']
['perlu', '##kah', 'isolasi', 'mandiri', 'di', 'rumah', '?', 'memperhatikan', 'kebersihan', 'diri', '?']
['apakah', 'berolahraga', 'renang', 'aman', '?', 'dengan', 'penyebaran', 'virus', 'cor', '##ona', 'saat', 'ini', '?']


###Use this to test Sentence-BERT

In [ ]:
# import torch

# for i, item in enumerate(testingData):
#   queryVector = model.encode(item["q"])
#   # Compute similarity using Cosine Similarity. Alternatively, we can also use Euclidean Distance.
#   cosineSimilarities = util.pytorch_cos_sim(queryVector, sentenceVectors)
#   cosineSimilarities = cosineSimilarities[0]
#   # Sort the result.
#   topResults = torch.topk(cosineSimilarities, k = 100)
#   # Print the result.
#   print("# Ground Truth (Unranked):")
#   print(f"gt{i} = {item['gt1']}")
#   # print(f"gt{i} = {item['gt2']}")
#   print("# Ranked Result:")
#   rangkedIndices = [index.item() for _, index in zip(topResults[0], topResults[1])]
#   print(f"rr{i} = {rangkedIndices}")
#   print("##################################################")

# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [63, 55, 61, 60, 94, 76, 14, 85, 6, 82, 98, 5, 7, 84, 87, 53, 0, 38, 39, 42, 3, 58, 44, 52, 30, 69, 20, 80, 95, 11, 90, 75, 2, 25, 46, 97, 99, 10, 8, 93, 54, 12, 65, 66, 89, 19, 32, 1, 57, 4, 56, 37, 9, 48, 81, 13, 70, 18, 86, 71, 51, 27, 49, 67, 59, 77, 45, 29, 33, 43, 17, 21, 91, 92, 47, 78, 64, 34, 96, 50, 15, 40, 73, 22, 24, 28, 16, 88, 62, 23, 72, 74, 26, 35, 41, 31, 36, 68, 79, 83]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [55, 94, 85, 76, 98, 19, 29, 5, 61, 3, 53, 0, 52, 60, 11, 44, 7, 84, 14, 90, 48, 71, 30, 6, 87, 25, 39, 27, 63, 15, 42, 95, 20, 51, 81, 37, 65, 93, 1, 13, 59, 49, 82, 12, 89, 54, 56, 96, 80, 50, 86, 66, 99, 64, 21, 33, 18, 45, 67, 77, 38, 78, 8, 75, 4, 91, 40, 70, 92, 36, 34, 46, 47, 62, 2, 9, 4

###Use this to test BERT/ALBERT from Huggingface

In [ ]:
# import torch
# from torch import Tensor

# # Copied from Sentence BERT source code:
# # https://github.com/UKPLab/sentence-transformers/blob/94f1e47e0cd5d8e8f4e1e91f75fe76f1bbbecc11/sentence_transformers/util.py
# def pytorch_cos_sim(a: Tensor, b: Tensor):
#     """
#     Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
#     This function can be used as a faster replacement for 1-scipy.spatial.distance.cdist(a,b)
#     :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
#     """
#     if not isinstance(a, torch.Tensor):
#         a = torch.tensor(a)

#     if not isinstance(b, torch.Tensor):
#         b = torch.tensor(b)

#     if len(a.shape) == 1:
#         a = a.unsqueeze(0)

#     if len(b.shape) == 1:
#         b = b.unsqueeze(0)

#     a_norm = a / a.norm(dim=1)[:, None]
#     b_norm = b / b.norm(dim=1)[:, None]
#     return torch.mm(a_norm, b_norm.transpose(0, 1))

# for i, item in enumerate(testingData):
#   # Embed the query
#   queryModelInput = tokenizer(item["q"], padding = True, return_tensors='pt')
#   with torch.no_grad(): queryModelOutput = model(**queryModelInput)
#   queryVector = meanPooling(queryModelOutput, queryModelInput['attention_mask'])
  
#   # Compute similarity using Cosine Similarity. Alternatively, we can also use Euclidean Distance.
#   cosineSimilarities = pytorch_cos_sim(queryVector, sentenceVectors)
#   cosineSimilarities = cosineSimilarities[0]

#   # Sort the result.
#   topResults = torch.topk(cosineSimilarities, k = 100)
#   # Print the result.
#   print("# Ground Truth (Unranked):")
#   print(f"gt{i} = {item['gt1']}")
#   # print(f"gt{i} = {item['gt2']}")
#   print("# Ranked Result:")
#   rangkedIndices = [index.item() for _, index in zip(topResults[0], topResults[1])]
#   print(f"rr{i} = {rangkedIndices}")
#   print("##################################################")


# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 63, 76, 30, 20, 5, 14, 19, 55, 29, 12, 59, 97, 34, 16, 44, 61, 18, 75, 51, 25, 48, 3, 0, 94, 7, 45, 42, 71, 95, 80, 53, 1, 47, 98, 27, 10, 60, 78, 89, 65, 52, 85, 21, 35, 39, 69, 17, 67, 13, 57, 87, 37, 64, 72, 58, 11, 66, 8, 70, 73, 4, 23, 15, 22, 36, 77, 31, 86, 40, 9, 92, 28, 56, 6, 83, 49, 88, 62, 90, 84, 41, 38, 24, 46, 26, 43, 74, 79, 99, 33, 91, 32, 68, 2, 81, 50, 96, 93, 54]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 76, 30, 63, 5, 97, 20, 14, 12, 29, 55, 61, 19, 34, 59, 7, 48, 3, 45, 58, 64, 21, 28, 25, 11, 1, 44, 78, 94, 70, 16, 22, 72, 75, 89, 42, 69, 27, 87, 53, 77, 18, 92, 71, 51, 98, 6, 37, 47, 95, 0, 31, 56, 23, 67, 80, 85, 65, 35, 60, 79, 73, 83, 39, 66, 91, 4, 36, 52, 32, 9, 99, 17, 57, 86, 62,

##1. Speed

SBERT is far faster than BERT of Huggingface. BERT is slow and crashes the RAM when processing large data. SBERT does the job very well.

### In S-BERT:
**XLM-RoBERTa** xlm-r-100langs-bert-base-nli-stsb-mean-tokens
- Downloading Pretrained Model: 1.01 GB (1 minute)
- Embeding 100 Sample Questions: 0.25 minutes (Using GPU).

**mBERT** bert-base-multilingual-cased
- Downloading Pretrained Model: 714 MB (21 seconds)
- Embeding 100 Sample Questions: 0.067 minutes (Using GPU)

**IndoBenchmark/IndoBERT** indobenchmark/indobert-base-p2
- Downloading Pretrained Model: 498 MB (33 seconds)
- Embeding 100 Sample Questions: 0.017 minutes (Using GPU)

**IndoLEM/IndoBERT** indolem/indobert-base-uncased
- Downloading Pretrained Model: 445 MB (7 seconds)
- Embeding 100 Sample Questions: 0.017 minutes (Using GPU)

### In BERT (Huggingface):
**mBERT** bert-base-multilingual-cased
- Downloading Pretrained Model: 714 MB (40 seconds)
- Embeding 100 Sample Questions: 1.717 minutes (Using GPU)

**IndoBenchmark/IndoBERT** indobenchmark/indobert-base-p2
- Downloading Pretrained Model: 498 MB
- Embeding 100 Sample Questions: 1.217 minutes

**IndoLEM/IndoBERT** indolem/indobert-base-uncased
- Downloading Pretrained Model: 445 MB
- Embeding 100 Sample Questions: 1.217 minutes

### In ALBERT (Huggingface):
**IndoBenchmark/IndoBERT** indobenchmark/indobert-lite-base-p2
- Downloading Pretrained Model: 46.7 MB (2 seconds)
- Embeding 100 Sample Questions: 1.2 minutes

##2. Precision & Recall
Testing result, 100 sampel data, 15 testing questions.

###Helper Function

In [ ]:
from statistics import mean

def evaluate():
  gtList = [gt0, gt1, gt2, gt3, gt4, gt5, gt6, gt7, gt8, gt9, gt10, gt11, gt12, gt13, gt14]
  rrList = [rr0, rr1, rr2, rr3, rr4, rr5, rr6, rr7, rr8, rr9, rr10, rr11, rr12, rr13, rr14]
  testNumber = 0; sumPrecision = 0;
  mrr_match = [];
  for gt, rr in zip(gtList, rrList):
    encounter = 0; gtLength = len(gt); precisionSum = 0;
    matchList =[]; recallList = []; precisionList = []; f1List = []
    for count, sampleIndex in enumerate(rr):
      isMatch = 0
      if sampleIndex in gt:
        isMatch = 1
        encounter += 1
        precisionSum += encounter/(count+1)
      
      recall = encounter/gtLength
      precision = encounter/(count+1)
      if precision + recall == 0: f1 = 0
      else: f1 = 2 * precision * recall / (precision + recall)

      matchList.append(isMatch)
      recallList.append(recall)
      precisionList.append(precision)
      f1List.append(f1)

    testNumber += 1
    averagePrecision = precisionSum/encounter
    sumPrecision += averagePrecision
    print(f"### Test {testNumber} ###")
    print("- Match @")
    print(matchList)
    print("- Recall @")
    print(recallList)
    print("- Precision @")
    print(precisionList)
    print("- F1 @")
    print(f1List)
    print("- Average Precision:")
    print(str(averagePrecision))
    print()

    for index, relevance in enumerate(matchList):
      if relevance == True:
         rank = 1/(index+1)
         mrr_match.append(rank)
         break
      else:
         continue

  print("### Macro Average Precision: ###")
  print(str(sumPrecision/15))
  print("### Mean Reciprocal Rank: ###")
  print(mean(mrr_match))

### SBERT xlm-r-100langs-bert-base-nli-stsb-mean-tokens.

####Ground Truth by 1st Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 433, 44, 46, 50, 51, 57, 59, 63, 65, 66, 67, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 63, 19, 76, 29, 30, 25, 78, 5, 12, 4, 14, 18, 49, 99, 51, 62, 48, 1, 3, 95, 34, 94, 53, 93, 83, 37, 60, 0, 44, 45, 7, 40, 69, 20, 22, 55, 17, 35, 10, 21, 89, 72, 8, 28, 13, 46, 9, 75, 23, 71, 61, 39, 77, 41, 65, 85, 16, 96, 42, 11, 66, 87, 98, 33, 92, 86, 97, 67, 52, 80, 70, 57, 50, 90, 15, 32, 79, 43, 31, 6, 24, 68, 84, 74, 26, 56, 36, 59, 88, 2, 54, 58, 47, 27, 81, 64, 38, 73, 91]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 29, 76, 93, 63, 19, 99, 30, 78, 25, 14, 4, 49, 62, 60, 5, 17, 12, 22, 3, 53, 18, 37, 34, 48, 45, 51, 28, 83, 10, 7, 0, 55, 1, 44, 95, 20, 69, 94, 72, 46, 89, 77, 21, 97, 32, 35, 40, 23, 61, 71, 41, 96, 39, 33, 85, 92, 8, 13, 9, 11, 98, 75, 6, 79, 70, 50, 65, 42, 66, 16, 57, 81, 86, 68, 74, 84, 87, 80, 43, 52, 26, 15, 90, 24, 2, 31, 59, 58, 54, 27, 67, 36, 88, 56, 47, 38, 64, 73, 91]
##################################################
# Ground Truth (Unranked):
gt2 = [31, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 29, 76, 19, 49, 63, 30, 99, 25, 62, 4, 78, 48, 5, 14, 12, 83, 93, 3, 18, 34, 37, 51, 1, 60, 17, 7, 20, 69, 45, 95, 53, 94, 28, 89, 22, 77, 21, 40, 0, 44, 10, 55, 46, 35, 92, 41, 72, 23, 97, 85, 61, 11, 9, 39, 79, 32, 96, 71, 66, 75, 8, 65, 50, 98, 70, 13, 58, 42, 80, 33, 86, 43, 90, 57, 24, 31, 74, 67, 84, 16, 36, 52, 87, 68, 54, 6, 26, 81, 59, 15, 27, 56, 88, 47, 38, 2, 64, 91, 73]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [17, 93, 98, 29, 60, 55, 46, 71, 28, 51, 95, 3, 27, 41, 91, 36, 18, 31, 58, 83, 99, 66, 14, 49, 59, 86, 53, 39, 24, 65, 75, 23, 34, 82, 62, 26, 32, 42, 85, 97, 74, 52, 44, 96, 16, 25, 88, 57, 37, 0, 33, 94, 8, 79, 11, 4, 10, 67, 78, 38, 89, 22, 15, 77, 48, 80, 84, 92, 90, 30, 43, 69, 21, 87, 68, 50, 9, 1, 20, 63, 19, 76, 7, 40, 54, 64, 35, 61, 70, 47, 2, 5, 81, 12, 72, 45, 73, 13, 6, 56]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [93, 22, 55, 82, 29, 17, 76, 97, 72, 99, 28, 53, 78, 60, 77, 30, 44, 63, 20, 46, 59, 2, 18, 87, 4, 10, 0, 31, 61, 84, 14, 38, 26, 19, 13, 51, 75, 23, 49, 71, 41, 35, 1, 21, 65, 69, 83, 12, 32, 3, 62, 37, 48, 27, 25, 5, 8, 16, 98, 11, 39, 7, 47, 50, 91, 9, 89, 94, 57, 42, 24, 80, 70, 95, 34, 67, 86, 90, 33, 85, 81, 58, 79, 88, 15, 92, 96, 64, 45, 40, 36, 52, 43, 54, 74, 68, 56, 66, 6, 73]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 18, 25, 26, 42, 44, 46, 51, 57, 73, 80]
# Ranked Result:
rr5 = [82, 63, 76, 19, 12, 30, 78, 4, 29, 5, 25, 14, 48, 49, 51, 18, 99, 62, 1, 53, 20, 22, 3, 0, 44, 95, 69, 94, 7, 72, 37, 45, 34, 40, 55, 60, 83, 93, 89, 17, 8, 10, 21, 35, 71, 16, 13, 23, 87, 39, 9, 97, 61, 75, 46, 28, 41, 92, 65, 42, 31, 98, 66, 11, 57, 50, 52, 96, 85, 15, 77, 67, 70, 59, 90, 86, 84, 33, 80, 32, 2, 6, 56, 68, 43, 24, 79, 36, 88, 74, 27, 81, 38, 26, 64, 47, 54, 73, 58, 91]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 71, 96]
# Ranked Result:
rr6 = [82, 63, 76, 19, 29, 12, 30, 78, 14, 5, 4, 18, 25, 53, 48, 49, 51, 99, 62, 20, 95, 44, 0, 22, 37, 1, 3, 93, 94, 7, 40, 10, 69, 60, 45, 55, 17, 72, 89, 83, 34, 8, 35, 71, 21, 16, 46, 23, 13, 39, 97, 9, 50, 87, 41, 42, 75, 61, 57, 31, 92, 11, 96, 98, 28, 65, 52, 66, 85, 70, 15, 86, 90, 84, 32, 77, 6, 68, 59, 67, 81, 74, 33, 80, 24, 2, 43, 38, 36, 79, 56, 54, 27, 88, 26, 73, 47, 64, 58, 91]
##################################################
# Ground Truth (Unranked):
gt7 = [3]
# Ranked Result:
rr7 = [82, 76, 49, 19, 29, 63, 4, 30, 5, 78, 14, 48, 99, 12, 62, 25, 51, 89, 53, 20, 3, 18, 7, 69, 37, 45, 83, 34, 95, 1, 93, 60, 17, 22, 0, 94, 44, 21, 10, 97, 55, 40, 92, 11, 72, 8, 35, 46, 61, 23, 85, 32, 9, 28, 96, 71, 70, 39, 41, 66, 75, 77, 50, 31, 79, 16, 42, 98, 57, 84, 68, 86, 59, 13, 52, 33, 90, 65, 87, 81, 15, 67, 27, 58, 74, 24, 64, 6, 80, 54, 43, 36, 91, 2, 56, 88, 38, 26, 73, 47]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [31, 23, 17, 91, 93, 51, 92, 53, 20, 87, 11, 50, 49, 48, 55, 22, 98, 72, 60, 44, 18, 71, 82, 99, 75, 4, 38, 24, 59, 42, 78, 95, 67, 69, 41, 26, 65, 46, 83, 57, 3, 37, 14, 97, 84, 58, 29, 62, 86, 94, 36, 28, 89, 8, 39, 15, 61, 30, 16, 74, 96, 34, 0, 19, 88, 63, 77, 12, 21, 7, 1, 52, 85, 43, 9, 32, 10, 54, 66, 27, 40, 79, 64, 35, 81, 5, 68, 76, 25, 45, 33, 2, 73, 90, 80, 47, 13, 56, 70, 6]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 49, 17, 37, 81, 62, 53, 76, 4, 29, 20, 48, 71, 99, 82, 93, 31, 60, 46, 57, 23, 89, 45, 61, 95, 32, 9, 77, 22, 83, 10, 51, 34, 41, 68, 67, 78, 92, 59, 69, 3, 42, 52, 21, 28, 18, 27, 15, 11, 25, 74, 14, 98, 55, 26, 39, 85, 79, 63, 75, 36, 16, 43, 80, 70, 72, 58, 38, 44, 12, 1, 86, 65, 35, 84, 8, 50, 7, 19, 96, 5, 0, 90, 24, 87, 88, 33, 2, 97, 40, 66, 73, 91, 54, 94, 13, 56, 6, 47, 64]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 8, 13, 15, 16, 38, 42, 51, 52, 57, 59, 66, 68, 95]
# Ranked Result:
rr10 = [82, 63, 14, 19, 51, 18, 25, 76, 55, 12, 44, 95, 30, 78, 29, 0, 53, 4, 48, 5, 60, 8, 99, 49, 1, 94, 62, 40, 3, 69, 34, 35, 17, 10, 20, 13, 16, 22, 21, 71, 37, 15, 7, 46, 83, 75, 85, 45, 96, 93, 52, 86, 98, 89, 9, 39, 65, 90, 42, 84, 72, 33, 23, 57, 31, 41, 59, 61, 67, 11, 50, 92, 66, 88, 87, 2, 27, 80, 47, 70, 24, 77, 28, 6, 26, 68, 36, 43, 56, 97, 74, 32, 79, 54, 73, 38, 81, 58, 64, 91]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 55, 65, 86, 88]
# Ranked Result:
rr11 = [14, 8, 95, 44, 18, 55, 51, 0, 63, 19, 53, 94, 15, 60, 82, 40, 35, 86, 25, 96, 85, 52, 13, 12, 78, 48, 16, 10, 90, 1, 30, 34, 88, 5, 75, 4, 69, 3, 71, 57, 29, 99, 9, 42, 49, 47, 27, 98, 24, 84, 62, 46, 65, 67, 33, 21, 66, 17, 68, 2, 50, 37, 11, 76, 59, 80, 36, 23, 45, 41, 74, 92, 70, 26, 56, 39, 31, 20, 6, 73, 83, 7, 89, 43, 54, 61, 87, 22, 79, 38, 93, 72, 32, 81, 64, 77, 58, 28, 91, 97]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69]
# Ranked Result:
rr12 = [49, 69, 19, 62, 48, 30, 20, 4, 78, 83, 25, 63, 12, 5, 99, 34, 89, 45, 14, 1, 82, 76, 29, 18, 51, 37, 21, 7, 94, 95, 3, 40, 53, 60, 61, 10, 92, 11, 9, 77, 85, 35, 8, 67, 23, 41, 44, 79, 46, 0, 96, 65, 42, 43, 86, 80, 75, 17, 58, 54, 66, 71, 24, 13, 90, 68, 70, 55, 22, 56, 28, 50, 32, 72, 57, 36, 39, 33, 98, 93, 15, 52, 84, 38, 87, 74, 6, 16, 97, 31, 47, 26, 81, 27, 59, 88, 64, 73, 2, 91]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [17, 31, 93, 23, 91, 83, 71, 60, 58, 32, 62, 46, 98, 49, 92, 79, 11, 37, 41, 48, 77, 99, 74, 95, 42, 29, 26, 75, 28, 65, 20, 50, 4, 55, 51, 67, 96, 24, 57, 81, 78, 3, 89, 44, 69, 84, 87, 22, 43, 53, 36, 86, 61, 38, 34, 59, 18, 85, 10, 72, 68, 33, 25, 14, 1, 30, 40, 9, 52, 21, 39, 66, 54, 70, 7, 45, 80, 0, 8, 19, 90, 15, 94, 82, 27, 88, 16, 97, 12, 73, 64, 76, 56, 63, 6, 35, 47, 2, 13, 5]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 19, 49, 62, 63, 4, 48, 83, 25, 12, 5, 1, 20, 3, 45, 69, 37, 18, 76, 78, 99, 34, 14, 7, 21, 77, 51, 40, 89, 61, 94, 82, 29, 95, 79, 53, 35, 9, 58, 10, 92, 39, 11, 85, 42, 67, 66, 56, 60, 23, 80, 41, 43, 50, 8, 81, 68, 71, 57, 54, 96, 13, 24, 70, 15, 36, 86, 28, 90, 46, 33, 22, 75, 44, 65, 74, 31, 52, 17, 6, 0, 32, 72, 84, 16, 38, 47, 26, 73, 55, 27, 98, 59, 87, 88, 64, 91, 93, 2, 97]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
- Recall @
[0.0, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.12121212121212122, 0.15151515151515152, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.18181818181818182, 0.21212121212121213, 0.24242424242424243, 0.24242424242424243, 0.24242424242424

####Ground Truth by 2nd Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 63, 19, 76, 29, 30, 25, 78, 5, 12, 4, 14, 18, 49, 99, 51, 62, 48, 1, 3, 95, 34, 94, 53, 93, 83, 37, 60, 0, 44, 45, 7, 40, 69, 20, 22, 55, 17, 35, 10, 21, 89, 72, 8, 28, 13, 46, 9, 75, 23, 71, 61, 39, 77, 41, 65, 85, 16, 96, 42, 11, 66, 87, 98, 33, 92, 86, 97, 67, 52, 80, 70, 57, 50, 90, 15, 32, 79, 43, 31, 6, 24, 68, 84, 74, 26, 56, 36, 59, 88, 2, 54, 58, 47, 27, 81, 64, 38, 73, 91]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 29, 76, 93, 63, 19, 99, 30, 78, 25, 14, 4, 49, 62, 60, 5, 17, 12, 22, 3, 53, 18, 37, 34, 48, 45, 51, 28, 83, 10, 7, 0, 55, 1, 44, 95, 20, 69, 94, 72, 46, 89, 77, 21, 97, 32, 35, 40, 23, 61, 71, 41, 96, 39, 33, 85, 92, 8, 13, 9, 11, 98, 75, 6, 79, 70, 50, 65, 42, 66, 16, 57, 81, 86, 68, 74, 84, 87, 80, 43, 52, 26, 15, 90, 24, 2, 31, 59, 58, 54, 27, 67, 36, 88, 56, 47, 38, 64, 73, 91]
##################################################
# Ground Truth (Unranked):
gt2 = [20, 29, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 29, 76, 19, 49, 63, 30, 99, 25, 62, 4, 78, 48, 5, 14, 12, 83, 93, 3, 18, 34, 37, 51, 1, 60, 17, 7, 20, 69, 45, 95, 53, 94, 28, 89, 22, 77, 21, 40, 0, 44, 10, 55, 46, 35, 92, 41, 72, 23, 97, 85, 61, 11, 9, 39, 79, 32, 96, 71, 66, 75, 8, 65, 50, 98, 70, 13, 58, 42, 80, 33, 86, 43, 90, 57, 24, 31, 74, 67, 84, 16, 36, 52, 87, 68, 54, 6, 26, 81, 59, 15, 27, 56, 88, 47, 38, 2, 64, 91, 73]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [17, 93, 98, 29, 60, 55, 46, 71, 28, 51, 95, 3, 27, 41, 91, 36, 18, 31, 58, 83, 99, 66, 14, 49, 59, 86, 53, 39, 24, 65, 75, 23, 34, 82, 62, 26, 32, 42, 85, 97, 74, 52, 44, 96, 16, 25, 88, 57, 37, 0, 33, 94, 8, 79, 11, 4, 10, 67, 78, 38, 89, 22, 15, 77, 48, 80, 84, 92, 90, 30, 43, 69, 21, 87, 68, 50, 9, 1, 20, 63, 19, 76, 7, 40, 54, 64, 35, 61, 70, 47, 2, 5, 81, 12, 72, 45, 73, 13, 6, 56]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [93, 22, 55, 82, 29, 17, 76, 97, 72, 99, 28, 53, 78, 60, 77, 30, 44, 63, 20, 46, 59, 2, 18, 87, 4, 10, 0, 31, 61, 84, 14, 38, 26, 19, 13, 51, 75, 23, 49, 71, 41, 35, 1, 21, 65, 69, 83, 12, 32, 3, 62, 37, 48, 27, 25, 5, 8, 16, 98, 11, 39, 7, 47, 50, 91, 9, 89, 94, 57, 42, 24, 80, 70, 95, 34, 67, 86, 90, 33, 85, 81, 58, 79, 88, 15, 92, 96, 64, 45, 40, 36, 52, 43, 54, 74, 68, 56, 66, 6, 73]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 25, 26, 42, 46, 51, 52, 59, 63, 80, 88, 95, 98]
# Ranked Result:
rr5 = [82, 63, 76, 19, 12, 30, 78, 4, 29, 5, 25, 14, 48, 49, 51, 18, 99, 62, 1, 53, 20, 22, 3, 0, 44, 95, 69, 94, 7, 72, 37, 45, 34, 40, 55, 60, 83, 93, 89, 17, 8, 10, 21, 35, 71, 16, 13, 23, 87, 39, 9, 97, 61, 75, 46, 28, 41, 92, 65, 42, 31, 98, 66, 11, 57, 50, 52, 96, 85, 15, 77, 67, 70, 59, 90, 86, 84, 33, 80, 32, 2, 6, 56, 68, 43, 24, 79, 36, 88, 74, 27, 81, 38, 26, 64, 47, 54, 73, 58, 91]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 54, 64, 71]
# Ranked Result:
rr6 = [82, 63, 76, 19, 29, 12, 30, 78, 14, 5, 4, 18, 25, 53, 48, 49, 51, 99, 62, 20, 95, 44, 0, 22, 37, 1, 3, 93, 94, 7, 40, 10, 69, 60, 45, 55, 17, 72, 89, 83, 34, 8, 35, 71, 21, 16, 46, 23, 13, 39, 97, 9, 50, 87, 41, 42, 75, 61, 57, 31, 92, 11, 96, 98, 28, 65, 52, 66, 85, 70, 15, 86, 90, 84, 32, 77, 6, 68, 59, 67, 81, 74, 33, 80, 24, 2, 43, 38, 36, 79, 56, 54, 27, 88, 26, 73, 47, 64, 58, 91]
##################################################
# Ground Truth (Unranked):
gt7 = [3, 20, 29, 34, 37, 77, 89]
# Ranked Result:
rr7 = [82, 76, 49, 19, 29, 63, 4, 30, 5, 78, 14, 48, 99, 12, 62, 25, 51, 89, 53, 20, 3, 18, 7, 69, 37, 45, 83, 34, 95, 1, 93, 60, 17, 22, 0, 94, 44, 21, 10, 97, 55, 40, 92, 11, 72, 8, 35, 46, 61, 23, 85, 32, 9, 28, 96, 71, 70, 39, 41, 66, 75, 77, 50, 31, 79, 16, 42, 98, 57, 84, 68, 86, 59, 13, 52, 33, 90, 65, 87, 81, 15, 67, 27, 58, 74, 24, 64, 6, 80, 54, 43, 36, 91, 2, 56, 88, 38, 26, 73, 47]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [31, 23, 17, 91, 93, 51, 92, 53, 20, 87, 11, 50, 49, 48, 55, 22, 98, 72, 60, 44, 18, 71, 82, 99, 75, 4, 38, 24, 59, 42, 78, 95, 67, 69, 41, 26, 65, 46, 83, 57, 3, 37, 14, 97, 84, 58, 29, 62, 86, 94, 36, 28, 89, 8, 39, 15, 61, 30, 16, 74, 96, 34, 0, 19, 88, 63, 77, 12, 21, 7, 1, 52, 85, 43, 9, 32, 10, 54, 66, 27, 40, 79, 64, 35, 81, 5, 68, 76, 25, 45, 33, 2, 73, 90, 80, 47, 13, 56, 70, 6]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 49, 17, 37, 81, 62, 53, 76, 4, 29, 20, 48, 71, 99, 82, 93, 31, 60, 46, 57, 23, 89, 45, 61, 95, 32, 9, 77, 22, 83, 10, 51, 34, 41, 68, 67, 78, 92, 59, 69, 3, 42, 52, 21, 28, 18, 27, 15, 11, 25, 74, 14, 98, 55, 26, 39, 85, 79, 63, 75, 36, 16, 43, 80, 70, 72, 58, 38, 44, 12, 1, 86, 65, 35, 84, 8, 50, 7, 19, 96, 5, 0, 90, 24, 87, 88, 33, 2, 97, 40, 66, 73, 91, 54, 94, 13, 56, 6, 47, 64]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 3, 8, 13, 15, 26, 38, 42, 51, 52, 57, 59, 66, 68, 88, 95]
# Ranked Result:
rr10 = [82, 63, 14, 19, 51, 18, 25, 76, 55, 12, 44, 95, 30, 78, 29, 0, 53, 4, 48, 5, 60, 8, 99, 49, 1, 94, 62, 40, 3, 69, 34, 35, 17, 10, 20, 13, 16, 22, 21, 71, 37, 15, 7, 46, 83, 75, 85, 45, 96, 93, 52, 86, 98, 89, 9, 39, 65, 90, 42, 84, 72, 33, 23, 57, 31, 41, 59, 61, 67, 11, 50, 92, 66, 88, 87, 2, 27, 80, 47, 70, 24, 77, 28, 6, 26, 68, 36, 43, 56, 97, 74, 32, 79, 54, 73, 38, 81, 58, 64, 91]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 86, 88]
# Ranked Result:
rr11 = [14, 8, 95, 44, 18, 55, 51, 0, 63, 19, 53, 94, 15, 60, 82, 40, 35, 86, 25, 96, 85, 52, 13, 12, 78, 48, 16, 10, 90, 1, 30, 34, 88, 5, 75, 4, 69, 3, 71, 57, 29, 99, 9, 42, 49, 47, 27, 98, 24, 84, 62, 46, 65, 67, 33, 21, 66, 17, 68, 2, 50, 37, 11, 76, 59, 80, 36, 23, 45, 41, 74, 92, 70, 26, 56, 39, 31, 20, 6, 73, 83, 7, 89, 43, 54, 61, 87, 22, 79, 38, 93, 72, 32, 81, 64, 77, 58, 28, 91, 97]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69, 34]
# Ranked Result:
rr12 = [49, 69, 19, 62, 48, 30, 20, 4, 78, 83, 25, 63, 12, 5, 99, 34, 89, 45, 14, 1, 82, 76, 29, 18, 51, 37, 21, 7, 94, 95, 3, 40, 53, 60, 61, 10, 92, 11, 9, 77, 85, 35, 8, 67, 23, 41, 44, 79, 46, 0, 96, 65, 42, 43, 86, 80, 75, 17, 58, 54, 66, 71, 24, 13, 90, 68, 70, 55, 22, 56, 28, 50, 32, 72, 57, 36, 39, 33, 98, 93, 15, 52, 84, 38, 87, 74, 6, 16, 97, 31, 47, 26, 81, 27, 59, 88, 64, 73, 2, 91]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [17, 31, 93, 23, 91, 83, 71, 60, 58, 32, 62, 46, 98, 49, 92, 79, 11, 37, 41, 48, 77, 99, 74, 95, 42, 29, 26, 75, 28, 65, 20, 50, 4, 55, 51, 67, 96, 24, 57, 81, 78, 3, 89, 44, 69, 84, 87, 22, 43, 53, 36, 86, 61, 38, 34, 59, 18, 85, 10, 72, 68, 33, 25, 14, 1, 30, 40, 9, 52, 21, 39, 66, 54, 70, 7, 45, 80, 0, 8, 19, 90, 15, 94, 82, 27, 88, 16, 97, 12, 73, 64, 76, 56, 63, 6, 35, 47, 2, 13, 5]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 19, 49, 62, 63, 4, 48, 83, 25, 12, 5, 1, 20, 3, 45, 69, 37, 18, 76, 78, 99, 34, 14, 7, 21, 77, 51, 40, 89, 61, 94, 82, 29, 95, 79, 53, 35, 9, 58, 10, 92, 39, 11, 85, 42, 67, 66, 56, 60, 23, 80, 41, 43, 50, 8, 81, 68, 71, 57, 54, 96, 13, 24, 70, 15, 36, 86, 28, 90, 46, 33, 22, 75, 44, 65, 74, 31, 52, 17, 6, 0, 32, 72, 84, 16, 38, 47, 26, 73, 55, 27, 98, 59, 87, 88, 64, 91, 93, 2, 97]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
- Recall @
[0.0, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.11428571428571428, 0.14285714285714285, 0.17142857142857143, 0.17142857142857143, 0.17142857142857143, 0.17142857142857143, 0.17142857142857143, 0.17142857142857143, 0.17142857142857143, 0.17142857142857143, 0.2, 0.22857142857142856, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.25

###SBERT bert-base-multilingual-cased

####Ground Truth by 1st Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 433, 44, 46, 50, 51, 57, 59, 63, 65, 66, 67, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [63, 55, 61, 60, 94, 76, 14, 85, 6, 82, 98, 5, 7, 84, 87, 53, 0, 38, 39, 42, 3, 58, 44, 52, 30, 69, 20, 80, 95, 11, 90, 75, 2, 25, 46, 97, 99, 10, 8, 93, 54, 12, 65, 66, 89, 19, 32, 1, 57, 4, 56, 37, 9, 48, 81, 13, 70, 18, 86, 71, 51, 27, 49, 67, 59, 77, 45, 29, 33, 43, 17, 21, 91, 92, 47, 78, 64, 34, 96, 50, 15, 40, 73, 22, 24, 28, 16, 88, 62, 23, 72, 74, 26, 35, 41, 31, 36, 68, 79, 83]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [55, 94, 85, 76, 98, 19, 29, 5, 61, 3, 53, 0, 52, 60, 11, 44, 7, 84, 14, 90, 48, 71, 30, 6, 87, 25, 39, 27, 63, 15, 42, 95, 20, 51, 81, 37, 65, 93, 1, 13, 59, 49, 82, 12, 89, 54, 56, 96, 80, 50, 86, 66, 99, 64, 21, 33, 18, 45, 67, 77, 38, 78, 8, 75, 4, 91, 40, 70, 92, 36, 34, 46, 47, 62, 2, 9, 43, 58, 32, 24, 41, 17, 28, 57, 10, 68, 16, 88, 83, 97, 69, 26, 22, 74, 79, 31, 73, 35, 72, 23]
##################################################
# Ground Truth (Unranked):
gt2 = [31, 34, 37, 77, 89]
# Ranked Result:
rr2 = [49, 5, 45, 20, 29, 77, 69, 89, 30, 7, 25, 61, 34, 37, 3, 11, 94, 21, 64, 1, 19, 91, 85, 76, 63, 53, 62, 70, 12, 58, 51, 82, 9, 66, 65, 80, 55, 83, 4, 92, 78, 60, 98, 87, 48, 95, 28, 8, 32, 96, 75, 43, 10, 93, 72, 99, 71, 41, 67, 39, 35, 59, 42, 44, 79, 74, 56, 24, 38, 84, 47, 97, 13, 36, 86, 14, 57, 81, 15, 18, 52, 16, 0, 33, 73, 23, 6, 68, 54, 27, 22, 50, 88, 17, 40, 26, 46, 90, 2, 31]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [14, 55, 39, 91, 48, 5, 66, 25, 71, 30, 98, 83, 11, 10, 90, 96, 26, 12, 33, 41, 45, 0, 82, 51, 60, 99, 24, 80, 78, 74, 85, 42, 7, 40, 1, 21, 43, 8, 94, 20, 92, 17, 9, 79, 75, 61, 50, 52, 62, 68, 36, 19, 53, 44, 46, 63, 70, 49, 58, 81, 95, 65, 59, 76, 64, 32, 89, 4, 13, 69, 87, 56, 88, 38, 93, 6, 15, 2, 37, 73, 31, 16, 22, 29, 27, 23, 3, 84, 57, 54, 72, 34, 18, 47, 77, 86, 28, 67, 97, 35]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [55, 84, 98, 94, 14, 90, 60, 17, 71, 39, 80, 87, 99, 76, 24, 52, 40, 11, 85, 6, 70, 42, 7, 19, 58, 66, 2, 50, 82, 30, 25, 5, 51, 61, 3, 89, 20, 53, 92, 96, 54, 46, 31, 47, 91, 33, 12, 95, 56, 44, 45, 38, 48, 10, 41, 75, 81, 78, 62, 88, 29, 68, 37, 65, 21, 64, 59, 73, 8, 1, 0, 15, 43, 86, 13, 27, 97, 22, 74, 26, 77, 9, 36, 4, 28, 93, 16, 57, 23, 69, 34, 79, 72, 63, 32, 83, 49, 18, 67, 35]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 18, 25, 26, 42, 44, 46, 51, 57, 73, 80]
# Ranked Result:
rr5 = [82, 76, 97, 14, 63, 12, 30, 44, 78, 17, 22, 7, 48, 55, 89, 72, 53, 51, 20, 4, 69, 10, 0, 18, 62, 37, 93, 87, 19, 71, 95, 29, 60, 42, 13, 94, 98, 58, 52, 11, 83, 8, 34, 66, 21, 25, 27, 15, 6, 45, 38, 67, 16, 59, 40, 80, 77, 57, 75, 23, 81, 46, 1, 32, 90, 24, 65, 9, 35, 92, 61, 41, 73, 85, 54, 39, 28, 99, 5, 31, 47, 68, 70, 64, 86, 96, 84, 26, 88, 50, 74, 36, 91, 79, 2, 43, 3, 56, 33, 49]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 71, 96]
# Ranked Result:
rr6 = [82, 7, 14, 44, 48, 51, 63, 71, 76, 10, 11, 53, 81, 19, 97, 12, 89, 22, 17, 55, 95, 30, 78, 0, 4, 52, 60, 18, 40, 87, 29, 31, 90, 85, 94, 98, 68, 37, 13, 58, 36, 8, 27, 93, 24, 15, 16, 84, 75, 6, 72, 23, 92, 83, 86, 32, 25, 42, 62, 65, 66, 39, 41, 67, 77, 54, 20, 38, 91, 50, 88, 26, 57, 73, 28, 70, 1, 80, 59, 56, 35, 74, 47, 45, 96, 69, 99, 2, 34, 46, 21, 3, 61, 79, 9, 43, 64, 33, 5, 49]
##################################################
# Ground Truth (Unranked):
gt7 = [3]
# Ranked Result:
rr7 = [82, 20, 89, 69, 78, 12, 30, 62, 45, 7, 29, 37, 49, 48, 63, 97, 83, 4, 72, 77, 5, 76, 34, 21, 11, 61, 58, 51, 19, 10, 22, 66, 91, 32, 55, 25, 44, 1, 94, 14, 18, 71, 17, 87, 16, 80, 23, 92, 9, 8, 64, 95, 13, 53, 41, 79, 70, 93, 67, 68, 74, 15, 36, 24, 65, 28, 96, 75, 42, 40, 99, 57, 81, 35, 3, 73, 60, 43, 47, 86, 0, 98, 26, 59, 27, 56, 54, 88, 52, 38, 39, 31, 50, 6, 85, 84, 46, 33, 90, 2]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 82, 7, 72, 89, 51, 48, 78, 12, 97, 63, 83, 76, 11, 71, 62, 22, 4, 91, 92, 10, 45, 20, 14, 69, 44, 77, 93, 17, 29, 32, 37, 58, 87, 19, 25, 36, 21, 8, 55, 24, 23, 53, 65, 34, 64, 75, 41, 40, 26, 68, 79, 16, 81, 96, 70, 60, 1, 66, 15, 31, 9, 74, 99, 43, 57, 95, 52, 80, 28, 0, 27, 18, 73, 39, 98, 50, 13, 59, 61, 5, 46, 6, 3, 88, 56, 49, 67, 85, 94, 54, 38, 42, 47, 84, 33, 86, 90, 35, 2]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 17, 7, 14, 55, 84, 51, 19, 60, 92, 22, 72, 71, 87, 11, 48, 93, 44, 82, 46, 53, 99, 81, 89, 29, 0, 31, 50, 76, 39, 27, 26, 91, 88, 58, 4, 24, 59, 97, 8, 78, 83, 41, 63, 12, 23, 40, 32, 10, 6, 62, 36, 52, 37, 28, 56, 68, 16, 77, 90, 75, 95, 98, 79, 45, 15, 54, 20, 1, 85, 80, 69, 34, 33, 73, 70, 96, 43, 74, 42, 21, 64, 18, 47, 9, 3, 65, 94, 66, 25, 57, 38, 86, 13, 2, 61, 67, 5, 35, 49]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 8, 13, 15, 16, 38, 42, 51, 52, 57, 59, 66, 68, 95]
# Ranked Result:
rr10 = [51, 16, 95, 15, 18, 71, 44, 13, 10, 4, 78, 12, 17, 8, 57, 52, 73, 19, 48, 66, 63, 68, 65, 55, 7, 75, 0, 53, 31, 24, 87, 25, 37, 98, 60, 67, 39, 14, 30, 94, 11, 23, 58, 81, 36, 86, 26, 88, 59, 22, 76, 9, 72, 34, 83, 1, 6, 90, 35, 77, 3, 92, 85, 41, 20, 93, 62, 29, 45, 89, 40, 96, 21, 27, 80, 32, 82, 42, 79, 46, 74, 50, 64, 38, 97, 56, 84, 28, 61, 43, 69, 54, 47, 70, 5, 2, 99, 91, 33, 49]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 55, 65, 86, 88]
# Ranked Result:
rr11 = [95, 10, 71, 16, 15, 4, 18, 51, 13, 44, 66, 24, 87, 94, 12, 7, 23, 78, 8, 65, 68, 52, 25, 31, 19, 57, 17, 73, 36, 98, 26, 35, 11, 75, 55, 48, 81, 67, 37, 90, 83, 53, 39, 88, 40, 58, 63, 86, 76, 6, 3, 84, 0, 92, 30, 9, 60, 1, 89, 85, 41, 42, 34, 14, 62, 59, 80, 20, 93, 96, 45, 32, 56, 72, 27, 77, 47, 50, 74, 21, 5, 38, 61, 2, 79, 70, 64, 54, 22, 29, 46, 43, 97, 82, 91, 28, 69, 99, 49, 33]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69]
# Ranked Result:
rr12 = [69, 78, 20, 49, 12, 62, 45, 37, 30, 4, 83, 72, 89, 48, 77, 66, 7, 29, 5, 34, 91, 63, 25, 82, 11, 21, 61, 19, 92, 51, 58, 94, 32, 71, 53, 17, 76, 1, 97, 9, 64, 44, 10, 93, 22, 13, 8, 65, 75, 55, 67, 16, 3, 18, 36, 87, 79, 96, 15, 14, 41, 95, 80, 24, 68, 70, 60, 40, 43, 99, 74, 81, 23, 26, 28, 57, 98, 86, 0, 54, 27, 42, 39, 50, 73, 59, 47, 85, 88, 56, 84, 52, 6, 35, 46, 38, 31, 90, 33, 2]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [91, 92, 83, 17, 71, 78, 24, 30, 36, 72, 11, 4, 93, 62, 12, 77, 45, 75, 8, 69, 51, 66, 48, 32, 41, 99, 58, 79, 37, 26, 7, 87, 10, 25, 39, 19, 3, 64, 23, 88, 43, 34, 53, 22, 16, 70, 68, 20, 44, 65, 50, 55, 96, 63, 40, 1, 52, 81, 21, 98, 9, 29, 84, 74, 89, 13, 95, 59, 57, 15, 60, 80, 56, 94, 28, 27, 76, 18, 6, 73, 54, 0, 46, 85, 49, 67, 14, 31, 97, 5, 42, 47, 61, 35, 33, 82, 86, 90, 38, 2]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 12, 62, 83, 78, 19, 72, 11, 7, 45, 20, 89, 22, 92, 48, 25, 77, 51, 69, 4, 32, 58, 93, 29, 91, 17, 56, 9, 37, 66, 21, 63, 94, 8, 87, 75, 1, 44, 53, 36, 34, 3, 60, 50, 10, 64, 41, 99, 14, 61, 79, 81, 24, 15, 86, 39, 5, 88, 0, 26, 42, 6, 43, 54, 52, 65, 55, 76, 18, 28, 40, 27, 67, 13, 59, 68, 80, 74, 71, 96, 47, 73, 70, 82, 95, 23, 57, 38, 84, 31, 16, 97, 85, 90, 49, 98, 35, 46, 33, 2]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
- Recall @
[0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.09090909090909091, 0.12121212121212122, 0.15151515151515152, 0.18181818181818182, 0.21212121212121213, 0.21212121212121213, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.2727272727272727, 0.30303030303030304, 0.30303030303030304, 0.30303030303030304, 0.

####Ground Truth by 2nd Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [63, 55, 61, 60, 94, 76, 14, 85, 6, 82, 98, 5, 7, 84, 87, 53, 0, 38, 39, 42, 3, 58, 44, 52, 30, 69, 20, 80, 95, 11, 90, 75, 2, 25, 46, 97, 99, 10, 8, 93, 54, 12, 65, 66, 89, 19, 32, 1, 57, 4, 56, 37, 9, 48, 81, 13, 70, 18, 86, 71, 51, 27, 49, 67, 59, 77, 45, 29, 33, 43, 17, 21, 91, 92, 47, 78, 64, 34, 96, 50, 15, 40, 73, 22, 24, 28, 16, 88, 62, 23, 72, 74, 26, 35, 41, 31, 36, 68, 79, 83]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [55, 94, 85, 76, 98, 19, 29, 5, 61, 3, 53, 0, 52, 60, 11, 44, 7, 84, 14, 90, 48, 71, 30, 6, 87, 25, 39, 27, 63, 15, 42, 95, 20, 51, 81, 37, 65, 93, 1, 13, 59, 49, 82, 12, 89, 54, 56, 96, 80, 50, 86, 66, 99, 64, 21, 33, 18, 45, 67, 77, 38, 78, 8, 75, 4, 91, 40, 70, 92, 36, 34, 46, 47, 62, 2, 9, 43, 58, 32, 24, 41, 17, 28, 57, 10, 68, 16, 88, 83, 97, 69, 26, 22, 74, 79, 31, 73, 35, 72, 23]
##################################################
# Ground Truth (Unranked):
gt2 = [20, 29, 34, 37, 77, 89]
# Ranked Result:
rr2 = [49, 5, 45, 20, 29, 77, 69, 89, 30, 7, 25, 61, 34, 37, 3, 11, 94, 21, 64, 1, 19, 91, 85, 76, 63, 53, 62, 70, 12, 58, 51, 82, 9, 66, 65, 80, 55, 83, 4, 92, 78, 60, 98, 87, 48, 95, 28, 8, 32, 96, 75, 43, 10, 93, 72, 99, 71, 41, 67, 39, 35, 59, 42, 44, 79, 74, 56, 24, 38, 84, 47, 97, 13, 36, 86, 14, 57, 81, 15, 18, 52, 16, 0, 33, 73, 23, 6, 68, 54, 27, 22, 50, 88, 17, 40, 26, 46, 90, 2, 31]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [14, 55, 39, 91, 48, 5, 66, 25, 71, 30, 98, 83, 11, 10, 90, 96, 26, 12, 33, 41, 45, 0, 82, 51, 60, 99, 24, 80, 78, 74, 85, 42, 7, 40, 1, 21, 43, 8, 94, 20, 92, 17, 9, 79, 75, 61, 50, 52, 62, 68, 36, 19, 53, 44, 46, 63, 70, 49, 58, 81, 95, 65, 59, 76, 64, 32, 89, 4, 13, 69, 87, 56, 88, 38, 93, 6, 15, 2, 37, 73, 31, 16, 22, 29, 27, 23, 3, 84, 57, 54, 72, 34, 18, 47, 77, 86, 28, 67, 97, 35]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [55, 84, 98, 94, 14, 90, 60, 17, 71, 39, 80, 87, 99, 76, 24, 52, 40, 11, 85, 6, 70, 42, 7, 19, 58, 66, 2, 50, 82, 30, 25, 5, 51, 61, 3, 89, 20, 53, 92, 96, 54, 46, 31, 47, 91, 33, 12, 95, 56, 44, 45, 38, 48, 10, 41, 75, 81, 78, 62, 88, 29, 68, 37, 65, 21, 64, 59, 73, 8, 1, 0, 15, 43, 86, 13, 27, 97, 22, 74, 26, 77, 9, 36, 4, 28, 93, 16, 57, 23, 69, 34, 79, 72, 63, 32, 83, 49, 18, 67, 35]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 25, 26, 42, 46, 51, 52, 59, 63, 80, 88, 95, 98]
# Ranked Result:
rr5 = [82, 76, 97, 14, 63, 12, 30, 44, 78, 17, 22, 7, 48, 55, 89, 72, 53, 51, 20, 4, 69, 10, 0, 18, 62, 37, 93, 87, 19, 71, 95, 29, 60, 42, 13, 94, 98, 58, 52, 11, 83, 8, 34, 66, 21, 25, 27, 15, 6, 45, 38, 67, 16, 59, 40, 80, 77, 57, 75, 23, 81, 46, 1, 32, 90, 24, 65, 9, 35, 92, 61, 41, 73, 85, 54, 39, 28, 99, 5, 31, 47, 68, 70, 64, 86, 96, 84, 26, 88, 50, 74, 36, 91, 79, 2, 43, 3, 56, 33, 49]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 54, 64, 71]
# Ranked Result:
rr6 = [82, 7, 14, 44, 48, 51, 63, 71, 76, 10, 11, 53, 81, 19, 97, 12, 89, 22, 17, 55, 95, 30, 78, 0, 4, 52, 60, 18, 40, 87, 29, 31, 90, 85, 94, 98, 68, 37, 13, 58, 36, 8, 27, 93, 24, 15, 16, 84, 75, 6, 72, 23, 92, 83, 86, 32, 25, 42, 62, 65, 66, 39, 41, 67, 77, 54, 20, 38, 91, 50, 88, 26, 57, 73, 28, 70, 1, 80, 59, 56, 35, 74, 47, 45, 96, 69, 99, 2, 34, 46, 21, 3, 61, 79, 9, 43, 64, 33, 5, 49]
##################################################
# Ground Truth (Unranked):
gt7 = [3, 20, 29, 34, 37, 77, 89]
# Ranked Result:
rr7 = [82, 20, 89, 69, 78, 12, 30, 62, 45, 7, 29, 37, 49, 48, 63, 97, 83, 4, 72, 77, 5, 76, 34, 21, 11, 61, 58, 51, 19, 10, 22, 66, 91, 32, 55, 25, 44, 1, 94, 14, 18, 71, 17, 87, 16, 80, 23, 92, 9, 8, 64, 95, 13, 53, 41, 79, 70, 93, 67, 68, 74, 15, 36, 24, 65, 28, 96, 75, 42, 40, 99, 57, 81, 35, 3, 73, 60, 43, 47, 86, 0, 98, 26, 59, 27, 56, 54, 88, 52, 38, 39, 31, 50, 6, 85, 84, 46, 33, 90, 2]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 82, 7, 72, 89, 51, 48, 78, 12, 97, 63, 83, 76, 11, 71, 62, 22, 4, 91, 92, 10, 45, 20, 14, 69, 44, 77, 93, 17, 29, 32, 37, 58, 87, 19, 25, 36, 21, 8, 55, 24, 23, 53, 65, 34, 64, 75, 41, 40, 26, 68, 79, 16, 81, 96, 70, 60, 1, 66, 15, 31, 9, 74, 99, 43, 57, 95, 52, 80, 28, 0, 27, 18, 73, 39, 98, 50, 13, 59, 61, 5, 46, 6, 3, 88, 56, 49, 67, 85, 94, 54, 38, 42, 47, 84, 33, 86, 90, 35, 2]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 17, 7, 14, 55, 84, 51, 19, 60, 92, 22, 72, 71, 87, 11, 48, 93, 44, 82, 46, 53, 99, 81, 89, 29, 0, 31, 50, 76, 39, 27, 26, 91, 88, 58, 4, 24, 59, 97, 8, 78, 83, 41, 63, 12, 23, 40, 32, 10, 6, 62, 36, 52, 37, 28, 56, 68, 16, 77, 90, 75, 95, 98, 79, 45, 15, 54, 20, 1, 85, 80, 69, 34, 33, 73, 70, 96, 43, 74, 42, 21, 64, 18, 47, 9, 3, 65, 94, 66, 25, 57, 38, 86, 13, 2, 61, 67, 5, 35, 49]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 3, 8, 13, 15, 26, 38, 42, 51, 52, 57, 59, 66, 68, 88, 95]
# Ranked Result:
rr10 = [51, 16, 95, 15, 18, 71, 44, 13, 10, 4, 78, 12, 17, 8, 57, 52, 73, 19, 48, 66, 63, 68, 65, 55, 7, 75, 0, 53, 31, 24, 87, 25, 37, 98, 60, 67, 39, 14, 30, 94, 11, 23, 58, 81, 36, 86, 26, 88, 59, 22, 76, 9, 72, 34, 83, 1, 6, 90, 35, 77, 3, 92, 85, 41, 20, 93, 62, 29, 45, 89, 40, 96, 21, 27, 80, 32, 82, 42, 79, 46, 74, 50, 64, 38, 97, 56, 84, 28, 61, 43, 69, 54, 47, 70, 5, 2, 99, 91, 33, 49]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 86, 88]
# Ranked Result:
rr11 = [95, 10, 71, 16, 15, 4, 18, 51, 13, 44, 66, 24, 87, 94, 12, 7, 23, 78, 8, 65, 68, 52, 25, 31, 19, 57, 17, 73, 36, 98, 26, 35, 11, 75, 55, 48, 81, 67, 37, 90, 83, 53, 39, 88, 40, 58, 63, 86, 76, 6, 3, 84, 0, 92, 30, 9, 60, 1, 89, 85, 41, 42, 34, 14, 62, 59, 80, 20, 93, 96, 45, 32, 56, 72, 27, 77, 47, 50, 74, 21, 5, 38, 61, 2, 79, 70, 64, 54, 22, 29, 46, 43, 97, 82, 91, 28, 69, 99, 49, 33]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69, 34]
# Ranked Result:
rr12 = [69, 78, 20, 49, 12, 62, 45, 37, 30, 4, 83, 72, 89, 48, 77, 66, 7, 29, 5, 34, 91, 63, 25, 82, 11, 21, 61, 19, 92, 51, 58, 94, 32, 71, 53, 17, 76, 1, 97, 9, 64, 44, 10, 93, 22, 13, 8, 65, 75, 55, 67, 16, 3, 18, 36, 87, 79, 96, 15, 14, 41, 95, 80, 24, 68, 70, 60, 40, 43, 99, 74, 81, 23, 26, 28, 57, 98, 86, 0, 54, 27, 42, 39, 50, 73, 59, 47, 85, 88, 56, 84, 52, 6, 35, 46, 38, 31, 90, 33, 2]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [91, 92, 83, 17, 71, 78, 24, 30, 36, 72, 11, 4, 93, 62, 12, 77, 45, 75, 8, 69, 51, 66, 48, 32, 41, 99, 58, 79, 37, 26, 7, 87, 10, 25, 39, 19, 3, 64, 23, 88, 43, 34, 53, 22, 16, 70, 68, 20, 44, 65, 50, 55, 96, 63, 40, 1, 52, 81, 21, 98, 9, 29, 84, 74, 89, 13, 95, 59, 57, 15, 60, 80, 56, 94, 28, 27, 76, 18, 6, 73, 54, 0, 46, 85, 49, 67, 14, 31, 97, 5, 42, 47, 61, 35, 33, 82, 86, 90, 38, 2]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 12, 62, 83, 78, 19, 72, 11, 7, 45, 20, 89, 22, 92, 48, 25, 77, 51, 69, 4, 32, 58, 93, 29, 91, 17, 56, 9, 37, 66, 21, 63, 94, 8, 87, 75, 1, 44, 53, 36, 34, 3, 60, 50, 10, 64, 41, 99, 14, 61, 79, 81, 24, 15, 86, 39, 5, 88, 0, 26, 42, 6, 43, 54, 52, 65, 55, 76, 18, 28, 40, 27, 67, 13, 59, 68, 80, 74, 71, 96, 47, 73, 70, 82, 95, 23, 57, 38, 84, 31, 16, 97, 85, 90, 49, 98, 35, 46, 33, 2]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
- Recall @
[0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.08571428571428572, 0.11428571428571428, 0.14285714285714285, 0.17142857142857143, 0.2, 0.2, 0.22857142857142856, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2857142857142857, 0.3142857142857143, 0.3142857142857143, 0.3142857142857143, 0.34285714285714286, 0.34285714285714286, 0.3428571

###SBERT indobenchmark/indobert-base-p2

####Ground Truth by 1st Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 433, 44, 46, 50, 51, 57, 59, 63, 65, 66, 67, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 76, 63, 14, 30, 51, 0, 97, 53, 19, 16, 57, 5, 12, 34, 95, 27, 80, 78, 1, 59, 55, 3, 52, 44, 49, 25, 18, 48, 98, 28, 45, 4, 8, 66, 7, 21, 15, 64, 20, 42, 75, 17, 94, 71, 40, 37, 13, 10, 60, 69, 88, 85, 38, 29, 11, 46, 26, 50, 67, 36, 65, 96, 90, 47, 70, 86, 35, 77, 31, 58, 39, 6, 73, 91, 33, 84, 41, 61, 56, 2, 99, 9, 22, 79, 74, 87, 92, 23, 83, 24, 62, 89, 68, 72, 81, 54, 43, 93, 32]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 76, 30, 63, 14, 97, 34, 5, 21, 53, 12, 51, 48, 19, 4, 78, 0, 45, 7, 29, 28, 27, 1, 49, 66, 64, 80, 58, 11, 59, 57, 20, 25, 77, 60, 17, 99, 6, 69, 50, 16, 37, 70, 22, 92, 91, 46, 95, 18, 62, 40, 36, 52, 85, 98, 3, 94, 55, 42, 44, 83, 56, 88, 79, 8, 71, 72, 96, 32, 81, 35, 67, 38, 31, 13, 23, 15, 47, 9, 87, 75, 74, 89, 33, 84, 39, 24, 86, 90, 41, 2, 73, 65, 93, 61, 10, 26, 68, 54, 43]
##################################################
# Ground Truth (Unranked):
gt2 = [31, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 76, 30, 63, 14, 34, 97, 21, 5, 4, 51, 19, 53, 28, 45, 12, 48, 78, 0, 49, 1, 66, 29, 20, 58, 7, 11, 77, 80, 64, 27, 99, 69, 57, 59, 25, 92, 91, 70, 16, 3, 50, 17, 37, 22, 6, 60, 18, 95, 62, 36, 55, 94, 44, 83, 46, 42, 52, 40, 85, 8, 71, 98, 79, 67, 88, 32, 72, 96, 23, 56, 75, 35, 81, 9, 15, 13, 47, 74, 87, 39, 38, 31, 89, 24, 86, 73, 93, 41, 84, 33, 10, 65, 2, 61, 26, 90, 68, 54, 43]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 76, 97, 30, 64, 28, 91, 63, 14, 20, 58, 53, 21, 59, 51, 80, 55, 12, 11, 19, 3, 1, 50, 34, 78, 45, 70, 92, 10, 99, 60, 71, 69, 66, 48, 77, 0, 73, 38, 27, 65, 36, 5, 67, 17, 57, 56, 4, 42, 95, 46, 44, 75, 32, 9, 98, 96, 22, 16, 86, 74, 31, 47, 23, 79, 83, 6, 15, 85, 72, 52, 61, 24, 87, 25, 88, 8, 35, 29, 33, 40, 7, 90, 81, 94, 37, 49, 26, 18, 43, 54, 39, 68, 89, 13, 2, 93, 41, 62, 84]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 30, 76, 97, 14, 60, 63, 45, 27, 49, 51, 3, 21, 55, 59, 64, 58, 61, 91, 53, 5, 70, 20, 17, 69, 73, 38, 89, 80, 19, 36, 33, 75, 66, 77, 79, 12, 1, 74, 42, 85, 87, 16, 71, 35, 0, 99, 28, 47, 10, 86, 98, 84, 8, 81, 4, 22, 11, 67, 9, 65, 40, 95, 18, 34, 31, 44, 6, 57, 50, 15, 24, 37, 2, 92, 39, 83, 7, 78, 52, 90, 29, 32, 94, 88, 62, 13, 96, 56, 68, 41, 23, 25, 46, 72, 48, 26, 54, 43, 93]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 18, 25, 26, 42, 44, 46, 51, 57, 73, 80]
# Ranked Result:
rr5 = [82, 63, 76, 30, 14, 12, 0, 78, 5, 51, 44, 97, 53, 20, 16, 25, 19, 95, 7, 17, 69, 57, 55, 18, 45, 10, 4, 59, 48, 49, 29, 71, 80, 13, 94, 1, 11, 75, 37, 34, 8, 31, 98, 15, 27, 89, 42, 61, 52, 3, 65, 21, 67, 72, 2, 40, 23, 35, 87, 62, 38, 22, 66, 77, 39, 86, 60, 58, 41, 46, 64, 96, 47, 73, 36, 84, 26, 70, 93, 85, 28, 83, 88, 9, 50, 81, 6, 74, 92, 90, 24, 99, 56, 68, 79, 33, 91, 43, 32, 54]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 71, 96]
# Ranked Result:
rr6 = [82, 76, 14, 63, 12, 97, 30, 53, 7, 0, 5, 78, 11, 20, 95, 48, 44, 31, 4, 55, 17, 69, 57, 19, 10, 25, 16, 29, 45, 18, 21, 51, 89, 37, 71, 59, 81, 23, 87, 22, 58, 80, 94, 27, 72, 1, 2, 13, 66, 75, 35, 36, 15, 40, 8, 38, 62, 92, 98, 34, 42, 49, 6, 46, 93, 64, 84, 65, 86, 67, 70, 50, 60, 96, 41, 74, 24, 77, 52, 61, 73, 3, 83, 39, 88, 56, 91, 28, 99, 47, 68, 85, 79, 90, 9, 26, 32, 33, 43, 54]
##################################################
# Ground Truth (Unranked):
gt7 = [3]
# Ranked Result:
rr7 = [82, 5, 30, 49, 20, 69, 76, 12, 89, 63, 78, 7, 61, 62, 29, 45, 77, 11, 4, 37, 21, 17, 19, 34, 25, 72, 48, 97, 44, 58, 94, 53, 93, 83, 57, 51, 14, 31, 16, 2, 18, 23, 59, 71, 95, 10, 66, 0, 81, 13, 87, 1, 80, 92, 67, 55, 3, 28, 42, 84, 79, 36, 35, 22, 91, 24, 75, 73, 70, 65, 99, 47, 8, 39, 9, 98, 40, 38, 86, 15, 74, 88, 60, 64, 50, 68, 52, 32, 41, 56, 46, 85, 26, 43, 96, 6, 27, 54, 90, 33]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [82, 30, 20, 91, 76, 69, 72, 5, 87, 11, 61, 63, 64, 59, 4, 89, 45, 23, 97, 58, 53, 17, 44, 78, 92, 22, 24, 93, 81, 7, 16, 70, 48, 12, 77, 2, 55, 31, 51, 29, 10, 79, 3, 95, 37, 19, 94, 71, 57, 0, 62, 75, 49, 18, 25, 83, 14, 84, 74, 73, 21, 80, 38, 98, 56, 35, 99, 42, 28, 67, 34, 36, 39, 60, 13, 8, 66, 65, 9, 46, 32, 96, 86, 1, 27, 43, 41, 47, 68, 50, 40, 33, 52, 85, 88, 15, 6, 54, 26, 90]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 82, 20, 76, 69, 89, 45, 55, 63, 17, 61, 44, 87, 5, 97, 7, 81, 59, 4, 10, 12, 31, 23, 71, 53, 58, 11, 22, 21, 19, 78, 72, 49, 92, 18, 29, 37, 93, 60, 62, 95, 70, 94, 35, 48, 51, 14, 77, 25, 16, 0, 84, 2, 3, 75, 42, 65, 38, 80, 32, 83, 36, 57, 13, 91, 98, 67, 64, 27, 86, 74, 24, 79, 41, 39, 28, 1, 47, 99, 8, 73, 85, 9, 66, 40, 34, 15, 96, 46, 6, 68, 50, 56, 43, 52, 33, 54, 26, 90, 88]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 8, 13, 15, 16, 38, 42, 51, 52, 57, 59, 66, 68, 95]
# Ranked Result:
rr10 = [63, 14, 0, 95, 16, 55, 44, 25, 51, 71, 75, 57, 82, 8, 15, 10, 59, 13, 65, 94, 78, 80, 86, 69, 18, 53, 3, 26, 52, 17, 12, 40, 68, 96, 42, 85, 2, 98, 20, 73, 67, 1, 41, 60, 35, 31, 39, 88, 34, 27, 48, 90, 9, 36, 6, 30, 47, 84, 11, 38, 74, 46, 7, 4, 19, 66, 56, 50, 23, 21, 37, 79, 45, 61, 33, 70, 64, 54, 43, 49, 58, 87, 76, 99, 5, 97, 89, 29, 83, 24, 92, 72, 22, 77, 28, 93, 32, 62, 81, 91]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 55, 65, 86, 88]
# Ranked Result:
rr11 = [71, 10, 16, 95, 94, 0, 18, 13, 57, 63, 65, 44, 15, 75, 25, 14, 86, 23, 98, 73, 55, 12, 80, 40, 35, 2, 31, 39, 53, 47, 78, 52, 96, 51, 8, 59, 67, 36, 87, 42, 38, 19, 68, 69, 26, 66, 48, 24, 20, 17, 41, 74, 56, 37, 46, 88, 7, 11, 1, 85, 9, 84, 90, 3, 81, 70, 60, 61, 4, 27, 29, 5, 50, 89, 21, 6, 34, 82, 92, 45, 43, 64, 30, 54, 79, 58, 93, 77, 33, 62, 99, 22, 28, 49, 72, 97, 83, 76, 32, 91]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69]
# Ranked Result:
rr12 = [69, 82, 20, 45, 29, 49, 11, 89, 34, 5, 78, 7, 62, 12, 37, 30, 19, 48, 21, 61, 63, 58, 76, 4, 25, 77, 83, 93, 92, 72, 23, 44, 17, 94, 16, 51, 81, 53, 1, 66, 14, 18, 99, 67, 55, 80, 3, 13, 71, 28, 91, 42, 0, 75, 97, 59, 65, 57, 95, 64, 36, 70, 10, 87, 24, 60, 31, 79, 35, 9, 73, 39, 32, 40, 8, 47, 38, 86, 22, 2, 46, 50, 98, 84, 56, 85, 15, 6, 74, 41, 43, 26, 52, 68, 96, 27, 88, 54, 33, 90]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 91, 69, 20, 45, 3, 77, 64, 79, 11, 58, 61, 72, 17, 82, 59, 87, 4, 70, 81, 53, 92, 78, 89, 44, 24, 55, 63, 23, 93, 73, 83, 34, 48, 74, 5, 7, 51, 37, 16, 75, 42, 80, 94, 35, 49, 31, 29, 22, 12, 67, 19, 25, 99, 95, 39, 71, 21, 98, 56, 10, 1, 18, 2, 97, 62, 36, 0, 66, 65, 57, 38, 28, 13, 86, 60, 46, 43, 9, 8, 14, 27, 96, 84, 76, 50, 32, 41, 47, 40, 26, 6, 68, 52, 85, 33, 88, 15, 54, 90]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 20, 45, 69, 7, 82, 89, 61, 12, 63, 5, 19, 11, 48, 21, 77, 25, 29, 78, 4, 17, 58, 83, 59, 62, 81, 23, 55, 37, 92, 87, 44, 94, 1, 76, 16, 35, 93, 18, 51, 49, 99, 3, 34, 72, 65, 31, 70, 13, 53, 71, 14, 95, 42, 64, 10, 67, 60, 66, 80, 86, 36, 32, 0, 75, 47, 57, 38, 28, 22, 97, 91, 40, 73, 56, 39, 8, 9, 79, 6, 27, 50, 24, 85, 41, 2, 74, 98, 15, 84, 43, 46, 52, 26, 68, 54, 96, 88, 33, 90]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
- Recall @
[0.0, 0.0, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.06060606060606061, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.12121212121212122, 0.15151515151515152, 0.15151515151515152, 0.15151515151515152, 0.15151515151515152, 0.18181818181818182, 0.18181818181818182, 0.21212121212121213, 0.21212121212121213, 0.24242424242424243, 0.2727272727272727, 0.2727272727272727, 0.30303030303030304, 0.30303030303030304, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.36363636363636365, 0.36363636363636365, 0.3939393939393939, 0.3939393939393939, 0.3939393939393939, 0.3939393939393939, 0.42424

####Ground Truth by 2nd Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 76, 63, 14, 30, 51, 0, 97, 53, 19, 16, 57, 5, 12, 34, 95, 27, 80, 78, 1, 59, 55, 3, 52, 44, 49, 25, 18, 48, 98, 28, 45, 4, 8, 66, 7, 21, 15, 64, 20, 42, 75, 17, 94, 71, 40, 37, 13, 10, 60, 69, 88, 85, 38, 29, 11, 46, 26, 50, 67, 36, 65, 96, 90, 47, 70, 86, 35, 77, 31, 58, 39, 6, 73, 91, 33, 84, 41, 61, 56, 2, 99, 9, 22, 79, 74, 87, 92, 23, 83, 24, 62, 89, 68, 72, 81, 54, 43, 93, 32]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 76, 30, 63, 14, 97, 34, 5, 21, 53, 12, 51, 48, 19, 4, 78, 0, 45, 7, 29, 28, 27, 1, 49, 66, 64, 80, 58, 11, 59, 57, 20, 25, 77, 60, 17, 99, 6, 69, 50, 16, 37, 70, 22, 92, 91, 46, 95, 18, 62, 40, 36, 52, 85, 98, 3, 94, 55, 42, 44, 83, 56, 88, 79, 8, 71, 72, 96, 32, 81, 35, 67, 38, 31, 13, 23, 15, 47, 9, 87, 75, 74, 89, 33, 84, 39, 24, 86, 90, 41, 2, 73, 65, 93, 61, 10, 26, 68, 54, 43]
##################################################
# Ground Truth (Unranked):
gt2 = [20, 29, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 76, 30, 63, 14, 34, 97, 21, 5, 4, 51, 19, 53, 28, 45, 12, 48, 78, 0, 49, 1, 66, 29, 20, 58, 7, 11, 77, 80, 64, 27, 99, 69, 57, 59, 25, 92, 91, 70, 16, 3, 50, 17, 37, 22, 6, 60, 18, 95, 62, 36, 55, 94, 44, 83, 46, 42, 52, 40, 85, 8, 71, 98, 79, 67, 88, 32, 72, 96, 23, 56, 75, 35, 81, 9, 15, 13, 47, 74, 87, 39, 38, 31, 89, 24, 86, 73, 93, 41, 84, 33, 10, 65, 2, 61, 26, 90, 68, 54, 43]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 76, 97, 30, 64, 28, 91, 63, 14, 20, 58, 53, 21, 59, 51, 80, 55, 12, 11, 19, 3, 1, 50, 34, 78, 45, 70, 92, 10, 99, 60, 71, 69, 66, 48, 77, 0, 73, 38, 27, 65, 36, 5, 67, 17, 57, 56, 4, 42, 95, 46, 44, 75, 32, 9, 98, 96, 22, 16, 86, 74, 31, 47, 23, 79, 83, 6, 15, 85, 72, 52, 61, 24, 87, 25, 88, 8, 35, 29, 33, 40, 7, 90, 81, 94, 37, 49, 26, 18, 43, 54, 39, 68, 89, 13, 2, 93, 41, 62, 84]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 30, 76, 97, 14, 60, 63, 45, 27, 49, 51, 3, 21, 55, 59, 64, 58, 61, 91, 53, 5, 70, 20, 17, 69, 73, 38, 89, 80, 19, 36, 33, 75, 66, 77, 79, 12, 1, 74, 42, 85, 87, 16, 71, 35, 0, 99, 28, 47, 10, 86, 98, 84, 8, 81, 4, 22, 11, 67, 9, 65, 40, 95, 18, 34, 31, 44, 6, 57, 50, 15, 24, 37, 2, 92, 39, 83, 7, 78, 52, 90, 29, 32, 94, 88, 62, 13, 96, 56, 68, 41, 23, 25, 46, 72, 48, 26, 54, 43, 93]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 25, 26, 42, 46, 51, 52, 59, 63, 80, 88, 95, 98]
# Ranked Result:
rr5 = [82, 63, 76, 30, 14, 12, 0, 78, 5, 51, 44, 97, 53, 20, 16, 25, 19, 95, 7, 17, 69, 57, 55, 18, 45, 10, 4, 59, 48, 49, 29, 71, 80, 13, 94, 1, 11, 75, 37, 34, 8, 31, 98, 15, 27, 89, 42, 61, 52, 3, 65, 21, 67, 72, 2, 40, 23, 35, 87, 62, 38, 22, 66, 77, 39, 86, 60, 58, 41, 46, 64, 96, 47, 73, 36, 84, 26, 70, 93, 85, 28, 83, 88, 9, 50, 81, 6, 74, 92, 90, 24, 99, 56, 68, 79, 33, 91, 43, 32, 54]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 54, 64, 71]
# Ranked Result:
rr6 = [82, 76, 14, 63, 12, 97, 30, 53, 7, 0, 5, 78, 11, 20, 95, 48, 44, 31, 4, 55, 17, 69, 57, 19, 10, 25, 16, 29, 45, 18, 21, 51, 89, 37, 71, 59, 81, 23, 87, 22, 58, 80, 94, 27, 72, 1, 2, 13, 66, 75, 35, 36, 15, 40, 8, 38, 62, 92, 98, 34, 42, 49, 6, 46, 93, 64, 84, 65, 86, 67, 70, 50, 60, 96, 41, 74, 24, 77, 52, 61, 73, 3, 83, 39, 88, 56, 91, 28, 99, 47, 68, 85, 79, 90, 9, 26, 32, 33, 43, 54]
##################################################
# Ground Truth (Unranked):
gt7 = [3, 20, 29, 34, 37, 77, 89]
# Ranked Result:
rr7 = [82, 5, 30, 49, 20, 69, 76, 12, 89, 63, 78, 7, 61, 62, 29, 45, 77, 11, 4, 37, 21, 17, 19, 34, 25, 72, 48, 97, 44, 58, 94, 53, 93, 83, 57, 51, 14, 31, 16, 2, 18, 23, 59, 71, 95, 10, 66, 0, 81, 13, 87, 1, 80, 92, 67, 55, 3, 28, 42, 84, 79, 36, 35, 22, 91, 24, 75, 73, 70, 65, 99, 47, 8, 39, 9, 98, 40, 38, 86, 15, 74, 88, 60, 64, 50, 68, 52, 32, 41, 56, 46, 85, 26, 43, 96, 6, 27, 54, 90, 33]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [82, 30, 20, 91, 76, 69, 72, 5, 87, 11, 61, 63, 64, 59, 4, 89, 45, 23, 97, 58, 53, 17, 44, 78, 92, 22, 24, 93, 81, 7, 16, 70, 48, 12, 77, 2, 55, 31, 51, 29, 10, 79, 3, 95, 37, 19, 94, 71, 57, 0, 62, 75, 49, 18, 25, 83, 14, 84, 74, 73, 21, 80, 38, 98, 56, 35, 99, 42, 28, 67, 34, 36, 39, 60, 13, 8, 66, 65, 9, 46, 32, 96, 86, 1, 27, 43, 41, 47, 68, 50, 40, 33, 52, 85, 88, 15, 6, 54, 26, 90]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 82, 20, 76, 69, 89, 45, 55, 63, 17, 61, 44, 87, 5, 97, 7, 81, 59, 4, 10, 12, 31, 23, 71, 53, 58, 11, 22, 21, 19, 78, 72, 49, 92, 18, 29, 37, 93, 60, 62, 95, 70, 94, 35, 48, 51, 14, 77, 25, 16, 0, 84, 2, 3, 75, 42, 65, 38, 80, 32, 83, 36, 57, 13, 91, 98, 67, 64, 27, 86, 74, 24, 79, 41, 39, 28, 1, 47, 99, 8, 73, 85, 9, 66, 40, 34, 15, 96, 46, 6, 68, 50, 56, 43, 52, 33, 54, 26, 90, 88]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 3, 8, 13, 15, 26, 38, 42, 51, 52, 57, 59, 66, 68, 88, 95]
# Ranked Result:
rr10 = [63, 14, 0, 95, 16, 55, 44, 25, 51, 71, 75, 57, 82, 8, 15, 10, 59, 13, 65, 94, 78, 80, 86, 69, 18, 53, 3, 26, 52, 17, 12, 40, 68, 96, 42, 85, 2, 98, 20, 73, 67, 1, 41, 60, 35, 31, 39, 88, 34, 27, 48, 90, 9, 36, 6, 30, 47, 84, 11, 38, 74, 46, 7, 4, 19, 66, 56, 50, 23, 21, 37, 79, 45, 61, 33, 70, 64, 54, 43, 49, 58, 87, 76, 99, 5, 97, 89, 29, 83, 24, 92, 72, 22, 77, 28, 93, 32, 62, 81, 91]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 86, 88]
# Ranked Result:
rr11 = [71, 10, 16, 95, 94, 0, 18, 13, 57, 63, 65, 44, 15, 75, 25, 14, 86, 23, 98, 73, 55, 12, 80, 40, 35, 2, 31, 39, 53, 47, 78, 52, 96, 51, 8, 59, 67, 36, 87, 42, 38, 19, 68, 69, 26, 66, 48, 24, 20, 17, 41, 74, 56, 37, 46, 88, 7, 11, 1, 85, 9, 84, 90, 3, 81, 70, 60, 61, 4, 27, 29, 5, 50, 89, 21, 6, 34, 82, 92, 45, 43, 64, 30, 54, 79, 58, 93, 77, 33, 62, 99, 22, 28, 49, 72, 97, 83, 76, 32, 91]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69, 34]
# Ranked Result:
rr12 = [69, 82, 20, 45, 29, 49, 11, 89, 34, 5, 78, 7, 62, 12, 37, 30, 19, 48, 21, 61, 63, 58, 76, 4, 25, 77, 83, 93, 92, 72, 23, 44, 17, 94, 16, 51, 81, 53, 1, 66, 14, 18, 99, 67, 55, 80, 3, 13, 71, 28, 91, 42, 0, 75, 97, 59, 65, 57, 95, 64, 36, 70, 10, 87, 24, 60, 31, 79, 35, 9, 73, 39, 32, 40, 8, 47, 38, 86, 22, 2, 46, 50, 98, 84, 56, 85, 15, 6, 74, 41, 43, 26, 52, 68, 96, 27, 88, 54, 33, 90]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 91, 69, 20, 45, 3, 77, 64, 79, 11, 58, 61, 72, 17, 82, 59, 87, 4, 70, 81, 53, 92, 78, 89, 44, 24, 55, 63, 23, 93, 73, 83, 34, 48, 74, 5, 7, 51, 37, 16, 75, 42, 80, 94, 35, 49, 31, 29, 22, 12, 67, 19, 25, 99, 95, 39, 71, 21, 98, 56, 10, 1, 18, 2, 97, 62, 36, 0, 66, 65, 57, 38, 28, 13, 86, 60, 46, 43, 9, 8, 14, 27, 96, 84, 76, 50, 32, 41, 47, 40, 26, 6, 68, 52, 85, 33, 88, 15, 54, 90]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 20, 45, 69, 7, 82, 89, 61, 12, 63, 5, 19, 11, 48, 21, 77, 25, 29, 78, 4, 17, 58, 83, 59, 62, 81, 23, 55, 37, 92, 87, 44, 94, 1, 76, 16, 35, 93, 18, 51, 49, 99, 3, 34, 72, 65, 31, 70, 13, 53, 71, 14, 95, 42, 64, 10, 67, 60, 66, 80, 86, 36, 32, 0, 75, 47, 57, 38, 28, 22, 97, 91, 40, 73, 56, 39, 8, 9, 79, 6, 27, 50, 24, 85, 41, 2, 74, 98, 15, 84, 43, 46, 52, 26, 68, 54, 96, 88, 33, 90]
#################################################

In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
- Recall @
[0.0, 0.0, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.11428571428571428, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.17142857142857143, 0.17142857142857143, 0.2, 0.2, 0.22857142857142856, 0.2571428571428571, 0.2571428571428571, 0.2857142857142857, 0.3142857142857143, 0.34285714285714286, 0.34285714285714286, 0.34285714285714286, 0.37142857142857144, 0.37142857142857144, 0.4, 0.4, 0.42857142857142855, 0.42857142857142855, 0.45714285714285713, 0.4857142857142857, 0.4857142857142857, 0.485714

### SBERT indolem/indobert-base-uncased

####Ground Truth by 1st Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 433, 44, 46, 50, 51, 57, 59, 63, 65, 66, 67, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 5, 53, 30, 17, 49, 63, 55, 0, 60, 61, 44, 14, 19, 27, 95, 51, 45, 4, 59, 42, 98, 29, 90, 22, 48, 80, 78, 69, 20, 40, 76, 18, 87, 72, 96, 3, 21, 46, 8, 35, 12, 67, 52, 89, 16, 91, 34, 66, 84, 36, 75, 47, 6, 62, 97, 71, 31, 81, 77, 88, 33, 1, 64, 26, 56, 13, 15, 70, 65, 41, 85, 11, 25, 9, 86, 23, 83, 92, 57, 7, 73, 39, 74, 68, 2, 58, 32, 28, 50, 43, 79, 94, 10, 37, 38, 24, 99, 93, 54]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 17, 53, 61, 44, 60, 30, 45, 0, 63, 19, 22, 89, 59, 98, 5, 27, 21, 4, 42, 95, 55, 78, 80, 14, 3, 87, 49, 51, 12, 96, 72, 29, 48, 46, 31, 90, 20, 7, 6, 34, 69, 40, 76, 18, 97, 66, 11, 52, 23, 47, 84, 64, 1, 62, 16, 94, 91, 35, 43, 77, 67, 25, 92, 36, 71, 2, 56, 85, 26, 58, 65, 99, 33, 93, 9, 15, 10, 73, 75, 8, 57, 38, 88, 83, 54, 13, 41, 81, 70, 68, 50, 86, 28, 74, 32, 79, 24, 39, 37]
##################################################
# Ground Truth (Unranked):
gt2 = [31, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 17, 53, 45, 0, 61, 29, 30, 60, 4, 63, 5, 89, 98, 44, 49, 87, 22, 14, 21, 78, 27, 59, 72, 3, 62, 55, 69, 19, 34, 95, 51, 42, 52, 18, 66, 90, 64, 48, 40, 12, 80, 46, 92, 35, 96, 33, 20, 84, 9, 36, 47, 81, 16, 11, 97, 67, 57, 76, 75, 94, 91, 56, 73, 71, 43, 23, 1, 6, 88, 8, 85, 25, 77, 15, 70, 68, 31, 38, 93, 26, 65, 83, 13, 58, 24, 7, 74, 99, 86, 50, 39, 79, 54, 41, 28, 10, 32, 37, 2]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 30, 5, 17, 49, 53, 55, 89, 63, 20, 69, 61, 78, 27, 14, 97, 76, 44, 59, 60, 62, 19, 87, 91, 48, 35, 98, 0, 81, 92, 70, 42, 66, 45, 56, 40, 18, 22, 46, 21, 36, 72, 16, 33, 34, 77, 96, 90, 80, 51, 29, 12, 4, 64, 8, 1, 67, 6, 52, 11, 88, 13, 3, 26, 71, 47, 65, 73, 9, 28, 95, 75, 23, 83, 86, 24, 57, 85, 94, 25, 93, 68, 32, 50, 99, 15, 43, 41, 58, 31, 84, 79, 38, 39, 10, 74, 37, 2, 54, 7]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 17, 53, 30, 60, 76, 61, 0, 5, 63, 59, 97, 44, 98, 14, 22, 95, 92, 89, 19, 21, 27, 49, 51, 46, 42, 45, 91, 4, 55, 96, 87, 11, 20, 31, 72, 66, 64, 77, 18, 78, 40, 36, 34, 94, 47, 6, 52, 90, 3, 62, 81, 26, 33, 48, 29, 67, 84, 57, 80, 16, 25, 1, 56, 69, 88, 35, 65, 15, 85, 8, 58, 99, 28, 86, 71, 73, 24, 23, 83, 75, 38, 41, 50, 93, 68, 13, 70, 7, 32, 54, 43, 39, 37, 74, 12, 9, 79, 10, 2]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 18, 25, 26, 42, 44, 46, 51, 57, 73, 80]
# Ranked Result:
rr5 = [82, 30, 63, 5, 53, 14, 60, 89, 17, 44, 76, 49, 4, 72, 19, 45, 22, 97, 20, 55, 78, 95, 40, 21, 0, 66, 29, 80, 59, 27, 62, 34, 48, 3, 51, 87, 18, 91, 8, 92, 61, 7, 16, 25, 12, 67, 46, 42, 35, 1, 69, 2, 36, 98, 23, 71, 15, 70, 52, 90, 31, 9, 65, 47, 64, 96, 28, 84, 88, 75, 10, 11, 6, 56, 77, 83, 26, 33, 94, 99, 57, 73, 32, 68, 85, 39, 13, 58, 81, 79, 43, 86, 50, 41, 38, 74, 37, 93, 24, 54]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 71, 96]
# Ranked Result:
rr6 = [82, 30, 89, 17, 72, 53, 44, 5, 60, 4, 78, 49, 63, 62, 45, 14, 22, 7, 92, 40, 76, 51, 29, 95, 97, 87, 21, 46, 66, 3, 80, 25, 42, 0, 98, 12, 48, 19, 27, 2, 20, 11, 36, 31, 64, 96, 55, 71, 1, 34, 90, 18, 23, 65, 16, 33, 59, 47, 56, 15, 69, 84, 91, 61, 67, 28, 58, 88, 70, 77, 94, 9, 35, 10, 83, 75, 68, 8, 6, 26, 41, 81, 32, 52, 13, 93, 57, 99, 38, 79, 50, 73, 43, 39, 86, 74, 85, 24, 37, 54]
##################################################
# Ground Truth (Unranked):
gt7 = [3]
# Ranked Result:
rr7 = [82, 30, 17, 5, 89, 44, 63, 4, 78, 53, 72, 97, 45, 62, 20, 49, 7, 14, 29, 22, 19, 60, 55, 12, 21, 66, 91, 0, 23, 18, 34, 2, 27, 87, 95, 98, 92, 16, 76, 48, 46, 35, 61, 69, 1, 9, 42, 93, 83, 71, 59, 70, 51, 40, 77, 58, 84, 36, 81, 32, 64, 15, 67, 25, 79, 56, 37, 28, 6, 3, 99, 68, 31, 65, 11, 26, 80, 88, 39, 96, 73, 10, 8, 33, 50, 74, 13, 90, 52, 85, 75, 94, 38, 24, 43, 57, 47, 41, 86, 54]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 44, 17, 22, 53, 89, 82, 60, 72, 78, 45, 25, 23, 63, 95, 5, 61, 98, 91, 80, 51, 40, 19, 35, 62, 18, 14, 0, 84, 70, 42, 27, 92, 87, 16, 46, 55, 59, 20, 11, 96, 49, 3, 21, 71, 6, 66, 48, 36, 67, 10, 94, 81, 4, 9, 7, 69, 8, 97, 32, 33, 90, 93, 28, 99, 13, 77, 56, 1, 12, 31, 34, 83, 15, 47, 73, 68, 43, 2, 29, 26, 65, 52, 86, 58, 85, 39, 88, 75, 74, 64, 57, 50, 41, 79, 24, 38, 37, 76, 54]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [82, 30, 17, 53, 44, 89, 60, 45, 63, 5, 0, 62, 72, 22, 23, 4, 92, 78, 98, 61, 49, 42, 46, 80, 14, 19, 20, 59, 21, 55, 11, 48, 87, 95, 97, 76, 66, 40, 27, 51, 18, 10, 70, 71, 3, 90, 99, 96, 36, 84, 91, 25, 29, 69, 35, 52, 15, 67, 34, 9, 43, 8, 16, 12, 47, 64, 7, 1, 6, 33, 85, 31, 88, 56, 94, 28, 2, 81, 73, 32, 65, 83, 58, 26, 57, 77, 68, 93, 38, 39, 75, 79, 13, 86, 24, 41, 74, 37, 50, 54]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 8, 13, 15, 16, 38, 42, 51, 52, 57, 59, 66, 68, 95]
# Ranked Result:
rr10 = [53, 44, 95, 42, 16, 17, 27, 2, 13, 40, 0, 35, 55, 75, 71, 31, 15, 70, 14, 84, 94, 65, 25, 41, 4, 18, 36, 1, 82, 73, 10, 78, 77, 6, 68, 32, 60, 26, 67, 46, 3, 66, 90, 86, 28, 8, 85, 50, 88, 30, 47, 39, 52, 72, 11, 34, 33, 80, 92, 21, 62, 98, 38, 22, 91, 87, 56, 79, 23, 45, 51, 61, 83, 19, 20, 48, 12, 81, 37, 69, 74, 43, 9, 57, 96, 49, 99, 58, 59, 64, 63, 89, 7, 29, 97, 93, 5, 54, 24, 76]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 55, 65, 86, 88]
# Ranked Result:
rr11 = [4, 40, 78, 95, 0, 94, 53, 44, 25, 23, 10, 42, 18, 66, 35, 15, 45, 30, 71, 27, 36, 87, 21, 67, 12, 16, 9, 22, 62, 98, 75, 47, 31, 46, 3, 73, 1, 68, 70, 13, 65, 84, 90, 80, 72, 2, 34, 57, 92, 56, 14, 88, 43, 86, 52, 32, 89, 39, 33, 38, 96, 50, 85, 17, 6, 29, 8, 77, 55, 11, 51, 41, 26, 48, 74, 37, 91, 69, 81, 28, 54, 83, 99, 60, 7, 64, 79, 24, 20, 82, 58, 19, 93, 63, 59, 97, 61, 49, 5, 76]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69]
# Ranked Result:
rr12 = [63, 97, 20, 34, 14, 76, 19, 49, 98, 66, 21, 91, 64, 69, 18, 9, 48, 52, 1, 59, 27, 16, 67, 36, 70, 83, 55, 56, 96, 57, 8, 15, 24, 12, 77, 33, 71, 73, 88, 38, 85, 45, 50, 79, 65, 75, 58, 28, 68, 78, 74, 13, 37, 30, 6, 39, 17, 32, 26, 4, 22, 86, 92, 44, 87, 35, 0, 29, 42, 43, 41, 46, 53, 82, 99, 89, 72, 95, 60, 90, 23, 3, 40, 7, 51, 11, 94, 80, 93, 25, 81, 47, 5, 62, 84, 31, 61, 54, 10, 2]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 17, 82, 44, 63, 53, 89, 60, 91, 5, 78, 45, 49, 72, 98, 22, 27, 19, 59, 18, 14, 23, 20, 35, 95, 0, 55, 80, 97, 46, 51, 87, 92, 66, 48, 62, 70, 21, 4, 16, 69, 40, 96, 42, 36, 25, 81, 84, 8, 33, 9, 90, 67, 64, 47, 61, 34, 3, 77, 11, 93, 52, 6, 56, 71, 99, 83, 88, 13, 12, 68, 15, 85, 94, 1, 74, 31, 28, 26, 24, 86, 29, 43, 39, 73, 65, 57, 76, 10, 58, 79, 75, 32, 50, 41, 2, 38, 37, 7, 54]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 44, 82, 4, 53, 63, 17, 20, 78, 89, 45, 14, 19, 72, 95, 87, 60, 97, 12, 49, 66, 25, 40, 27, 21, 92, 0, 16, 22, 62, 98, 59, 5, 61, 67, 23, 18, 9, 29, 48, 36, 42, 35, 55, 83, 70, 51, 34, 90, 91, 76, 46, 80, 52, 15, 69, 77, 1, 75, 13, 94, 71, 88, 65, 7, 32, 68, 8, 84, 3, 56, 73, 64, 2, 99, 26, 81, 10, 93, 57, 33, 74, 28, 85, 79, 11, 96, 58, 38, 86, 47, 43, 41, 37, 6, 24, 31, 50, 39, 54]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
- Recall @
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030303030303030304, 0.030303030303030304, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.12121212121212122, 0.15151515151515152, 0.15151515151515152, 0.15151515151515152, 0.18181818181818182, 0.21212121212121213, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.30303030303030304, 0.30303030303030304, 0.30303030303030304, 0.30303030303030304, 0.3333

####Ground Truth by 2nd Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 5, 53, 30, 17, 49, 63, 55, 0, 60, 61, 44, 14, 19, 27, 95, 51, 45, 4, 59, 42, 98, 29, 90, 22, 48, 80, 78, 69, 20, 40, 76, 18, 87, 72, 96, 3, 21, 46, 8, 35, 12, 67, 52, 89, 16, 91, 34, 66, 84, 36, 75, 47, 6, 62, 97, 71, 31, 81, 77, 88, 33, 1, 64, 26, 56, 13, 15, 70, 65, 41, 85, 11, 25, 9, 86, 23, 83, 92, 57, 7, 73, 39, 74, 68, 2, 58, 32, 28, 50, 43, 79, 94, 10, 37, 38, 24, 99, 93, 54]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 17, 53, 61, 44, 60, 30, 45, 0, 63, 19, 22, 89, 59, 98, 5, 27, 21, 4, 42, 95, 55, 78, 80, 14, 3, 87, 49, 51, 12, 96, 72, 29, 48, 46, 31, 90, 20, 7, 6, 34, 69, 40, 76, 18, 97, 66, 11, 52, 23, 47, 84, 64, 1, 62, 16, 94, 91, 35, 43, 77, 67, 25, 92, 36, 71, 2, 56, 85, 26, 58, 65, 99, 33, 93, 9, 15, 10, 73, 75, 8, 57, 38, 88, 83, 54, 13, 41, 81, 70, 68, 50, 86, 28, 74, 32, 79, 24, 39, 37]
##################################################
# Ground Truth (Unranked):
gt2 = [20, 29, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 17, 53, 45, 0, 61, 29, 30, 60, 4, 63, 5, 89, 98, 44, 49, 87, 22, 14, 21, 78, 27, 59, 72, 3, 62, 55, 69, 19, 34, 95, 51, 42, 52, 18, 66, 90, 64, 48, 40, 12, 80, 46, 92, 35, 96, 33, 20, 84, 9, 36, 47, 81, 16, 11, 97, 67, 57, 76, 75, 94, 91, 56, 73, 71, 43, 23, 1, 6, 88, 8, 85, 25, 77, 15, 70, 68, 31, 38, 93, 26, 65, 83, 13, 58, 24, 7, 74, 99, 86, 50, 39, 79, 54, 41, 28, 10, 32, 37, 2]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 30, 5, 17, 49, 53, 55, 89, 63, 20, 69, 61, 78, 27, 14, 97, 76, 44, 59, 60, 62, 19, 87, 91, 48, 35, 98, 0, 81, 92, 70, 42, 66, 45, 56, 40, 18, 22, 46, 21, 36, 72, 16, 33, 34, 77, 96, 90, 80, 51, 29, 12, 4, 64, 8, 1, 67, 6, 52, 11, 88, 13, 3, 26, 71, 47, 65, 73, 9, 28, 95, 75, 23, 83, 86, 24, 57, 85, 94, 25, 93, 68, 32, 50, 99, 15, 43, 41, 58, 31, 84, 79, 38, 39, 10, 74, 37, 2, 54, 7]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 17, 53, 30, 60, 76, 61, 0, 5, 63, 59, 97, 44, 98, 14, 22, 95, 92, 89, 19, 21, 27, 49, 51, 46, 42, 45, 91, 4, 55, 96, 87, 11, 20, 31, 72, 66, 64, 77, 18, 78, 40, 36, 34, 94, 47, 6, 52, 90, 3, 62, 81, 26, 33, 48, 29, 67, 84, 57, 80, 16, 25, 1, 56, 69, 88, 35, 65, 15, 85, 8, 58, 99, 28, 86, 71, 73, 24, 23, 83, 75, 38, 41, 50, 93, 68, 13, 70, 7, 32, 54, 43, 39, 37, 74, 12, 9, 79, 10, 2]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 25, 26, 42, 46, 51, 52, 59, 63, 80, 88, 95, 98]
# Ranked Result:
rr5 = [82, 30, 63, 5, 53, 14, 60, 89, 17, 44, 76, 49, 4, 72, 19, 45, 22, 97, 20, 55, 78, 95, 40, 21, 0, 66, 29, 80, 59, 27, 62, 34, 48, 3, 51, 87, 18, 91, 8, 92, 61, 7, 16, 25, 12, 67, 46, 42, 35, 1, 69, 2, 36, 98, 23, 71, 15, 70, 52, 90, 31, 9, 65, 47, 64, 96, 28, 84, 88, 75, 10, 11, 6, 56, 77, 83, 26, 33, 94, 99, 57, 73, 32, 68, 85, 39, 13, 58, 81, 79, 43, 86, 50, 41, 38, 74, 37, 93, 24, 54]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 54, 64, 71]
# Ranked Result:
rr6 = [82, 30, 89, 17, 72, 53, 44, 5, 60, 4, 78, 49, 63, 62, 45, 14, 22, 7, 92, 40, 76, 51, 29, 95, 97, 87, 21, 46, 66, 3, 80, 25, 42, 0, 98, 12, 48, 19, 27, 2, 20, 11, 36, 31, 64, 96, 55, 71, 1, 34, 90, 18, 23, 65, 16, 33, 59, 47, 56, 15, 69, 84, 91, 61, 67, 28, 58, 88, 70, 77, 94, 9, 35, 10, 83, 75, 68, 8, 6, 26, 41, 81, 32, 52, 13, 93, 57, 99, 38, 79, 50, 73, 43, 39, 86, 74, 85, 24, 37, 54]
##################################################
# Ground Truth (Unranked):
gt7 = [3, 20, 29, 34, 37, 77, 89]
# Ranked Result:
rr7 = [82, 30, 17, 5, 89, 44, 63, 4, 78, 53, 72, 97, 45, 62, 20, 49, 7, 14, 29, 22, 19, 60, 55, 12, 21, 66, 91, 0, 23, 18, 34, 2, 27, 87, 95, 98, 92, 16, 76, 48, 46, 35, 61, 69, 1, 9, 42, 93, 83, 71, 59, 70, 51, 40, 77, 58, 84, 36, 81, 32, 64, 15, 67, 25, 79, 56, 37, 28, 6, 3, 99, 68, 31, 65, 11, 26, 80, 88, 39, 96, 73, 10, 8, 33, 50, 74, 13, 90, 52, 85, 75, 94, 38, 24, 43, 57, 47, 41, 86, 54]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 44, 17, 22, 53, 89, 82, 60, 72, 78, 45, 25, 23, 63, 95, 5, 61, 98, 91, 80, 51, 40, 19, 35, 62, 18, 14, 0, 84, 70, 42, 27, 92, 87, 16, 46, 55, 59, 20, 11, 96, 49, 3, 21, 71, 6, 66, 48, 36, 67, 10, 94, 81, 4, 9, 7, 69, 8, 97, 32, 33, 90, 93, 28, 99, 13, 77, 56, 1, 12, 31, 34, 83, 15, 47, 73, 68, 43, 2, 29, 26, 65, 52, 86, 58, 85, 39, 88, 75, 74, 64, 57, 50, 41, 79, 24, 38, 37, 76, 54]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [82, 30, 17, 53, 44, 89, 60, 45, 63, 5, 0, 62, 72, 22, 23, 4, 92, 78, 98, 61, 49, 42, 46, 80, 14, 19, 20, 59, 21, 55, 11, 48, 87, 95, 97, 76, 66, 40, 27, 51, 18, 10, 70, 71, 3, 90, 99, 96, 36, 84, 91, 25, 29, 69, 35, 52, 15, 67, 34, 9, 43, 8, 16, 12, 47, 64, 7, 1, 6, 33, 85, 31, 88, 56, 94, 28, 2, 81, 73, 32, 65, 83, 58, 26, 57, 77, 68, 93, 38, 39, 75, 79, 13, 86, 24, 41, 74, 37, 50, 54]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 3, 8, 13, 15, 26, 38, 42, 51, 52, 57, 59, 66, 68, 88, 95]
# Ranked Result:
rr10 = [53, 44, 95, 42, 16, 17, 27, 2, 13, 40, 0, 35, 55, 75, 71, 31, 15, 70, 14, 84, 94, 65, 25, 41, 4, 18, 36, 1, 82, 73, 10, 78, 77, 6, 68, 32, 60, 26, 67, 46, 3, 66, 90, 86, 28, 8, 85, 50, 88, 30, 47, 39, 52, 72, 11, 34, 33, 80, 92, 21, 62, 98, 38, 22, 91, 87, 56, 79, 23, 45, 51, 61, 83, 19, 20, 48, 12, 81, 37, 69, 74, 43, 9, 57, 96, 49, 99, 58, 59, 64, 63, 89, 7, 29, 97, 93, 5, 54, 24, 76]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 86, 88]
# Ranked Result:
rr11 = [4, 40, 78, 95, 0, 94, 53, 44, 25, 23, 10, 42, 18, 66, 35, 15, 45, 30, 71, 27, 36, 87, 21, 67, 12, 16, 9, 22, 62, 98, 75, 47, 31, 46, 3, 73, 1, 68, 70, 13, 65, 84, 90, 80, 72, 2, 34, 57, 92, 56, 14, 88, 43, 86, 52, 32, 89, 39, 33, 38, 96, 50, 85, 17, 6, 29, 8, 77, 55, 11, 51, 41, 26, 48, 74, 37, 91, 69, 81, 28, 54, 83, 99, 60, 7, 64, 79, 24, 20, 82, 58, 19, 93, 63, 59, 97, 61, 49, 5, 76]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69, 34]
# Ranked Result:
rr12 = [63, 97, 20, 34, 14, 76, 19, 49, 98, 66, 21, 91, 64, 69, 18, 9, 48, 52, 1, 59, 27, 16, 67, 36, 70, 83, 55, 56, 96, 57, 8, 15, 24, 12, 77, 33, 71, 73, 88, 38, 85, 45, 50, 79, 65, 75, 58, 28, 68, 78, 74, 13, 37, 30, 6, 39, 17, 32, 26, 4, 22, 86, 92, 44, 87, 35, 0, 29, 42, 43, 41, 46, 53, 82, 99, 89, 72, 95, 60, 90, 23, 3, 40, 7, 51, 11, 94, 80, 93, 25, 81, 47, 5, 62, 84, 31, 61, 54, 10, 2]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 17, 82, 44, 63, 53, 89, 60, 91, 5, 78, 45, 49, 72, 98, 22, 27, 19, 59, 18, 14, 23, 20, 35, 95, 0, 55, 80, 97, 46, 51, 87, 92, 66, 48, 62, 70, 21, 4, 16, 69, 40, 96, 42, 36, 25, 81, 84, 8, 33, 9, 90, 67, 64, 47, 61, 34, 3, 77, 11, 93, 52, 6, 56, 71, 99, 83, 88, 13, 12, 68, 15, 85, 94, 1, 74, 31, 28, 26, 24, 86, 29, 43, 39, 73, 65, 57, 76, 10, 58, 79, 75, 32, 50, 41, 2, 38, 37, 7, 54]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 44, 82, 4, 53, 63, 17, 20, 78, 89, 45, 14, 19, 72, 95, 87, 60, 97, 12, 49, 66, 25, 40, 27, 21, 92, 0, 16, 22, 62, 98, 59, 5, 61, 67, 23, 18, 9, 29, 48, 36, 42, 35, 55, 83, 70, 51, 34, 90, 91, 76, 46, 80, 52, 15, 69, 77, 1, 75, 13, 94, 71, 88, 65, 7, 32, 68, 8, 84, 3, 56, 73, 64, 2, 99, 26, 81, 10, 93, 57, 33, 74, 28, 85, 79, 11, 96, 58, 38, 86, 47, 43, 41, 37, 6, 24, 31, 50, 39, 54]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0]
- Recall @
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.11428571428571428, 0.14285714285714285, 0.17142857142857143, 0.17142857142857143, 0.2, 0.22857142857142856, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2857142857142857, 0.2857142857142857, 0.2857142857142857, 0.2857142857142857, 0.2857142857142857, 0.2857142857142857, 0.3142857142857143, 0.3142857142857143, 0.3142857142857143, 0.3142857142857143, 0.34285714285714286, 0.3428571428

###BERT bert-base-multilingual-cased

####Ground Truth by 1st Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 433, 44, 46, 50, 51, 57, 59, 63, 65, 66, 67, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [63, 55, 61, 60, 94, 76, 14, 85, 6, 82, 98, 5, 7, 84, 87, 53, 0, 38, 39, 42, 3, 58, 44, 52, 30, 69, 20, 80, 95, 11, 90, 75, 2, 25, 46, 97, 99, 10, 8, 93, 54, 12, 65, 66, 89, 19, 32, 1, 57, 4, 56, 37, 9, 48, 81, 13, 70, 18, 86, 71, 51, 27, 49, 67, 59, 77, 45, 29, 33, 43, 17, 21, 91, 92, 47, 78, 64, 34, 96, 50, 15, 40, 73, 22, 24, 28, 16, 88, 62, 23, 72, 74, 26, 35, 41, 31, 36, 68, 79, 83]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [55, 94, 85, 76, 98, 19, 29, 5, 61, 3, 53, 0, 52, 60, 11, 44, 7, 84, 14, 90, 48, 71, 30, 6, 87, 25, 39, 27, 63, 15, 42, 95, 20, 51, 81, 37, 65, 93, 1, 13, 59, 49, 82, 12, 89, 54, 56, 96, 80, 50, 86, 66, 99, 64, 21, 33, 18, 45, 67, 77, 38, 78, 8, 75, 4, 91, 40, 70, 92, 36, 34, 46, 47, 62, 2, 9, 43, 58, 32, 24, 41, 17, 28, 57, 10, 68, 16, 88, 83, 97, 69, 26, 22, 74, 79, 31, 73, 35, 72, 23]
##################################################
# Ground Truth (Unranked):
gt2 = [31, 34, 37, 77, 89]
# Ranked Result:
rr2 = [49, 5, 45, 20, 29, 77, 69, 89, 30, 7, 25, 61, 34, 37, 3, 11, 94, 21, 64, 1, 19, 91, 85, 76, 63, 53, 62, 70, 12, 58, 51, 82, 9, 66, 65, 80, 55, 83, 4, 92, 78, 60, 98, 87, 48, 95, 28, 8, 32, 96, 75, 43, 10, 93, 72, 99, 71, 41, 67, 39, 35, 59, 42, 44, 79, 74, 56, 24, 38, 84, 47, 97, 13, 36, 86, 14, 57, 81, 15, 18, 52, 16, 0, 33, 73, 23, 6, 68, 54, 27, 22, 50, 88, 17, 40, 26, 46, 90, 2, 31]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [14, 55, 39, 91, 48, 5, 66, 25, 71, 30, 98, 83, 11, 10, 90, 96, 26, 12, 33, 41, 45, 0, 82, 51, 60, 99, 24, 80, 78, 74, 85, 42, 7, 40, 1, 21, 43, 8, 94, 20, 92, 17, 9, 79, 75, 61, 50, 52, 62, 68, 36, 19, 53, 44, 46, 63, 70, 49, 58, 81, 95, 65, 59, 76, 64, 32, 89, 4, 13, 69, 87, 56, 88, 38, 93, 6, 15, 2, 37, 73, 31, 16, 22, 29, 27, 23, 3, 84, 57, 54, 72, 34, 18, 47, 77, 86, 28, 67, 97, 35]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [55, 84, 98, 94, 14, 90, 60, 17, 71, 39, 80, 87, 99, 76, 24, 52, 40, 11, 85, 6, 70, 42, 7, 19, 58, 66, 2, 50, 82, 30, 25, 5, 51, 61, 3, 89, 20, 53, 92, 96, 54, 46, 31, 47, 91, 33, 12, 95, 56, 44, 45, 38, 48, 10, 41, 75, 81, 78, 62, 88, 29, 68, 37, 65, 21, 64, 59, 73, 8, 1, 0, 15, 43, 86, 13, 27, 97, 22, 74, 26, 77, 9, 36, 4, 28, 93, 16, 57, 23, 69, 34, 79, 72, 63, 32, 83, 49, 18, 67, 35]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 18, 25, 26, 42, 44, 46, 51, 57, 73, 80]
# Ranked Result:
rr5 = [82, 76, 97, 14, 63, 12, 30, 44, 78, 17, 22, 7, 48, 55, 89, 72, 53, 51, 20, 4, 69, 10, 0, 18, 62, 37, 93, 87, 19, 71, 95, 29, 60, 42, 13, 94, 98, 58, 52, 11, 83, 8, 34, 66, 21, 25, 27, 15, 6, 45, 38, 67, 16, 59, 40, 80, 77, 57, 75, 23, 81, 46, 1, 32, 90, 24, 65, 9, 35, 92, 61, 41, 73, 85, 54, 39, 28, 99, 5, 31, 47, 68, 70, 64, 86, 96, 84, 26, 88, 50, 74, 36, 91, 79, 2, 43, 3, 56, 33, 49]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 71, 96]
# Ranked Result:
rr6 = [82, 7, 14, 44, 48, 51, 63, 71, 76, 10, 11, 53, 81, 19, 97, 12, 89, 22, 17, 55, 95, 30, 78, 0, 4, 52, 60, 18, 40, 87, 29, 31, 90, 85, 94, 98, 68, 37, 13, 58, 36, 8, 27, 93, 24, 15, 16, 84, 75, 6, 72, 23, 92, 83, 86, 32, 25, 42, 62, 65, 66, 39, 41, 67, 77, 54, 20, 38, 91, 50, 88, 26, 57, 73, 28, 70, 1, 80, 59, 56, 35, 74, 47, 45, 96, 69, 99, 2, 34, 46, 21, 3, 61, 79, 9, 43, 64, 33, 5, 49]
##################################################
# Ground Truth (Unranked):
gt7 = [3]
# Ranked Result:
rr7 = [82, 20, 89, 69, 78, 12, 30, 62, 45, 7, 29, 37, 49, 48, 63, 97, 83, 4, 72, 77, 5, 76, 34, 21, 11, 61, 58, 51, 19, 10, 22, 66, 91, 32, 55, 25, 44, 1, 94, 14, 18, 71, 17, 87, 16, 80, 23, 92, 9, 8, 64, 95, 13, 53, 41, 79, 70, 93, 67, 68, 74, 15, 36, 24, 65, 28, 96, 75, 42, 40, 99, 57, 81, 35, 3, 73, 60, 43, 47, 86, 0, 98, 26, 59, 27, 56, 54, 88, 52, 38, 39, 31, 50, 6, 85, 84, 46, 33, 90, 2]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 82, 7, 72, 89, 51, 48, 78, 12, 97, 63, 83, 76, 11, 71, 62, 22, 4, 91, 92, 10, 45, 20, 14, 69, 44, 77, 93, 17, 29, 32, 37, 58, 87, 19, 25, 36, 21, 8, 55, 24, 23, 53, 65, 34, 64, 75, 41, 40, 26, 68, 79, 16, 81, 96, 70, 60, 1, 66, 15, 31, 9, 74, 99, 43, 57, 95, 52, 80, 28, 0, 27, 18, 73, 39, 98, 50, 13, 59, 61, 5, 46, 6, 3, 88, 56, 49, 67, 85, 94, 54, 38, 42, 47, 84, 33, 86, 90, 35, 2]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 17, 7, 14, 55, 84, 51, 19, 60, 92, 22, 72, 71, 87, 11, 48, 93, 44, 82, 46, 53, 99, 81, 89, 29, 0, 31, 50, 76, 39, 27, 26, 91, 88, 58, 4, 24, 59, 97, 8, 78, 83, 41, 63, 12, 23, 40, 32, 10, 6, 62, 36, 52, 37, 28, 56, 68, 16, 77, 90, 75, 95, 98, 79, 45, 15, 54, 20, 1, 85, 80, 69, 34, 33, 73, 70, 96, 43, 74, 42, 21, 64, 18, 47, 9, 3, 65, 94, 66, 25, 57, 38, 86, 13, 2, 61, 67, 5, 35, 49]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 8, 13, 15, 16, 38, 42, 51, 52, 57, 59, 66, 68, 95]
# Ranked Result:
rr10 = [51, 16, 95, 15, 18, 71, 44, 13, 10, 4, 78, 12, 17, 8, 57, 52, 73, 19, 48, 66, 63, 68, 65, 55, 7, 75, 0, 53, 31, 24, 87, 25, 37, 98, 60, 67, 39, 14, 30, 94, 11, 23, 58, 81, 36, 86, 26, 88, 59, 22, 76, 9, 72, 34, 83, 1, 6, 90, 35, 77, 3, 92, 85, 41, 20, 93, 62, 29, 45, 89, 40, 96, 21, 27, 80, 32, 82, 42, 79, 46, 74, 50, 64, 38, 97, 56, 84, 28, 61, 43, 69, 54, 47, 70, 5, 2, 99, 91, 33, 49]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 55, 65, 86, 88]
# Ranked Result:
rr11 = [95, 10, 71, 16, 15, 4, 18, 51, 13, 44, 66, 24, 87, 94, 12, 7, 23, 78, 8, 65, 68, 52, 25, 31, 19, 57, 17, 73, 36, 98, 26, 35, 11, 75, 55, 48, 81, 67, 37, 90, 83, 53, 39, 88, 40, 58, 63, 86, 76, 6, 3, 84, 0, 92, 30, 9, 60, 1, 89, 85, 41, 42, 34, 14, 62, 59, 80, 20, 93, 96, 45, 32, 56, 72, 27, 77, 47, 50, 74, 21, 5, 38, 61, 2, 79, 70, 64, 54, 22, 29, 46, 43, 97, 82, 91, 28, 69, 99, 49, 33]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69]
# Ranked Result:
rr12 = [69, 78, 20, 49, 12, 62, 45, 37, 30, 4, 83, 72, 89, 48, 77, 66, 7, 29, 5, 34, 91, 63, 25, 82, 11, 21, 61, 19, 92, 51, 58, 94, 32, 71, 53, 17, 76, 1, 97, 9, 64, 44, 10, 93, 22, 13, 8, 65, 75, 55, 67, 16, 3, 18, 36, 87, 79, 96, 15, 14, 41, 95, 80, 24, 68, 70, 60, 40, 43, 99, 74, 81, 23, 26, 28, 57, 98, 86, 0, 54, 27, 42, 39, 50, 73, 59, 47, 85, 88, 56, 84, 52, 6, 35, 46, 38, 31, 90, 33, 2]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [91, 92, 83, 17, 71, 78, 24, 30, 36, 72, 11, 4, 93, 62, 12, 77, 45, 75, 8, 69, 51, 66, 48, 32, 41, 99, 58, 79, 37, 26, 7, 87, 10, 25, 39, 19, 3, 64, 23, 88, 43, 34, 53, 22, 16, 70, 68, 20, 44, 65, 50, 55, 96, 63, 40, 1, 52, 81, 21, 98, 9, 29, 84, 74, 89, 13, 95, 59, 57, 15, 60, 80, 56, 94, 28, 27, 76, 18, 6, 73, 54, 0, 46, 85, 49, 67, 14, 31, 97, 5, 42, 47, 61, 35, 33, 82, 86, 90, 38, 2]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 12, 62, 83, 78, 19, 72, 11, 7, 45, 20, 89, 22, 92, 48, 25, 77, 51, 69, 4, 32, 58, 93, 29, 91, 17, 56, 9, 37, 66, 21, 63, 94, 8, 87, 75, 1, 44, 53, 36, 34, 3, 60, 50, 10, 64, 41, 99, 14, 61, 79, 81, 24, 15, 86, 39, 5, 88, 0, 26, 42, 6, 43, 54, 52, 65, 55, 76, 18, 28, 40, 27, 67, 13, 59, 68, 80, 74, 71, 96, 47, 73, 70, 82, 95, 23, 57, 38, 84, 31, 16, 97, 85, 90, 49, 98, 35, 46, 33, 2]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
- Recall @
[0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.09090909090909091, 0.12121212121212122, 0.15151515151515152, 0.18181818181818182, 0.21212121212121213, 0.21212121212121213, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.2727272727272727, 0.30303030303030304, 0.30303030303030304, 0.30303030303030304, 0.

####Ground Truth by 2nd Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [63, 55, 61, 60, 94, 76, 14, 85, 6, 82, 98, 5, 7, 84, 87, 53, 0, 38, 39, 42, 3, 58, 44, 52, 30, 69, 20, 80, 95, 11, 90, 75, 2, 25, 46, 97, 99, 10, 8, 93, 54, 12, 65, 66, 89, 19, 32, 1, 57, 4, 56, 37, 9, 48, 81, 13, 70, 18, 86, 71, 51, 27, 49, 67, 59, 77, 45, 29, 33, 43, 17, 21, 91, 92, 47, 78, 64, 34, 96, 50, 15, 40, 73, 22, 24, 28, 16, 88, 62, 23, 72, 74, 26, 35, 41, 31, 36, 68, 79, 83]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [55, 94, 85, 76, 98, 19, 29, 5, 61, 3, 53, 0, 52, 60, 11, 44, 7, 84, 14, 90, 48, 71, 30, 6, 87, 25, 39, 27, 63, 15, 42, 95, 20, 51, 81, 37, 65, 93, 1, 13, 59, 49, 82, 12, 89, 54, 56, 96, 80, 50, 86, 66, 99, 64, 21, 33, 18, 45, 67, 77, 38, 78, 8, 75, 4, 91, 40, 70, 92, 36, 34, 46, 47, 62, 2, 9, 43, 58, 32, 24, 41, 17, 28, 57, 10, 68, 16, 88, 83, 97, 69, 26, 22, 74, 79, 31, 73, 35, 72, 23]
##################################################
# Ground Truth (Unranked):
gt2 = [20, 29, 34, 37, 77, 89]
# Ranked Result:
rr2 = [49, 5, 45, 20, 29, 77, 69, 89, 30, 7, 25, 61, 34, 37, 3, 11, 94, 21, 64, 1, 19, 91, 85, 76, 63, 53, 62, 70, 12, 58, 51, 82, 9, 66, 65, 80, 55, 83, 4, 92, 78, 60, 98, 87, 48, 95, 28, 8, 32, 96, 75, 43, 10, 93, 72, 99, 71, 41, 67, 39, 35, 59, 42, 44, 79, 74, 56, 24, 38, 84, 47, 97, 13, 36, 86, 14, 57, 81, 15, 18, 52, 16, 0, 33, 73, 23, 6, 68, 54, 27, 22, 50, 88, 17, 40, 26, 46, 90, 2, 31]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [14, 55, 39, 91, 48, 5, 66, 25, 71, 30, 98, 83, 11, 10, 90, 96, 26, 12, 33, 41, 45, 0, 82, 51, 60, 99, 24, 80, 78, 74, 85, 42, 7, 40, 1, 21, 43, 8, 94, 20, 92, 17, 9, 79, 75, 61, 50, 52, 62, 68, 36, 19, 53, 44, 46, 63, 70, 49, 58, 81, 95, 65, 59, 76, 64, 32, 89, 4, 13, 69, 87, 56, 88, 38, 93, 6, 15, 2, 37, 73, 31, 16, 22, 29, 27, 23, 3, 84, 57, 54, 72, 34, 18, 47, 77, 86, 28, 67, 97, 35]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [55, 84, 98, 94, 14, 90, 60, 17, 71, 39, 80, 87, 99, 76, 24, 52, 40, 11, 85, 6, 70, 42, 7, 19, 58, 66, 2, 50, 82, 30, 25, 5, 51, 61, 3, 89, 20, 53, 92, 96, 54, 46, 31, 47, 91, 33, 12, 95, 56, 44, 45, 38, 48, 10, 41, 75, 81, 78, 62, 88, 29, 68, 37, 65, 21, 64, 59, 73, 8, 1, 0, 15, 43, 86, 13, 27, 97, 22, 74, 26, 77, 9, 36, 4, 28, 93, 16, 57, 23, 69, 34, 79, 72, 63, 32, 83, 49, 18, 67, 35]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 25, 26, 42, 46, 51, 52, 59, 63, 80, 88, 95, 98]
# Ranked Result:
rr5 = [82, 76, 97, 14, 63, 12, 30, 44, 78, 17, 22, 7, 48, 55, 89, 72, 53, 51, 20, 4, 69, 10, 0, 18, 62, 37, 93, 87, 19, 71, 95, 29, 60, 42, 13, 94, 98, 58, 52, 11, 83, 8, 34, 66, 21, 25, 27, 15, 6, 45, 38, 67, 16, 59, 40, 80, 77, 57, 75, 23, 81, 46, 1, 32, 90, 24, 65, 9, 35, 92, 61, 41, 73, 85, 54, 39, 28, 99, 5, 31, 47, 68, 70, 64, 86, 96, 84, 26, 88, 50, 74, 36, 91, 79, 2, 43, 3, 56, 33, 49]
rr5 = [82, 63, 76, 30, 14, 12, 0, 78, 5, 51, 44, 97, 53, 20, 16, 25, 19, 95, 7, 17, 69, 57, 55, 18, 45, 10, 4, 59, 48, 49, 29, 71, 80, 13, 94, 1, 11, 75, 37, 34, 8, 31, 98, 15, 27, 89, 42, 61, 52, 3, 65, 21, 67, 72, 2, 40, 23, 35, 87, 62, 38, 22, 66, 77, 39, 86, 60, 58, 26, 41, 46, 64, 96, 47, 73, 36, 84, 70, 93, 85, 28, 88, 83, 9, 81, 6, 74, 92, 90, 24, 50, 99, 68, 56, 79, 33, 91, 43, 32, 54]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 54, 64, 71]
# Ranked Result:
rr6 = [82, 7, 14, 44, 48, 51, 63, 71, 76, 10, 11, 53, 81, 19, 97, 12, 89, 22, 17, 55, 95, 30, 78, 0, 4, 52, 60, 18, 40, 87, 29, 31, 90, 85, 94, 98, 68, 37, 13, 58, 36, 8, 27, 93, 24, 15, 16, 84, 75, 6, 72, 23, 92, 83, 86, 32, 25, 42, 62, 65, 66, 39, 41, 67, 77, 54, 20, 38, 91, 50, 88, 26, 57, 73, 28, 70, 1, 80, 59, 56, 35, 74, 47, 45, 96, 69, 99, 2, 34, 46, 21, 3, 61, 79, 9, 43, 64, 33, 5, 49]
##################################################
# Ground Truth (Unranked):
gt7 = [3, 20, 29, 34, 37, 77, 89]
# Ranked Result:
rr7 = [82, 20, 89, 69, 78, 12, 30, 62, 45, 7, 29, 37, 49, 48, 63, 97, 83, 4, 72, 77, 5, 76, 34, 21, 11, 61, 58, 51, 19, 10, 22, 66, 91, 32, 55, 25, 44, 1, 94, 14, 18, 71, 17, 87, 16, 80, 23, 92, 9, 8, 64, 95, 13, 53, 41, 79, 70, 93, 67, 68, 74, 15, 36, 24, 65, 28, 96, 75, 42, 40, 99, 57, 81, 35, 3, 73, 60, 43, 47, 86, 0, 98, 26, 59, 27, 56, 54, 88, 52, 38, 39, 31, 50, 6, 85, 84, 46, 33, 90, 2]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 82, 7, 72, 89, 51, 48, 78, 12, 97, 63, 83, 76, 11, 71, 62, 22, 4, 91, 92, 10, 45, 20, 14, 69, 44, 77, 93, 17, 29, 32, 37, 58, 87, 19, 25, 36, 21, 8, 55, 24, 23, 53, 65, 34, 64, 75, 41, 40, 26, 68, 79, 16, 81, 96, 70, 60, 1, 66, 15, 31, 9, 74, 99, 43, 57, 95, 52, 80, 28, 0, 27, 18, 73, 39, 98, 50, 13, 59, 61, 5, 46, 6, 3, 88, 56, 49, 67, 85, 94, 54, 38, 42, 47, 84, 33, 86, 90, 35, 2]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 17, 7, 14, 55, 84, 51, 19, 60, 92, 22, 72, 71, 87, 11, 48, 93, 44, 82, 46, 53, 99, 81, 89, 29, 0, 31, 50, 76, 39, 27, 26, 91, 88, 58, 4, 24, 59, 97, 8, 78, 83, 41, 63, 12, 23, 40, 32, 10, 6, 62, 36, 52, 37, 28, 56, 68, 16, 77, 90, 75, 95, 98, 79, 45, 15, 54, 20, 1, 85, 80, 69, 34, 33, 73, 70, 96, 43, 74, 42, 21, 64, 18, 47, 9, 3, 65, 94, 66, 25, 57, 38, 86, 13, 2, 61, 67, 5, 35, 49]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 3, 8, 13, 15, 26, 38, 42, 51, 52, 57, 59, 66, 68, 88, 95]
# Ranked Result:
rr10 = [51, 16, 95, 15, 18, 71, 44, 13, 10, 4, 78, 12, 17, 8, 57, 52, 73, 19, 48, 66, 63, 68, 65, 55, 7, 75, 0, 53, 31, 24, 87, 25, 37, 98, 60, 67, 39, 14, 30, 94, 11, 23, 58, 81, 36, 86, 26, 88, 59, 22, 76, 9, 72, 34, 83, 1, 6, 90, 35, 77, 3, 92, 85, 41, 20, 93, 62, 29, 45, 89, 40, 96, 21, 27, 80, 32, 82, 42, 79, 46, 74, 50, 64, 38, 97, 56, 84, 28, 61, 43, 69, 54, 47, 70, 5, 2, 99, 91, 33, 49]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 86, 88]
# Ranked Result:
rr11 = [95, 10, 71, 16, 15, 4, 18, 51, 13, 44, 66, 24, 87, 94, 12, 7, 23, 78, 8, 65, 68, 52, 25, 31, 19, 57, 17, 73, 36, 98, 26, 35, 11, 75, 55, 48, 81, 67, 37, 90, 83, 53, 39, 88, 40, 58, 63, 86, 76, 6, 3, 84, 0, 92, 30, 9, 60, 1, 89, 85, 41, 42, 34, 14, 62, 59, 80, 20, 93, 96, 45, 32, 56, 72, 27, 77, 47, 50, 74, 21, 5, 38, 61, 2, 79, 70, 64, 54, 22, 29, 46, 43, 97, 82, 91, 28, 69, 99, 49, 33]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69, 34]
# Ranked Result:
rr12 = [69, 78, 20, 49, 12, 62, 45, 37, 30, 4, 83, 72, 89, 48, 77, 66, 7, 29, 5, 34, 91, 63, 25, 82, 11, 21, 61, 19, 92, 51, 58, 94, 32, 71, 53, 17, 76, 1, 97, 9, 64, 44, 10, 93, 22, 13, 8, 65, 75, 55, 67, 16, 3, 18, 36, 87, 79, 96, 15, 14, 41, 95, 80, 24, 68, 70, 60, 40, 43, 99, 74, 81, 23, 26, 28, 57, 98, 86, 0, 54, 27, 42, 39, 50, 73, 59, 47, 85, 88, 56, 84, 52, 6, 35, 46, 38, 31, 90, 33, 2]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [91, 92, 83, 17, 71, 78, 24, 30, 36, 72, 11, 4, 93, 62, 12, 77, 45, 75, 8, 69, 51, 66, 48, 32, 41, 99, 58, 79, 37, 26, 7, 87, 10, 25, 39, 19, 3, 64, 23, 88, 43, 34, 53, 22, 16, 70, 68, 20, 44, 65, 50, 55, 96, 63, 40, 1, 52, 81, 21, 98, 9, 29, 84, 74, 89, 13, 95, 59, 57, 15, 60, 80, 56, 94, 28, 27, 76, 18, 6, 73, 54, 0, 46, 85, 49, 67, 14, 31, 97, 5, 42, 47, 61, 35, 33, 82, 86, 90, 38, 2]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 12, 62, 83, 78, 19, 72, 11, 7, 45, 20, 89, 22, 92, 48, 25, 77, 51, 69, 4, 32, 58, 93, 29, 91, 17, 56, 9, 37, 66, 21, 63, 94, 8, 87, 75, 1, 44, 53, 36, 34, 3, 60, 50, 10, 64, 41, 99, 14, 61, 79, 81, 24, 15, 86, 39, 5, 88, 0, 26, 42, 6, 43, 54, 52, 65, 55, 76, 18, 28, 40, 27, 67, 13, 59, 68, 80, 74, 71, 96, 47, 73, 70, 82, 95, 23, 57, 38, 84, 31, 16, 97, 85, 90, 49, 98, 35, 46, 33, 2]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
- Recall @
[0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.08571428571428572, 0.11428571428571428, 0.14285714285714285, 0.17142857142857143, 0.2, 0.2, 0.22857142857142856, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2857142857142857, 0.3142857142857143, 0.3142857142857143, 0.3142857142857143, 0.34285714285714286, 0.34285714285714286, 0.3428571

###BERT indobenchmark/indobert-base-p2

####Ground Truth by 1st Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 433, 44, 46, 50, 51, 57, 59, 63, 65, 66, 67, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 76, 63, 14, 30, 51, 0, 97, 53, 19, 16, 57, 5, 12, 34, 95, 27, 80, 78, 1, 59, 55, 3, 52, 44, 49, 25, 18, 48, 98, 28, 45, 4, 8, 66, 7, 21, 15, 64, 20, 42, 75, 17, 94, 71, 40, 37, 13, 10, 60, 69, 88, 85, 38, 29, 26, 11, 46, 67, 36, 65, 96, 90, 47, 70, 86, 35, 50, 77, 31, 58, 39, 6, 73, 91, 33, 84, 41, 61, 56, 2, 99, 9, 22, 79, 74, 87, 92, 23, 83, 24, 62, 89, 68, 72, 81, 43, 93, 32, 54]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 76, 30, 63, 14, 97, 34, 5, 21, 53, 12, 51, 48, 19, 4, 78, 0, 45, 7, 29, 28, 27, 1, 49, 66, 64, 80, 58, 11, 59, 57, 20, 25, 77, 60, 17, 99, 6, 69, 16, 50, 37, 70, 22, 92, 91, 46, 95, 18, 62, 40, 36, 52, 85, 98, 3, 94, 55, 42, 44, 83, 56, 79, 8, 71, 72, 88, 96, 32, 81, 35, 67, 38, 31, 13, 23, 15, 47, 9, 87, 75, 74, 89, 33, 84, 39, 24, 86, 90, 41, 2, 73, 65, 93, 61, 26, 10, 68, 54, 43]
##################################################
# Ground Truth (Unranked):
gt2 = [31, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 76, 30, 63, 14, 34, 97, 21, 5, 4, 51, 19, 53, 28, 45, 12, 48, 78, 0, 49, 1, 66, 29, 20, 58, 7, 11, 77, 80, 64, 27, 99, 69, 57, 59, 25, 92, 91, 70, 16, 3, 50, 17, 37, 22, 6, 60, 18, 95, 62, 36, 55, 94, 44, 83, 46, 42, 52, 40, 85, 8, 71, 98, 79, 67, 32, 72, 56, 96, 23, 75, 35, 81, 9, 15, 13, 88, 47, 74, 87, 39, 38, 31, 89, 24, 86, 73, 93, 41, 84, 33, 26, 10, 65, 2, 61, 90, 68, 54, 43]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 76, 97, 30, 64, 28, 91, 63, 14, 20, 58, 53, 21, 59, 51, 80, 55, 12, 11, 19, 3, 1, 34, 78, 45, 70, 92, 50, 10, 99, 60, 71, 69, 66, 48, 77, 0, 73, 38, 27, 65, 36, 5, 67, 56, 17, 57, 4, 42, 95, 46, 44, 75, 32, 9, 98, 96, 22, 16, 86, 74, 31, 47, 23, 79, 83, 6, 54, 15, 85, 72, 52, 61, 24, 87, 25, 8, 88, 35, 29, 33, 40, 7, 90, 26, 81, 94, 37, 49, 18, 43, 39, 68, 89, 13, 2, 93, 41, 62, 84]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 30, 76, 97, 14, 60, 63, 45, 27, 49, 51, 3, 21, 55, 59, 64, 58, 61, 91, 53, 5, 70, 20, 17, 69, 73, 38, 89, 80, 19, 36, 33, 75, 66, 77, 79, 12, 1, 74, 42, 85, 87, 16, 71, 35, 0, 99, 28, 47, 10, 86, 98, 84, 8, 81, 4, 22, 11, 67, 9, 65, 40, 95, 18, 34, 31, 44, 6, 57, 15, 24, 50, 37, 2, 92, 39, 83, 7, 78, 52, 90, 29, 32, 94, 88, 56, 62, 13, 96, 68, 41, 23, 25, 46, 72, 26, 48, 54, 43, 93]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 18, 25, 26, 42, 44, 46, 51, 57, 73, 80]
# Ranked Result:
rr5 = [82, 63, 76, 30, 14, 12, 0, 78, 5, 51, 44, 97, 53, 20, 16, 25, 19, 95, 7, 17, 69, 57, 55, 18, 45, 10, 4, 59, 48, 49, 29, 71, 80, 13, 94, 1, 11, 75, 37, 34, 8, 31, 98, 15, 27, 89, 42, 61, 52, 3, 65, 21, 67, 72, 2, 40, 23, 35, 87, 62, 38, 22, 66, 77, 39, 86, 60, 58, 26, 41, 46, 64, 96, 47, 73, 36, 84, 70, 93, 85, 28, 88, 83, 9, 81, 6, 74, 92, 90, 24, 50, 99, 68, 56, 79, 33, 91, 43, 32, 54]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 71, 96]
# Ranked Result:
rr6 = [82, 76, 14, 63, 12, 97, 30, 53, 7, 0, 5, 78, 11, 20, 95, 48, 44, 31, 4, 55, 17, 69, 57, 19, 10, 25, 16, 29, 45, 18, 21, 51, 89, 37, 71, 59, 81, 23, 87, 22, 58, 80, 94, 27, 72, 1, 2, 13, 66, 75, 35, 36, 15, 40, 8, 38, 62, 92, 98, 34, 42, 49, 6, 46, 93, 64, 84, 65, 86, 67, 70, 60, 96, 41, 74, 24, 77, 52, 61, 73, 50, 3, 83, 39, 56, 88, 91, 28, 99, 47, 68, 85, 79, 90, 26, 9, 32, 54, 33, 43]
##################################################
# Ground Truth (Unranked):
gt7 = [3]
# Ranked Result:
rr7 = [82, 5, 30, 49, 20, 69, 76, 12, 89, 63, 78, 7, 61, 62, 29, 45, 77, 11, 4, 37, 21, 17, 19, 34, 25, 72, 48, 97, 44, 58, 94, 53, 93, 83, 57, 51, 14, 31, 16, 2, 18, 23, 59, 71, 95, 10, 66, 0, 81, 13, 87, 1, 80, 92, 67, 55, 3, 28, 42, 84, 79, 36, 35, 22, 91, 24, 75, 73, 70, 65, 99, 47, 8, 39, 9, 98, 40, 38, 86, 15, 74, 60, 64, 68, 52, 32, 88, 41, 46, 56, 50, 85, 26, 43, 96, 6, 27, 54, 90, 33]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [82, 30, 20, 91, 76, 69, 72, 5, 87, 11, 61, 63, 64, 59, 4, 89, 45, 23, 97, 58, 53, 17, 44, 78, 92, 22, 24, 93, 81, 7, 16, 70, 48, 12, 77, 2, 55, 31, 51, 29, 10, 79, 3, 95, 37, 19, 94, 71, 57, 0, 62, 75, 49, 18, 25, 83, 14, 84, 74, 73, 21, 80, 38, 98, 35, 99, 56, 42, 28, 67, 34, 36, 39, 60, 13, 8, 66, 65, 9, 46, 54, 32, 96, 86, 1, 27, 43, 41, 47, 68, 50, 40, 33, 52, 85, 88, 15, 6, 26, 90]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 82, 20, 76, 69, 89, 45, 55, 63, 17, 61, 44, 87, 5, 97, 7, 81, 59, 4, 10, 12, 31, 23, 71, 53, 58, 11, 22, 21, 19, 78, 72, 49, 92, 18, 29, 37, 93, 60, 62, 95, 70, 94, 35, 48, 51, 14, 77, 25, 16, 0, 84, 2, 3, 75, 42, 65, 38, 80, 32, 83, 36, 57, 13, 91, 98, 67, 64, 27, 86, 74, 24, 79, 41, 39, 28, 1, 47, 99, 8, 73, 85, 9, 66, 40, 34, 15, 54, 96, 46, 6, 68, 50, 56, 43, 52, 33, 26, 88, 90]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 8, 13, 15, 16, 38, 42, 51, 52, 57, 59, 66, 68, 95]
# Ranked Result:
rr10 = [63, 14, 0, 95, 16, 55, 44, 25, 51, 71, 75, 57, 82, 8, 15, 10, 59, 13, 65, 94, 78, 26, 80, 86, 69, 18, 53, 3, 52, 17, 12, 40, 68, 96, 42, 85, 2, 98, 20, 73, 67, 1, 41, 60, 35, 88, 31, 39, 34, 27, 48, 90, 9, 36, 6, 30, 47, 84, 11, 38, 74, 46, 7, 4, 19, 66, 56, 23, 21, 37, 79, 45, 61, 33, 70, 64, 43, 49, 58, 50, 87, 76, 99, 5, 97, 89, 54, 29, 83, 24, 92, 72, 22, 77, 28, 93, 32, 62, 81, 91]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 55, 65, 86, 88]
# Ranked Result:
rr11 = [71, 10, 16, 95, 94, 0, 18, 13, 57, 63, 65, 44, 15, 75, 25, 14, 86, 23, 98, 73, 55, 12, 80, 40, 35, 2, 31, 39, 53, 47, 78, 52, 96, 51, 8, 59, 67, 36, 87, 42, 38, 19, 68, 69, 66, 26, 48, 24, 88, 20, 17, 41, 74, 37, 56, 46, 7, 11, 1, 85, 9, 84, 90, 3, 81, 70, 60, 61, 4, 27, 29, 5, 89, 21, 6, 34, 82, 50, 92, 45, 43, 64, 30, 54, 79, 58, 93, 77, 33, 62, 99, 22, 28, 49, 72, 97, 83, 76, 32, 91]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69]
# Ranked Result:
rr12 = [69, 82, 20, 45, 29, 49, 11, 89, 34, 5, 78, 7, 62, 12, 37, 30, 19, 48, 21, 61, 63, 58, 76, 4, 25, 77, 83, 93, 92, 72, 23, 44, 17, 94, 16, 51, 81, 53, 1, 66, 14, 18, 99, 67, 55, 80, 3, 13, 71, 28, 91, 42, 0, 75, 97, 59, 65, 57, 95, 64, 36, 70, 10, 87, 24, 60, 31, 79, 35, 9, 73, 39, 32, 40, 8, 47, 38, 86, 22, 2, 46, 50, 98, 84, 85, 56, 26, 15, 6, 74, 41, 43, 52, 68, 96, 27, 54, 88, 33, 90]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 91, 69, 20, 45, 3, 77, 64, 79, 11, 58, 61, 72, 17, 82, 59, 87, 4, 70, 81, 53, 92, 78, 89, 44, 24, 55, 63, 23, 93, 73, 83, 34, 48, 74, 5, 7, 51, 37, 16, 75, 42, 80, 94, 35, 49, 31, 29, 22, 12, 67, 19, 25, 99, 95, 39, 71, 21, 98, 56, 10, 1, 18, 2, 97, 62, 36, 0, 66, 65, 57, 38, 28, 13, 86, 60, 46, 43, 9, 8, 14, 54, 27, 96, 50, 84, 76, 26, 32, 41, 47, 40, 6, 68, 52, 85, 88, 33, 15, 90]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 20, 45, 69, 7, 82, 89, 61, 12, 63, 5, 19, 11, 48, 21, 77, 25, 29, 78, 4, 17, 58, 83, 59, 62, 81, 23, 55, 37, 92, 87, 44, 94, 1, 76, 16, 35, 93, 18, 51, 49, 99, 3, 34, 72, 65, 31, 70, 13, 53, 71, 14, 95, 42, 64, 10, 67, 60, 66, 80, 86, 36, 32, 0, 75, 47, 57, 38, 28, 22, 97, 91, 40, 73, 56, 39, 8, 9, 79, 6, 27, 50, 24, 85, 41, 2, 74, 98, 15, 54, 26, 84, 43, 46, 52, 88, 68, 96, 33, 90]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
- Recall @
[0.0, 0.0, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.06060606060606061, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.12121212121212122, 0.15151515151515152, 0.15151515151515152, 0.15151515151515152, 0.15151515151515152, 0.18181818181818182, 0.18181818181818182, 0.21212121212121213, 0.21212121212121213, 0.24242424242424243, 0.2727272727272727, 0.2727272727272727, 0.30303030303030304, 0.30303030303030304, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.36363636363636365, 0.36363636363636365, 0.3939393939393939, 0.3939393939393939, 0.3939393939393939, 0.3939393939393939, 0.42424

####Ground Truth by 2nd Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 76, 63, 14, 30, 51, 0, 97, 53, 19, 16, 57, 5, 12, 34, 95, 27, 80, 78, 1, 59, 55, 3, 52, 44, 49, 25, 18, 48, 98, 28, 45, 4, 8, 66, 7, 21, 15, 64, 20, 42, 75, 17, 94, 71, 40, 37, 13, 10, 60, 69, 88, 85, 38, 29, 26, 11, 46, 67, 36, 65, 96, 90, 47, 70, 86, 35, 50, 77, 31, 58, 39, 6, 73, 91, 33, 84, 41, 61, 56, 2, 99, 9, 22, 79, 74, 87, 92, 23, 83, 24, 62, 89, 68, 72, 81, 43, 93, 32, 54]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 76, 30, 63, 14, 97, 34, 5, 21, 53, 12, 51, 48, 19, 4, 78, 0, 45, 7, 29, 28, 27, 1, 49, 66, 64, 80, 58, 11, 59, 57, 20, 25, 77, 60, 17, 99, 6, 69, 16, 50, 37, 70, 22, 92, 91, 46, 95, 18, 62, 40, 36, 52, 85, 98, 3, 94, 55, 42, 44, 83, 56, 79, 8, 71, 72, 88, 96, 32, 81, 35, 67, 38, 31, 13, 23, 15, 47, 9, 87, 75, 74, 89, 33, 84, 39, 24, 86, 90, 41, 2, 73, 65, 93, 61, 26, 10, 68, 54, 43]
##################################################
# Ground Truth (Unranked):
gt2 = [20, 29, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 76, 30, 63, 14, 34, 97, 21, 5, 4, 51, 19, 53, 28, 45, 12, 48, 78, 0, 49, 1, 66, 29, 20, 58, 7, 11, 77, 80, 64, 27, 99, 69, 57, 59, 25, 92, 91, 70, 16, 3, 50, 17, 37, 22, 6, 60, 18, 95, 62, 36, 55, 94, 44, 83, 46, 42, 52, 40, 85, 8, 71, 98, 79, 67, 32, 72, 56, 96, 23, 75, 35, 81, 9, 15, 13, 88, 47, 74, 87, 39, 38, 31, 89, 24, 86, 73, 93, 41, 84, 33, 26, 10, 65, 2, 61, 90, 68, 54, 43]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 76, 97, 30, 64, 28, 91, 63, 14, 20, 58, 53, 21, 59, 51, 80, 55, 12, 11, 19, 3, 1, 34, 78, 45, 70, 92, 50, 10, 99, 60, 71, 69, 66, 48, 77, 0, 73, 38, 27, 65, 36, 5, 67, 56, 17, 57, 4, 42, 95, 46, 44, 75, 32, 9, 98, 96, 22, 16, 86, 74, 31, 47, 23, 79, 83, 6, 54, 15, 85, 72, 52, 61, 24, 87, 25, 8, 88, 35, 29, 33, 40, 7, 90, 26, 81, 94, 37, 49, 18, 43, 39, 68, 89, 13, 2, 93, 41, 62, 84]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 30, 76, 97, 14, 60, 63, 45, 27, 49, 51, 3, 21, 55, 59, 64, 58, 61, 91, 53, 5, 70, 20, 17, 69, 73, 38, 89, 80, 19, 36, 33, 75, 66, 77, 79, 12, 1, 74, 42, 85, 87, 16, 71, 35, 0, 99, 28, 47, 10, 86, 98, 84, 8, 81, 4, 22, 11, 67, 9, 65, 40, 95, 18, 34, 31, 44, 6, 57, 15, 24, 50, 37, 2, 92, 39, 83, 7, 78, 52, 90, 29, 32, 94, 88, 56, 62, 13, 96, 68, 41, 23, 25, 46, 72, 26, 48, 54, 43, 93]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 25, 26, 42, 46, 51, 52, 59, 63, 80, 88, 95, 98]
# Ranked Result:
rr5 = [82, 63, 76, 30, 14, 12, 0, 78, 5, 51, 44, 97, 53, 20, 16, 25, 19, 95, 7, 17, 69, 57, 55, 18, 45, 10, 4, 59, 48, 49, 29, 71, 80, 13, 94, 1, 11, 75, 37, 34, 8, 31, 98, 15, 27, 89, 42, 61, 52, 3, 65, 21, 67, 72, 2, 40, 23, 35, 87, 62, 38, 22, 66, 77, 39, 86, 60, 58, 26, 41, 46, 64, 96, 47, 73, 36, 84, 70, 93, 85, 28, 88, 83, 9, 81, 6, 74, 92, 90, 24, 50, 99, 68, 56, 79, 33, 91, 43, 32, 54]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 54, 64, 71]
# Ranked Result:
rr6 = [82, 76, 14, 63, 12, 97, 30, 53, 7, 0, 5, 78, 11, 20, 95, 48, 44, 31, 4, 55, 17, 69, 57, 19, 10, 25, 16, 29, 45, 18, 21, 51, 89, 37, 71, 59, 81, 23, 87, 22, 58, 80, 94, 27, 72, 1, 2, 13, 66, 75, 35, 36, 15, 40, 8, 38, 62, 92, 98, 34, 42, 49, 6, 46, 93, 64, 84, 65, 86, 67, 70, 60, 96, 41, 74, 24, 77, 52, 61, 73, 50, 3, 83, 39, 56, 88, 91, 28, 99, 47, 68, 85, 79, 90, 26, 9, 32, 54, 33, 43]
##################################################
# Ground Truth (Unranked):
gt7 = [3, 20, 29, 34, 37, 77, 89]
# Ranked Result:
rr7 = [82, 5, 30, 49, 20, 69, 76, 12, 89, 63, 78, 7, 61, 62, 29, 45, 77, 11, 4, 37, 21, 17, 19, 34, 25, 72, 48, 97, 44, 58, 94, 53, 93, 83, 57, 51, 14, 31, 16, 2, 18, 23, 59, 71, 95, 10, 66, 0, 81, 13, 87, 1, 80, 92, 67, 55, 3, 28, 42, 84, 79, 36, 35, 22, 91, 24, 75, 73, 70, 65, 99, 47, 8, 39, 9, 98, 40, 38, 86, 15, 74, 60, 64, 68, 52, 32, 88, 41, 46, 56, 50, 85, 26, 43, 96, 6, 27, 54, 90, 33]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [82, 30, 20, 91, 76, 69, 72, 5, 87, 11, 61, 63, 64, 59, 4, 89, 45, 23, 97, 58, 53, 17, 44, 78, 92, 22, 24, 93, 81, 7, 16, 70, 48, 12, 77, 2, 55, 31, 51, 29, 10, 79, 3, 95, 37, 19, 94, 71, 57, 0, 62, 75, 49, 18, 25, 83, 14, 84, 74, 73, 21, 80, 38, 98, 35, 99, 56, 42, 28, 67, 34, 36, 39, 60, 13, 8, 66, 65, 9, 46, 54, 32, 96, 86, 1, 27, 43, 41, 47, 68, 50, 40, 33, 52, 85, 88, 15, 6, 26, 90]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 82, 20, 76, 69, 89, 45, 55, 63, 17, 61, 44, 87, 5, 97, 7, 81, 59, 4, 10, 12, 31, 23, 71, 53, 58, 11, 22, 21, 19, 78, 72, 49, 92, 18, 29, 37, 93, 60, 62, 95, 70, 94, 35, 48, 51, 14, 77, 25, 16, 0, 84, 2, 3, 75, 42, 65, 38, 80, 32, 83, 36, 57, 13, 91, 98, 67, 64, 27, 86, 74, 24, 79, 41, 39, 28, 1, 47, 99, 8, 73, 85, 9, 66, 40, 34, 15, 54, 96, 46, 6, 68, 50, 56, 43, 52, 33, 26, 88, 90]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 3, 8, 13, 15, 26, 38, 42, 51, 52, 57, 59, 66, 68, 88, 95]
# Ranked Result:
rr10 = [63, 14, 0, 95, 16, 55, 44, 25, 51, 71, 75, 57, 82, 8, 15, 10, 59, 13, 65, 94, 78, 26, 80, 86, 69, 18, 53, 3, 52, 17, 12, 40, 68, 96, 42, 85, 2, 98, 20, 73, 67, 1, 41, 60, 35, 88, 31, 39, 34, 27, 48, 90, 9, 36, 6, 30, 47, 84, 11, 38, 74, 46, 7, 4, 19, 66, 56, 23, 21, 37, 79, 45, 61, 33, 70, 64, 43, 49, 58, 50, 87, 76, 99, 5, 97, 89, 54, 29, 83, 24, 92, 72, 22, 77, 28, 93, 32, 62, 81, 91]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 86, 88]
# Ranked Result:
rr11 = [71, 10, 16, 95, 94, 0, 18, 13, 57, 63, 65, 44, 15, 75, 25, 14, 86, 23, 98, 73, 55, 12, 80, 40, 35, 2, 31, 39, 53, 47, 78, 52, 96, 51, 8, 59, 67, 36, 87, 42, 38, 19, 68, 69, 66, 26, 48, 24, 88, 20, 17, 41, 74, 37, 56, 46, 7, 11, 1, 85, 9, 84, 90, 3, 81, 70, 60, 61, 4, 27, 29, 5, 89, 21, 6, 34, 82, 50, 92, 45, 43, 64, 30, 54, 79, 58, 93, 77, 33, 62, 99, 22, 28, 49, 72, 97, 83, 76, 32, 91]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69, 34]
# Ranked Result:
rr12 = [69, 82, 20, 45, 29, 49, 11, 89, 34, 5, 78, 7, 62, 12, 37, 30, 19, 48, 21, 61, 63, 58, 76, 4, 25, 77, 83, 93, 92, 72, 23, 44, 17, 94, 16, 51, 81, 53, 1, 66, 14, 18, 99, 67, 55, 80, 3, 13, 71, 28, 91, 42, 0, 75, 97, 59, 65, 57, 95, 64, 36, 70, 10, 87, 24, 60, 31, 79, 35, 9, 73, 39, 32, 40, 8, 47, 38, 86, 22, 2, 46, 50, 98, 84, 85, 56, 26, 15, 6, 74, 41, 43, 52, 68, 96, 27, 54, 88, 33, 90]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 91, 69, 20, 45, 3, 77, 64, 79, 11, 58, 61, 72, 17, 82, 59, 87, 4, 70, 81, 53, 92, 78, 89, 44, 24, 55, 63, 23, 93, 73, 83, 34, 48, 74, 5, 7, 51, 37, 16, 75, 42, 80, 94, 35, 49, 31, 29, 22, 12, 67, 19, 25, 99, 95, 39, 71, 21, 98, 56, 10, 1, 18, 2, 97, 62, 36, 0, 66, 65, 57, 38, 28, 13, 86, 60, 46, 43, 9, 8, 14, 54, 27, 96, 50, 84, 76, 26, 32, 41, 47, 40, 6, 68, 52, 85, 88, 33, 15, 90]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 20, 45, 69, 7, 82, 89, 61, 12, 63, 5, 19, 11, 48, 21, 77, 25, 29, 78, 4, 17, 58, 83, 59, 62, 81, 23, 55, 37, 92, 87, 44, 94, 1, 76, 16, 35, 93, 18, 51, 49, 99, 3, 34, 72, 65, 31, 70, 13, 53, 71, 14, 95, 42, 64, 10, 67, 60, 66, 80, 86, 36, 32, 0, 75, 47, 57, 38, 28, 22, 97, 91, 40, 73, 56, 39, 8, 9, 79, 6, 27, 50, 24, 85, 41, 2, 74, 98, 15, 54, 26, 84, 43, 46, 52, 88, 68, 96, 33, 90]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
- Recall @
[0.0, 0.0, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.11428571428571428, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.17142857142857143, 0.17142857142857143, 0.2, 0.2, 0.22857142857142856, 0.2571428571428571, 0.2571428571428571, 0.2857142857142857, 0.3142857142857143, 0.34285714285714286, 0.34285714285714286, 0.34285714285714286, 0.37142857142857144, 0.37142857142857144, 0.4, 0.4, 0.42857142857142855, 0.42857142857142855, 0.45714285714285713, 0.4857142857142857, 0.4857142857142857, 0.485714

###BERT indolem/indobert-base-uncased

####Ground Truth by 1st Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 433, 44, 46, 50, 51, 57, 59, 63, 65, 66, 67, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 5, 53, 30, 17, 49, 63, 55, 0, 60, 61, 44, 14, 19, 27, 95, 51, 45, 4, 59, 42, 98, 29, 90, 22, 48, 80, 78, 69, 20, 40, 76, 18, 87, 72, 96, 3, 21, 46, 8, 35, 12, 67, 52, 89, 16, 91, 34, 66, 84, 36, 75, 47, 6, 62, 97, 71, 31, 81, 77, 33, 88, 1, 26, 64, 56, 13, 15, 70, 65, 41, 85, 11, 25, 9, 86, 23, 83, 92, 57, 7, 54, 73, 39, 74, 68, 2, 58, 32, 28, 50, 43, 79, 94, 10, 37, 38, 24, 99, 93]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 17, 53, 61, 44, 60, 30, 45, 0, 63, 19, 22, 89, 59, 98, 5, 27, 21, 4, 42, 95, 55, 78, 80, 14, 3, 87, 49, 51, 12, 96, 72, 29, 48, 46, 31, 90, 20, 7, 6, 34, 69, 40, 76, 18, 97, 66, 11, 52, 23, 47, 84, 64, 1, 62, 16, 94, 91, 35, 43, 77, 67, 25, 92, 36, 71, 2, 56, 85, 26, 58, 65, 99, 33, 93, 9, 15, 10, 73, 75, 8, 57, 54, 38, 88, 83, 13, 41, 81, 70, 68, 50, 86, 28, 74, 32, 79, 24, 39, 37]
##################################################
# Ground Truth (Unranked):
gt2 = [31, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 17, 53, 45, 0, 61, 29, 30, 60, 4, 63, 5, 89, 98, 44, 49, 87, 22, 14, 21, 78, 27, 59, 72, 3, 62, 55, 69, 19, 34, 95, 51, 42, 52, 18, 66, 90, 64, 48, 40, 12, 80, 46, 92, 35, 96, 33, 20, 84, 9, 36, 47, 81, 16, 11, 97, 67, 56, 57, 76, 75, 94, 91, 73, 71, 43, 23, 1, 6, 8, 85, 88, 25, 77, 15, 70, 68, 26, 31, 38, 93, 65, 83, 13, 58, 24, 7, 54, 74, 99, 86, 50, 39, 79, 41, 28, 10, 32, 37, 2]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 30, 5, 17, 49, 53, 55, 89, 63, 20, 69, 61, 78, 27, 14, 97, 76, 44, 59, 60, 62, 19, 87, 91, 48, 35, 98, 0, 81, 92, 70, 42, 56, 66, 45, 40, 18, 22, 46, 21, 36, 72, 16, 33, 34, 77, 96, 90, 80, 51, 29, 12, 4, 64, 8, 1, 67, 6, 52, 11, 13, 3, 88, 26, 71, 47, 65, 73, 9, 28, 95, 75, 23, 83, 86, 54, 50, 24, 57, 85, 94, 25, 93, 68, 32, 99, 15, 43, 41, 58, 31, 84, 79, 38, 39, 10, 74, 37, 2, 7]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 17, 53, 30, 60, 76, 61, 0, 5, 63, 59, 97, 44, 98, 14, 22, 95, 92, 89, 19, 21, 27, 49, 51, 46, 42, 45, 91, 4, 55, 96, 87, 11, 20, 31, 72, 66, 64, 77, 18, 78, 40, 36, 34, 94, 47, 6, 52, 26, 90, 3, 62, 81, 33, 48, 29, 67, 56, 84, 57, 80, 16, 25, 1, 69, 88, 35, 65, 15, 85, 8, 58, 99, 28, 86, 71, 73, 50, 24, 23, 54, 83, 75, 38, 41, 93, 68, 13, 70, 7, 32, 43, 39, 37, 74, 12, 9, 79, 10, 2]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 18, 25, 26, 42, 44, 46, 51, 57, 73, 80]
# Ranked Result:
rr5 = [82, 30, 63, 5, 53, 14, 60, 89, 17, 44, 76, 49, 4, 72, 19, 45, 22, 97, 20, 55, 78, 95, 40, 21, 0, 66, 29, 80, 59, 27, 62, 34, 48, 3, 51, 87, 18, 91, 8, 92, 61, 7, 16, 25, 12, 67, 46, 42, 35, 1, 69, 2, 36, 98, 23, 71, 15, 70, 52, 90, 31, 88, 9, 65, 47, 64, 96, 28, 84, 75, 10, 26, 11, 6, 56, 77, 83, 33, 94, 99, 57, 73, 32, 68, 85, 39, 13, 58, 81, 50, 79, 43, 86, 54, 41, 38, 74, 37, 93, 24]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 71, 96]
# Ranked Result:
rr6 = [82, 30, 89, 17, 72, 53, 44, 5, 60, 4, 78, 49, 63, 62, 45, 14, 22, 7, 92, 40, 76, 51, 29, 95, 97, 87, 21, 46, 66, 3, 80, 25, 42, 0, 98, 12, 48, 19, 27, 2, 20, 11, 36, 31, 64, 96, 55, 71, 1, 34, 90, 18, 23, 65, 16, 33, 59, 47, 56, 15, 69, 84, 91, 61, 67, 28, 26, 88, 58, 70, 77, 94, 9, 35, 10, 83, 75, 68, 8, 6, 41, 81, 32, 52, 13, 93, 57, 99, 38, 79, 50, 73, 43, 39, 86, 74, 85, 54, 24, 37]
##################################################
# Ground Truth (Unranked):
gt7 = [3]
# Ranked Result:
rr7 = [82, 30, 17, 5, 89, 44, 63, 4, 78, 53, 72, 97, 45, 62, 20, 49, 7, 14, 29, 22, 19, 60, 55, 12, 21, 66, 91, 0, 23, 18, 34, 2, 27, 87, 95, 98, 92, 16, 76, 48, 46, 35, 61, 69, 1, 9, 42, 93, 83, 71, 59, 70, 51, 40, 77, 58, 84, 36, 81, 32, 64, 15, 67, 25, 79, 37, 28, 56, 6, 3, 99, 68, 26, 31, 65, 11, 80, 88, 39, 96, 73, 10, 8, 33, 50, 54, 74, 13, 90, 52, 85, 75, 94, 38, 24, 43, 57, 47, 41, 86]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 44, 17, 22, 53, 89, 82, 60, 72, 78, 45, 25, 23, 63, 95, 5, 61, 98, 91, 80, 51, 40, 19, 35, 62, 18, 14, 0, 84, 70, 42, 27, 92, 87, 16, 46, 55, 59, 20, 11, 96, 49, 3, 21, 71, 6, 66, 48, 36, 67, 10, 94, 81, 4, 9, 7, 69, 8, 97, 32, 33, 90, 93, 28, 56, 99, 13, 77, 1, 12, 31, 34, 83, 15, 47, 26, 73, 68, 43, 2, 29, 65, 88, 52, 86, 58, 85, 39, 50, 75, 74, 64, 54, 57, 41, 79, 24, 38, 37, 76]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [82, 30, 17, 53, 44, 89, 60, 45, 63, 5, 0, 62, 72, 22, 23, 4, 92, 78, 98, 61, 49, 42, 46, 80, 14, 19, 20, 59, 21, 55, 11, 48, 87, 95, 97, 76, 66, 40, 27, 51, 18, 10, 70, 71, 3, 90, 99, 96, 36, 84, 91, 25, 29, 69, 35, 52, 15, 67, 34, 9, 43, 8, 16, 12, 47, 64, 7, 88, 1, 6, 33, 85, 31, 56, 94, 26, 28, 2, 81, 73, 32, 65, 83, 58, 57, 54, 77, 68, 93, 38, 39, 75, 79, 50, 13, 86, 24, 41, 74, 37]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 8, 13, 15, 16, 38, 42, 51, 52, 57, 59, 66, 68, 95]
# Ranked Result:
rr10 = [53, 44, 95, 42, 16, 17, 27, 2, 13, 40, 0, 35, 55, 75, 71, 31, 15, 70, 14, 84, 94, 65, 25, 41, 4, 18, 36, 26, 1, 82, 73, 10, 78, 77, 6, 68, 32, 60, 67, 46, 3, 66, 90, 88, 86, 28, 8, 85, 30, 47, 39, 52, 72, 50, 11, 34, 33, 80, 92, 21, 62, 98, 38, 22, 91, 87, 56, 79, 23, 45, 51, 61, 83, 19, 20, 48, 12, 81, 37, 69, 74, 43, 54, 9, 57, 96, 49, 99, 58, 59, 64, 63, 89, 7, 29, 97, 93, 5, 24, 76]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 55, 65, 86, 88]
# Ranked Result:
rr11 = [4, 40, 78, 95, 0, 94, 53, 44, 25, 23, 10, 42, 18, 66, 35, 15, 45, 30, 71, 27, 36, 87, 21, 67, 12, 16, 9, 22, 62, 98, 75, 47, 31, 46, 3, 73, 1, 68, 70, 13, 65, 84, 90, 80, 72, 2, 34, 57, 92, 88, 56, 14, 43, 86, 52, 32, 89, 39, 33, 38, 96, 85, 17, 50, 6, 29, 8, 26, 77, 55, 11, 51, 41, 48, 74, 37, 91, 54, 69, 81, 28, 83, 99, 60, 7, 64, 79, 24, 20, 82, 58, 19, 93, 63, 59, 97, 61, 49, 5, 76]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69]
# Ranked Result:
rr12 = [63, 97, 20, 34, 14, 76, 19, 49, 98, 66, 21, 91, 64, 69, 18, 9, 48, 52, 1, 59, 27, 16, 67, 36, 70, 83, 55, 96, 57, 56, 8, 15, 50, 24, 12, 77, 33, 71, 73, 38, 88, 85, 45, 79, 65, 75, 58, 28, 68, 78, 74, 13, 37, 30, 6, 39, 17, 32, 26, 4, 22, 86, 92, 54, 44, 87, 35, 0, 29, 42, 43, 41, 46, 53, 82, 99, 89, 72, 95, 60, 90, 23, 3, 40, 7, 51, 11, 94, 80, 93, 25, 81, 47, 5, 62, 84, 31, 61, 10, 2]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 17, 82, 44, 63, 53, 89, 60, 91, 5, 78, 45, 49, 72, 98, 22, 27, 19, 59, 18, 14, 23, 20, 35, 95, 0, 55, 80, 97, 46, 51, 87, 92, 66, 48, 62, 70, 21, 4, 16, 69, 40, 96, 42, 36, 25, 81, 84, 8, 33, 9, 90, 67, 64, 47, 61, 34, 3, 88, 77, 11, 93, 56, 52, 6, 71, 99, 83, 13, 12, 26, 68, 15, 85, 94, 1, 74, 31, 28, 24, 86, 29, 43, 39, 73, 50, 65, 57, 76, 10, 54, 58, 79, 75, 32, 41, 2, 38, 37, 7]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 44, 82, 4, 53, 63, 17, 20, 78, 89, 45, 14, 19, 72, 95, 87, 60, 97, 12, 49, 66, 25, 40, 27, 21, 92, 0, 16, 22, 62, 98, 59, 5, 61, 67, 23, 18, 9, 29, 48, 36, 42, 35, 55, 83, 70, 51, 34, 90, 91, 76, 46, 80, 52, 15, 69, 77, 1, 75, 13, 94, 71, 88, 65, 7, 32, 68, 8, 84, 3, 26, 56, 73, 64, 2, 99, 81, 10, 93, 57, 33, 74, 28, 85, 79, 11, 96, 58, 38, 86, 47, 50, 43, 41, 37, 6, 24, 31, 39, 54]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
- Recall @
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.030303030303030304, 0.030303030303030304, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.12121212121212122, 0.15151515151515152, 0.15151515151515152, 0.15151515151515152, 0.18181818181818182, 0.21212121212121213, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.24242424242424243, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.30303030303030304, 0.30303030303030304, 0.30303030303030304, 0.30303030303030304, 0.3333

####Ground Truth by 2nd Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 5, 53, 30, 17, 49, 63, 55, 0, 60, 61, 44, 14, 19, 27, 95, 51, 45, 4, 59, 42, 98, 29, 90, 22, 48, 80, 78, 69, 20, 40, 76, 18, 87, 72, 96, 3, 21, 46, 8, 35, 12, 67, 52, 89, 16, 91, 34, 66, 84, 36, 75, 47, 6, 62, 97, 71, 31, 81, 77, 33, 88, 1, 26, 64, 56, 13, 15, 70, 65, 41, 85, 11, 25, 9, 86, 23, 83, 92, 57, 7, 54, 73, 39, 74, 68, 2, 58, 32, 28, 50, 43, 79, 94, 10, 37, 38, 24, 99, 93]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 17, 53, 61, 44, 60, 30, 45, 0, 63, 19, 22, 89, 59, 98, 5, 27, 21, 4, 42, 95, 55, 78, 80, 14, 3, 87, 49, 51, 12, 96, 72, 29, 48, 46, 31, 90, 20, 7, 6, 34, 69, 40, 76, 18, 97, 66, 11, 52, 23, 47, 84, 64, 1, 62, 16, 94, 91, 35, 43, 77, 67, 25, 92, 36, 71, 2, 56, 85, 26, 58, 65, 99, 33, 93, 9, 15, 10, 73, 75, 8, 57, 54, 38, 88, 83, 13, 41, 81, 70, 68, 50, 86, 28, 74, 32, 79, 24, 39, 37]
##################################################
# Ground Truth (Unranked):
gt2 = [20, 29, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 17, 53, 45, 0, 61, 29, 30, 60, 4, 63, 5, 89, 98, 44, 49, 87, 22, 14, 21, 78, 27, 59, 72, 3, 62, 55, 69, 19, 34, 95, 51, 42, 52, 18, 66, 90, 64, 48, 40, 12, 80, 46, 92, 35, 96, 33, 20, 84, 9, 36, 47, 81, 16, 11, 97, 67, 56, 57, 76, 75, 94, 91, 73, 71, 43, 23, 1, 6, 8, 85, 88, 25, 77, 15, 70, 68, 26, 31, 38, 93, 65, 83, 13, 58, 24, 7, 54, 74, 99, 86, 50, 39, 79, 41, 28, 10, 32, 37, 2]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 30, 5, 17, 49, 53, 55, 89, 63, 20, 69, 61, 78, 27, 14, 97, 76, 44, 59, 60, 62, 19, 87, 91, 48, 35, 98, 0, 81, 92, 70, 42, 56, 66, 45, 40, 18, 22, 46, 21, 36, 72, 16, 33, 34, 77, 96, 90, 80, 51, 29, 12, 4, 64, 8, 1, 67, 6, 52, 11, 13, 3, 88, 26, 71, 47, 65, 73, 9, 28, 95, 75, 23, 83, 86, 54, 50, 24, 57, 85, 94, 25, 93, 68, 32, 99, 15, 43, 41, 58, 31, 84, 79, 38, 39, 10, 74, 37, 2, 7]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 17, 53, 30, 60, 76, 61, 0, 5, 63, 59, 97, 44, 98, 14, 22, 95, 92, 89, 19, 21, 27, 49, 51, 46, 42, 45, 91, 4, 55, 96, 87, 11, 20, 31, 72, 66, 64, 77, 18, 78, 40, 36, 34, 94, 47, 6, 52, 26, 90, 3, 62, 81, 33, 48, 29, 67, 56, 84, 57, 80, 16, 25, 1, 69, 88, 35, 65, 15, 85, 8, 58, 99, 28, 86, 71, 73, 50, 24, 23, 54, 83, 75, 38, 41, 93, 68, 13, 70, 7, 32, 43, 39, 37, 74, 12, 9, 79, 10, 2]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 25, 26, 42, 46, 51, 52, 59, 63, 80, 88, 95, 98]
# Ranked Result:
rr5 = [82, 30, 63, 5, 53, 14, 60, 89, 17, 44, 76, 49, 4, 72, 19, 45, 22, 97, 20, 55, 78, 95, 40, 21, 0, 66, 29, 80, 59, 27, 62, 34, 48, 3, 51, 87, 18, 91, 8, 92, 61, 7, 16, 25, 12, 67, 46, 42, 35, 1, 69, 2, 36, 98, 23, 71, 15, 70, 52, 90, 31, 88, 9, 65, 47, 64, 96, 28, 84, 75, 10, 26, 11, 6, 56, 77, 83, 33, 94, 99, 57, 73, 32, 68, 85, 39, 13, 58, 81, 50, 79, 43, 86, 54, 41, 38, 74, 37, 93, 24]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 54, 64, 71]
# Ranked Result:
rr6 = [82, 30, 89, 17, 72, 53, 44, 5, 60, 4, 78, 49, 63, 62, 45, 14, 22, 7, 92, 40, 76, 51, 29, 95, 97, 87, 21, 46, 66, 3, 80, 25, 42, 0, 98, 12, 48, 19, 27, 2, 20, 11, 36, 31, 64, 96, 55, 71, 1, 34, 90, 18, 23, 65, 16, 33, 59, 47, 56, 15, 69, 84, 91, 61, 67, 28, 26, 88, 58, 70, 77, 94, 9, 35, 10, 83, 75, 68, 8, 6, 41, 81, 32, 52, 13, 93, 57, 99, 38, 79, 50, 73, 43, 39, 86, 74, 85, 54, 24, 37]
##################################################
# Ground Truth (Unranked):
gt7 = [3, 20, 29, 34, 37, 77, 89]
# Ranked Result:
rr7 = [82, 30, 17, 5, 89, 44, 63, 4, 78, 53, 72, 97, 45, 62, 20, 49, 7, 14, 29, 22, 19, 60, 55, 12, 21, 66, 91, 0, 23, 18, 34, 2, 27, 87, 95, 98, 92, 16, 76, 48, 46, 35, 61, 69, 1, 9, 42, 93, 83, 71, 59, 70, 51, 40, 77, 58, 84, 36, 81, 32, 64, 15, 67, 25, 79, 37, 28, 56, 6, 3, 99, 68, 26, 31, 65, 11, 80, 88, 39, 96, 73, 10, 8, 33, 50, 54, 74, 13, 90, 52, 85, 75, 94, 38, 24, 43, 57, 47, 41, 86]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 44, 17, 22, 53, 89, 82, 60, 72, 78, 45, 25, 23, 63, 95, 5, 61, 98, 91, 80, 51, 40, 19, 35, 62, 18, 14, 0, 84, 70, 42, 27, 92, 87, 16, 46, 55, 59, 20, 11, 96, 49, 3, 21, 71, 6, 66, 48, 36, 67, 10, 94, 81, 4, 9, 7, 69, 8, 97, 32, 33, 90, 93, 28, 56, 99, 13, 77, 1, 12, 31, 34, 83, 15, 47, 26, 73, 68, 43, 2, 29, 65, 88, 52, 86, 58, 85, 39, 50, 75, 74, 64, 54, 57, 41, 79, 24, 38, 37, 76]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [82, 30, 17, 53, 44, 89, 60, 45, 63, 5, 0, 62, 72, 22, 23, 4, 92, 78, 98, 61, 49, 42, 46, 80, 14, 19, 20, 59, 21, 55, 11, 48, 87, 95, 97, 76, 66, 40, 27, 51, 18, 10, 70, 71, 3, 90, 99, 96, 36, 84, 91, 25, 29, 69, 35, 52, 15, 67, 34, 9, 43, 8, 16, 12, 47, 64, 7, 88, 1, 6, 33, 85, 31, 56, 94, 26, 28, 2, 81, 73, 32, 65, 83, 58, 57, 54, 77, 68, 93, 38, 39, 75, 79, 50, 13, 86, 24, 41, 74, 37]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 3, 8, 13, 15, 26, 38, 42, 51, 52, 57, 59, 66, 68, 88, 95]
# Ranked Result:
rr10 = [53, 44, 95, 42, 16, 17, 27, 2, 13, 40, 0, 35, 55, 75, 71, 31, 15, 70, 14, 84, 94, 65, 25, 41, 4, 18, 36, 26, 1, 82, 73, 10, 78, 77, 6, 68, 32, 60, 67, 46, 3, 66, 90, 88, 86, 28, 8, 85, 30, 47, 39, 52, 72, 50, 11, 34, 33, 80, 92, 21, 62, 98, 38, 22, 91, 87, 56, 79, 23, 45, 51, 61, 83, 19, 20, 48, 12, 81, 37, 69, 74, 43, 54, 9, 57, 96, 49, 99, 58, 59, 64, 63, 89, 7, 29, 97, 93, 5, 24, 76]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 86, 88]
# Ranked Result:
rr11 = [4, 40, 78, 95, 0, 94, 53, 44, 25, 23, 10, 42, 18, 66, 35, 15, 45, 30, 71, 27, 36, 87, 21, 67, 12, 16, 9, 22, 62, 98, 75, 47, 31, 46, 3, 73, 1, 68, 70, 13, 65, 84, 90, 80, 72, 2, 34, 57, 92, 88, 56, 14, 43, 86, 52, 32, 89, 39, 33, 38, 96, 85, 17, 50, 6, 29, 8, 26, 77, 55, 11, 51, 41, 48, 74, 37, 91, 54, 69, 81, 28, 83, 99, 60, 7, 64, 79, 24, 20, 82, 58, 19, 93, 63, 59, 97, 61, 49, 5, 76]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69, 34]
# Ranked Result:
rr12 = [63, 97, 20, 34, 14, 76, 19, 49, 98, 66, 21, 91, 64, 69, 18, 9, 48, 52, 1, 59, 27, 16, 67, 36, 70, 83, 55, 96, 57, 56, 8, 15, 50, 24, 12, 77, 33, 71, 73, 38, 88, 85, 45, 79, 65, 75, 58, 28, 68, 78, 74, 13, 37, 30, 6, 39, 17, 32, 26, 4, 22, 86, 92, 54, 44, 87, 35, 0, 29, 42, 43, 41, 46, 53, 82, 99, 89, 72, 95, 60, 90, 23, 3, 40, 7, 51, 11, 94, 80, 93, 25, 81, 47, 5, 62, 84, 31, 61, 10, 2]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 17, 82, 44, 63, 53, 89, 60, 91, 5, 78, 45, 49, 72, 98, 22, 27, 19, 59, 18, 14, 23, 20, 35, 95, 0, 55, 80, 97, 46, 51, 87, 92, 66, 48, 62, 70, 21, 4, 16, 69, 40, 96, 42, 36, 25, 81, 84, 8, 33, 9, 90, 67, 64, 47, 61, 34, 3, 88, 77, 11, 93, 56, 52, 6, 71, 99, 83, 13, 12, 26, 68, 15, 85, 94, 1, 74, 31, 28, 24, 86, 29, 43, 39, 73, 50, 65, 57, 76, 10, 54, 58, 79, 75, 32, 41, 2, 38, 37, 7]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 44, 82, 4, 53, 63, 17, 20, 78, 89, 45, 14, 19, 72, 95, 87, 60, 97, 12, 49, 66, 25, 40, 27, 21, 92, 0, 16, 22, 62, 98, 59, 5, 61, 67, 23, 18, 9, 29, 48, 36, 42, 35, 55, 83, 70, 51, 34, 90, 91, 76, 46, 80, 52, 15, 69, 77, 1, 75, 13, 94, 71, 88, 65, 7, 32, 68, 8, 84, 3, 26, 56, 73, 64, 2, 99, 81, 10, 93, 57, 33, 74, 28, 85, 79, 11, 96, 58, 38, 86, 47, 50, 43, 41, 37, 6, 24, 31, 39, 54]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]
- Recall @
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.08571428571428572, 0.11428571428571428, 0.14285714285714285, 0.17142857142857143, 0.17142857142857143, 0.2, 0.22857142857142856, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2857142857142857, 0.2857142857142857, 0.2857142857142857, 0.2857142857142857, 0.2857142857142857, 0.2857142857142857, 0.3142857142857143, 0.3142857142857143, 0.3142857142857143, 0.3142857142857143, 0.34285714285714286, 0.3428571428

###ALBERT indobenchmark/indobert-lite-base-p2

####Ground Truth by 1st Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 433, 44, 46, 50, 51, 57, 59, 63, 65, 66, 67, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 63, 76, 30, 20, 5, 14, 19, 55, 29, 12, 59, 97, 34, 16, 44, 61, 18, 75, 51, 25, 48, 3, 0, 94, 7, 45, 42, 71, 95, 80, 53, 1, 47, 98, 27, 10, 60, 78, 89, 65, 52, 85, 21, 35, 39, 69, 17, 67, 13, 57, 87, 37, 64, 72, 58, 11, 66, 8, 70, 73, 4, 23, 15, 22, 36, 77, 31, 86, 40, 9, 92, 28, 56, 6, 83, 49, 88, 62, 90, 84, 41, 38, 24, 46, 26, 43, 74, 79, 99, 33, 91, 32, 68, 2, 81, 50, 96, 93, 54]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 76, 30, 63, 5, 97, 20, 14, 12, 29, 55, 61, 19, 34, 59, 7, 48, 3, 45, 58, 64, 21, 28, 25, 11, 1, 44, 78, 94, 70, 16, 22, 72, 75, 89, 42, 69, 27, 87, 53, 77, 18, 92, 71, 51, 98, 6, 37, 47, 95, 0, 31, 56, 23, 67, 80, 85, 65, 35, 60, 79, 73, 83, 39, 66, 91, 4, 36, 52, 32, 9, 99, 17, 57, 86, 62, 8, 13, 15, 40, 49, 24, 10, 46, 38, 33, 74, 26, 41, 90, 88, 50, 84, 81, 93, 68, 43, 2, 96, 54]
##################################################
# Ground Truth (Unranked):
gt2 = [31, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 76, 30, 5, 63, 20, 97, 29, 14, 34, 12, 61, 19, 45, 59, 55, 48, 21, 7, 64, 28, 3, 78, 58, 11, 69, 72, 77, 25, 89, 1, 70, 22, 92, 75, 37, 94, 98, 91, 16, 44, 42, 27, 18, 87, 49, 53, 79, 62, 51, 47, 0, 67, 6, 73, 60, 85, 99, 23, 80, 71, 83, 4, 9, 56, 17, 39, 35, 31, 95, 32, 66, 36, 52, 65, 24, 57, 46, 86, 15, 33, 8, 40, 10, 13, 41, 74, 93, 50, 84, 68, 81, 26, 88, 43, 38, 90, 2, 96, 54]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 76, 30, 97, 63, 20, 5, 29, 55, 45, 28, 64, 59, 12, 19, 58, 34, 48, 92, 78, 14, 91, 23, 32, 79, 69, 21, 61, 11, 89, 72, 77, 70, 3, 7, 25, 22, 75, 99, 16, 37, 56, 98, 83, 47, 73, 6, 24, 44, 1, 39, 93, 65, 87, 42, 9, 17, 53, 27, 49, 71, 95, 18, 67, 31, 85, 94, 60, 33, 36, 66, 0, 80, 46, 35, 51, 52, 4, 86, 57, 10, 74, 81, 62, 8, 13, 43, 41, 84, 2, 15, 38, 50, 40, 26, 88, 68, 96, 90, 54]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 30, 76, 97, 5, 45, 55, 20, 63, 64, 14, 77, 34, 73, 19, 28, 61, 72, 11, 3, 75, 49, 27, 70, 21, 33, 59, 79, 83, 12, 58, 99, 67, 6, 78, 24, 92, 16, 60, 25, 69, 91, 39, 1, 29, 98, 87, 37, 42, 47, 56, 94, 32, 31, 23, 35, 85, 15, 57, 9, 95, 8, 7, 48, 40, 17, 62, 65, 74, 80, 52, 66, 84, 81, 51, 86, 36, 18, 89, 22, 53, 71, 0, 41, 38, 13, 88, 26, 68, 46, 50, 96, 10, 90, 2, 44, 43, 93, 54, 4]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 18, 25, 26, 42, 44, 46, 51, 57, 73, 80]
# Ranked Result:
rr5 = [82, 63, 76, 30, 20, 14, 55, 19, 12, 29, 25, 59, 44, 97, 16, 5, 18, 53, 0, 3, 75, 7, 34, 61, 71, 51, 48, 78, 17, 42, 95, 94, 69, 58, 89, 60, 10, 72, 27, 1, 65, 98, 80, 45, 21, 87, 13, 4, 39, 37, 67, 85, 22, 8, 11, 15, 52, 47, 64, 73, 35, 57, 40, 92, 31, 36, 66, 70, 77, 23, 83, 62, 86, 6, 9, 28, 38, 46, 41, 56, 88, 79, 84, 90, 99, 49, 24, 26, 43, 74, 93, 33, 2, 68, 32, 81, 91, 50, 96, 54]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 71, 96]
# Ranked Result:
rr6 = [82, 63, 76, 20, 30, 12, 55, 14, 97, 19, 7, 44, 29, 59, 61, 25, 71, 16, 89, 5, 95, 58, 18, 48, 75, 31, 11, 53, 94, 78, 3, 69, 4, 22, 0, 10, 60, 23, 87, 34, 92, 42, 39, 98, 37, 64, 67, 65, 72, 17, 1, 51, 70, 45, 27, 47, 13, 40, 15, 21, 85, 6, 36, 35, 80, 28, 73, 8, 66, 57, 86, 56, 52, 84, 93, 24, 2, 83, 77, 79, 38, 9, 90, 41, 81, 99, 32, 62, 74, 91, 46, 68, 50, 49, 26, 88, 43, 33, 96, 54]
##################################################
# Ground Truth (Unranked):
gt7 = [3]
# Ranked Result:
rr7 = [82, 76, 30, 5, 20, 63, 29, 45, 12, 14, 61, 34, 69, 97, 19, 78, 49, 55, 62, 21, 89, 16, 48, 25, 3, 59, 11, 7, 58, 37, 94, 77, 75, 71, 67, 18, 42, 0, 44, 79, 66, 17, 4, 73, 80, 98, 36, 35, 95, 83, 51, 92, 1, 64, 53, 70, 65, 9, 31, 28, 60, 39, 57, 47, 87, 91, 13, 72, 24, 8, 52, 23, 85, 15, 10, 86, 27, 22, 99, 32, 56, 74, 6, 40, 43, 68, 2, 88, 26, 41, 46, 38, 81, 84, 90, 93, 33, 50, 96, 54]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 97, 82, 20, 45, 76, 91, 29, 92, 72, 64, 63, 5, 58, 78, 61, 12, 89, 23, 69, 11, 22, 48, 19, 59, 55, 79, 93, 28, 77, 32, 7, 3, 24, 99, 75, 34, 21, 16, 14, 73, 17, 70, 4, 44, 98, 87, 25, 83, 67, 71, 49, 31, 37, 39, 0, 42, 62, 9, 47, 56, 2, 60, 36, 94, 81, 74, 51, 66, 18, 95, 65, 1, 6, 80, 53, 10, 57, 50, 8, 86, 40, 85, 35, 43, 68, 52, 15, 84, 41, 33, 46, 38, 13, 27, 88, 96, 26, 90, 54]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 82, 20, 76, 45, 69, 63, 55, 97, 5, 61, 92, 11, 12, 89, 16, 3, 29, 14, 58, 49, 34, 23, 75, 87, 4, 78, 19, 17, 25, 21, 91, 64, 94, 73, 67, 77, 7, 71, 37, 59, 10, 42, 62, 39, 60, 36, 72, 18, 95, 83, 79, 44, 66, 48, 35, 15, 70, 65, 32, 24, 31, 0, 57, 9, 98, 81, 28, 74, 80, 1, 51, 13, 47, 27, 22, 52, 8, 99, 86, 53, 85, 93, 2, 68, 6, 41, 38, 43, 40, 84, 33, 88, 26, 56, 50, 90, 96, 46, 54]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 8, 13, 15, 16, 38, 42, 51, 52, 57, 59, 66, 68, 95]
# Ranked Result:
rr10 = [63, 16, 14, 55, 82, 71, 25, 95, 75, 0, 44, 13, 51, 20, 18, 59, 65, 94, 30, 42, 10, 3, 15, 8, 60, 34, 80, 69, 53, 73, 39, 98, 85, 17, 86, 12, 52, 57, 40, 19, 47, 67, 78, 66, 27, 35, 29, 1, 9, 36, 37, 87, 61, 26, 48, 58, 76, 21, 88, 64, 6, 23, 72, 41, 97, 38, 90, 11, 68, 70, 7, 45, 31, 4, 84, 56, 74, 77, 2, 89, 96, 43, 83, 46, 24, 79, 92, 33, 62, 28, 99, 22, 5, 32, 81, 50, 49, 93, 54, 91]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 55, 65, 86, 88]
# Ranked Result:
rr11 = [16, 71, 75, 14, 18, 95, 63, 0, 44, 98, 55, 94, 10, 59, 42, 12, 47, 15, 67, 65, 23, 20, 73, 39, 3, 57, 87, 52, 86, 53, 80, 2, 36, 35, 19, 13, 40, 25, 51, 34, 31, 24, 85, 64, 27, 60, 84, 8, 29, 30, 66, 78, 61, 1, 90, 56, 48, 41, 38, 7, 11, 9, 74, 17, 37, 92, 4, 70, 68, 69, 6, 89, 58, 88, 96, 43, 22, 72, 45, 46, 82, 79, 26, 97, 28, 50, 21, 33, 81, 99, 77, 93, 83, 32, 76, 5, 91, 54, 49, 62]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69]
# Ranked Result:
rr12 = [69, 30, 20, 34, 45, 82, 19, 48, 78, 11, 5, 25, 83, 37, 12, 49, 55, 29, 67, 66, 4, 94, 63, 21, 3, 16, 61, 77, 62, 1, 65, 58, 9, 75, 13, 70, 14, 42, 64, 7, 87, 76, 89, 95, 36, 80, 18, 44, 79, 72, 71, 32, 92, 17, 59, 39, 23, 35, 73, 8, 53, 31, 99, 91, 60, 51, 28, 86, 57, 0, 43, 98, 47, 74, 40, 26, 15, 81, 6, 24, 41, 52, 85, 38, 56, 10, 27, 68, 93, 88, 97, 2, 84, 22, 50, 46, 90, 96, 33, 54]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 45, 91, 78, 64, 79, 11, 58, 55, 72, 20, 92, 69, 61, 23, 5, 32, 77, 73, 97, 48, 3, 93, 19, 28, 24, 70, 83, 17, 12, 59, 16, 87, 39, 99, 75, 34, 81, 29, 37, 82, 67, 36, 66, 21, 89, 22, 98, 49, 44, 25, 63, 74, 65, 42, 1, 9, 7, 2, 4, 31, 95, 94, 8, 57, 71, 51, 62, 86, 76, 18, 15, 80, 56, 14, 0, 47, 35, 84, 6, 33, 13, 43, 10, 41, 88, 68, 52, 38, 60, 40, 53, 26, 50, 27, 96, 85, 46, 90, 54]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 20, 45, 61, 63, 5, 11, 69, 3, 34, 19, 58, 12, 55, 78, 1, 16, 87, 48, 14, 94, 7, 67, 83, 64, 25, 77, 92, 76, 59, 42, 29, 82, 28, 75, 65, 21, 66, 70, 37, 4, 73, 23, 35, 95, 36, 49, 89, 18, 39, 13, 79, 44, 9, 80, 72, 47, 71, 91, 86, 24, 31, 97, 56, 60, 8, 15, 52, 32, 27, 17, 99, 6, 51, 85, 57, 53, 40, 0, 62, 74, 38, 41, 10, 81, 98, 26, 43, 93, 50, 22, 88, 90, 68, 2, 84, 46, 33, 96, 54]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
- Recall @
[0.0, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.030303030303030304, 0.06060606060606061, 0.06060606060606061, 0.06060606060606061, 0.09090909090909091, 0.12121212121212122, 0.12121212121212122, 0.15151515151515152, 0.18181818181818182, 0.21212121212121213, 0.21212121212121213, 0.21212121212121213, 0.24242424242424243, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.2727272727272727, 0.30303030303030304, 0.30303030303030304, 0.3333333333333333, 0.36363636363636365, 0.36363636363636365, 0

####Ground Truth by 2nd Assessor.

In [ ]:
# Ground Truth (Unranked):
gt0 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 23, 26, 38, 39, 42, 43, 44, 45, 50, 51, 52, 57, 59, 63, 65, 66, 67, 70, 73, 74, 75, 80, 86, 88, 95, 98]
# Ranked Result:
rr0 = [82, 63, 76, 30, 20, 5, 14, 19, 55, 29, 12, 59, 97, 34, 16, 44, 61, 18, 75, 51, 25, 48, 3, 0, 94, 7, 45, 42, 71, 95, 80, 53, 1, 47, 98, 27, 10, 60, 78, 89, 65, 52, 85, 21, 35, 39, 69, 17, 67, 13, 57, 87, 37, 64, 72, 58, 11, 66, 8, 70, 73, 4, 23, 15, 22, 36, 77, 31, 86, 40, 9, 92, 28, 56, 6, 83, 49, 88, 62, 90, 84, 41, 38, 24, 46, 26, 43, 74, 79, 99, 33, 91, 32, 68, 2, 81, 50, 96, 93, 54]
##################################################
# Ground Truth (Unranked):
gt1 = [36, 54, 64]
# Ranked Result:
rr1 = [82, 76, 30, 63, 5, 97, 20, 14, 12, 29, 55, 61, 19, 34, 59, 7, 48, 3, 45, 58, 64, 21, 28, 25, 11, 1, 44, 78, 94, 70, 16, 22, 72, 75, 89, 42, 69, 27, 87, 53, 77, 18, 92, 71, 51, 98, 6, 37, 47, 95, 0, 31, 56, 23, 67, 80, 85, 65, 35, 60, 79, 73, 83, 39, 66, 91, 4, 36, 52, 32, 9, 99, 17, 57, 86, 62, 8, 13, 15, 40, 49, 24, 10, 46, 38, 33, 74, 26, 41, 90, 88, 50, 84, 81, 93, 68, 43, 2, 96, 54]
##################################################
# Ground Truth (Unranked):
gt2 = [20, 29, 34, 37, 77, 89]
# Ranked Result:
rr2 = [82, 76, 30, 5, 63, 20, 97, 29, 14, 34, 12, 61, 19, 45, 59, 55, 48, 21, 7, 64, 28, 3, 78, 58, 11, 69, 72, 77, 25, 89, 1, 70, 22, 92, 75, 37, 94, 98, 91, 16, 44, 42, 27, 18, 87, 49, 53, 79, 62, 51, 47, 0, 67, 6, 73, 60, 85, 99, 23, 80, 71, 83, 4, 9, 56, 17, 39, 35, 31, 95, 32, 66, 36, 52, 65, 24, 57, 46, 86, 15, 33, 8, 40, 10, 13, 41, 74, 93, 50, 84, 68, 81, 26, 88, 43, 38, 90, 2, 96, 54]
##################################################
# Ground Truth (Unranked):
gt3 = [91]
# Ranked Result:
rr3 = [82, 76, 30, 97, 63, 20, 5, 29, 55, 45, 28, 64, 59, 12, 19, 58, 34, 48, 92, 78, 14, 91, 23, 32, 79, 69, 21, 61, 11, 89, 72, 77, 70, 3, 7, 25, 22, 75, 99, 16, 37, 56, 98, 83, 47, 73, 6, 24, 44, 1, 39, 93, 65, 87, 42, 9, 17, 53, 27, 49, 71, 95, 18, 67, 31, 85, 94, 60, 33, 36, 66, 0, 80, 46, 35, 51, 52, 4, 86, 57, 10, 74, 81, 62, 8, 13, 43, 41, 84, 2, 15, 38, 50, 40, 26, 88, 68, 96, 90, 54]
##################################################
# Ground Truth (Unranked):
gt4 = [30]
# Ranked Result:
rr4 = [82, 30, 76, 97, 5, 45, 55, 20, 63, 64, 14, 77, 34, 73, 19, 28, 61, 72, 11, 3, 75, 49, 27, 70, 21, 33, 59, 79, 83, 12, 58, 99, 67, 6, 78, 24, 92, 16, 60, 25, 69, 91, 39, 1, 29, 98, 87, 37, 42, 47, 56, 94, 32, 31, 23, 35, 85, 15, 57, 9, 95, 8, 7, 48, 40, 17, 62, 65, 74, 80, 52, 66, 84, 81, 51, 86, 36, 18, 89, 22, 53, 71, 0, 41, 38, 13, 88, 26, 68, 46, 50, 96, 10, 90, 2, 44, 43, 93, 54, 4]
##################################################
# Ground Truth (Unranked):
gt5 = [0, 1, 3, 8, 10, 13, 15, 16, 18, 25, 26, 42, 46, 51, 52, 59, 63, 80, 88, 95, 98]
# Ranked Result:
rr5 = [82, 63, 76, 30, 20, 14, 55, 19, 12, 29, 25, 59, 44, 97, 16, 5, 18, 53, 0, 3, 75, 7, 34, 61, 71, 51, 48, 78, 17, 42, 95, 94, 69, 58, 89, 60, 10, 72, 27, 1, 65, 98, 80, 45, 21, 87, 13, 4, 39, 37, 67, 85, 22, 8, 11, 15, 52, 47, 64, 73, 35, 57, 40, 92, 31, 36, 66, 70, 77, 23, 83, 62, 86, 6, 9, 28, 38, 46, 41, 56, 88, 79, 84, 90, 99, 49, 24, 26, 43, 74, 93, 33, 2, 68, 32, 81, 91, 50, 96, 54]
##################################################
# Ground Truth (Unranked):
gt6 = [35, 36, 38, 54, 64, 71]
# Ranked Result:
rr6 = [82, 63, 76, 20, 30, 12, 55, 14, 97, 19, 7, 44, 29, 59, 61, 25, 71, 16, 89, 5, 95, 58, 18, 48, 75, 31, 11, 53, 94, 78, 3, 69, 4, 22, 0, 10, 60, 23, 87, 34, 92, 42, 39, 98, 37, 64, 67, 65, 72, 17, 1, 51, 70, 45, 27, 47, 13, 40, 15, 21, 85, 6, 36, 35, 80, 28, 73, 8, 66, 57, 86, 56, 52, 84, 93, 24, 2, 83, 77, 79, 38, 9, 90, 41, 81, 99, 32, 62, 74, 91, 46, 68, 50, 49, 26, 88, 43, 33, 96, 54]
##################################################
# Ground Truth (Unranked):
gt7 = [3, 20, 29, 34, 37, 77, 89]
# Ranked Result:
rr7 = [82, 76, 30, 5, 20, 63, 29, 45, 12, 14, 61, 34, 69, 97, 19, 78, 49, 55, 62, 21, 89, 16, 48, 25, 3, 59, 11, 7, 58, 37, 94, 77, 75, 71, 67, 18, 42, 0, 44, 79, 66, 17, 4, 73, 80, 98, 36, 35, 95, 83, 51, 92, 1, 64, 53, 70, 65, 9, 31, 28, 60, 39, 57, 47, 87, 91, 13, 72, 24, 8, 52, 23, 85, 15, 10, 86, 27, 22, 99, 32, 56, 74, 6, 40, 43, 68, 2, 88, 26, 41, 46, 38, 81, 84, 90, 93, 33, 50, 96, 54]
##################################################
# Ground Truth (Unranked):
gt8 = [91]
# Ranked Result:
rr8 = [30, 97, 82, 20, 45, 76, 91, 29, 92, 72, 64, 63, 5, 58, 78, 61, 12, 89, 23, 69, 11, 22, 48, 19, 59, 55, 79, 93, 28, 77, 32, 7, 3, 24, 99, 75, 34, 21, 16, 14, 73, 17, 70, 4, 44, 98, 87, 25, 83, 67, 71, 49, 31, 37, 39, 0, 42, 62, 9, 47, 56, 2, 60, 36, 94, 81, 74, 51, 66, 18, 95, 65, 1, 6, 80, 53, 10, 57, 50, 8, 86, 40, 85, 35, 43, 68, 52, 15, 84, 41, 33, 46, 38, 13, 27, 88, 96, 26, 90, 54]
##################################################
# Ground Truth (Unranked):
gt9 = [30]
# Ranked Result:
rr9 = [30, 82, 20, 76, 45, 69, 63, 55, 97, 5, 61, 92, 11, 12, 89, 16, 3, 29, 14, 58, 49, 34, 23, 75, 87, 4, 78, 19, 17, 25, 21, 91, 64, 94, 73, 67, 77, 7, 71, 37, 59, 10, 42, 62, 39, 60, 36, 72, 18, 95, 83, 79, 44, 66, 48, 35, 15, 70, 65, 32, 24, 31, 0, 57, 9, 98, 81, 28, 74, 80, 1, 51, 13, 47, 27, 22, 52, 8, 99, 86, 53, 85, 93, 2, 68, 6, 41, 38, 43, 40, 84, 33, 88, 26, 56, 50, 90, 96, 46, 54]
##################################################
# Ground Truth (Unranked):
gt10 = [0, 1, 2, 3, 8, 13, 15, 26, 38, 42, 51, 52, 57, 59, 66, 68, 88, 95]
# Ranked Result:
rr10 = [63, 16, 14, 55, 82, 71, 25, 95, 75, 0, 44, 13, 51, 20, 18, 59, 65, 94, 30, 42, 10, 3, 15, 8, 60, 34, 80, 69, 53, 73, 39, 98, 85, 17, 86, 12, 52, 57, 40, 19, 47, 67, 78, 66, 27, 35, 29, 1, 9, 36, 37, 87, 61, 26, 48, 58, 76, 21, 88, 64, 6, 23, 72, 41, 97, 38, 90, 11, 68, 70, 7, 45, 31, 4, 84, 56, 74, 77, 2, 89, 96, 43, 83, 46, 24, 79, 92, 33, 62, 28, 99, 22, 5, 32, 81, 50, 49, 93, 54, 91]
##################################################
# Ground Truth (Unranked):
gt11 = [1, 2, 8, 10, 13, 15, 16, 18, 24, 36, 39, 41, 47, 86, 88]
# Ranked Result:
rr11 = [16, 71, 75, 14, 18, 95, 63, 0, 44, 98, 55, 94, 10, 59, 42, 12, 47, 15, 67, 65, 23, 20, 73, 39, 3, 57, 87, 52, 86, 53, 80, 2, 36, 35, 19, 13, 40, 25, 51, 34, 31, 24, 85, 64, 27, 60, 84, 8, 29, 30, 66, 78, 61, 1, 90, 56, 48, 41, 38, 7, 11, 9, 74, 17, 37, 92, 4, 70, 68, 69, 6, 89, 58, 88, 96, 43, 22, 72, 45, 46, 82, 79, 26, 97, 28, 50, 21, 33, 81, 99, 77, 93, 83, 32, 76, 5, 91, 54, 49, 62]
##################################################
# Ground Truth (Unranked):
gt12 = [20, 69, 34]
# Ranked Result:
rr12 = [69, 30, 20, 34, 45, 82, 19, 48, 78, 11, 5, 25, 83, 37, 12, 49, 55, 29, 67, 66, 4, 94, 63, 21, 3, 16, 61, 77, 62, 1, 65, 58, 9, 75, 13, 70, 14, 42, 64, 7, 87, 76, 89, 95, 36, 80, 18, 44, 79, 72, 71, 32, 92, 17, 59, 39, 23, 35, 73, 8, 53, 31, 99, 91, 60, 51, 28, 86, 57, 0, 43, 98, 47, 74, 40, 26, 15, 81, 6, 24, 41, 52, 85, 38, 56, 10, 27, 68, 93, 88, 97, 2, 84, 22, 50, 46, 90, 96, 33, 54]
##################################################
# Ground Truth (Unranked):
gt13 = [91]
# Ranked Result:
rr13 = [30, 45, 91, 78, 64, 79, 11, 58, 55, 72, 20, 92, 69, 61, 23, 5, 32, 77, 73, 97, 48, 3, 93, 19, 28, 24, 70, 83, 17, 12, 59, 16, 87, 39, 99, 75, 34, 81, 29, 37, 82, 67, 36, 66, 21, 89, 22, 98, 49, 44, 25, 63, 74, 65, 42, 1, 9, 7, 2, 4, 31, 95, 94, 8, 57, 71, 51, 62, 86, 76, 18, 15, 80, 56, 14, 0, 47, 35, 84, 6, 33, 13, 43, 10, 41, 88, 68, 52, 38, 60, 40, 53, 26, 50, 27, 96, 85, 46, 90, 54]
##################################################
# Ground Truth (Unranked):
gt14 = [30]
# Ranked Result:
rr14 = [30, 20, 45, 61, 63, 5, 11, 69, 3, 34, 19, 58, 12, 55, 78, 1, 16, 87, 48, 14, 94, 7, 67, 83, 64, 25, 77, 92, 76, 59, 42, 29, 82, 28, 75, 65, 21, 66, 70, 37, 4, 73, 23, 35, 95, 36, 49, 89, 18, 39, 13, 79, 44, 9, 80, 72, 47, 71, 91, 86, 24, 31, 97, 56, 60, 8, 15, 52, 32, 27, 17, 99, 6, 51, 85, 57, 53, 40, 0, 62, 74, 38, 41, 10, 81, 98, 26, 43, 93, 50, 22, 88, 90, 68, 2, 84, 46, 33, 96, 54]
##################################################


In [ ]:
evaluate()

### Test 1 ###
- Match @
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
- Recall @
[0.0, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.02857142857142857, 0.05714285714285714, 0.05714285714285714, 0.05714285714285714, 0.08571428571428572, 0.11428571428571428, 0.11428571428571428, 0.14285714285714285, 0.17142857142857143, 0.2, 0.2, 0.2, 0.22857142857142856, 0.2571428571428571, 0.2571428571428571, 0.2571428571428571, 0.2857142857142857, 0.3142857142857143, 0.3142857142857143, 0.34285714285714286, 0.37142857142857144, 0.37142857142857144, 0.4, 0.4, 0.42857142857142855, 0.42857142857142855, 0.457142

##3. Vocabulary
If the word is broken down to be some subwords, it means Out of Vocabulary (OOV).

###xlm-r-100langs-bert-base-nli-stsb-mean-tokens.

In [ ]:
# ['▁gejala', '▁corona']
# ['▁penu', 'lar', 'an', '▁corona']
# ['▁pencegahan', '▁corona']
# ['▁isola', 'si', '▁mandiri']
# ['▁o', 'lah', '▁raga']
# ['▁Apa', '▁gejala', '▁corona', '?']
# ['▁Apa', '▁saya', '▁ter', 'tu', 'lar', '▁corona', '?']
# ['▁Apa', '▁yang', '▁dapat', '▁mencegah', '▁corona', '?']
# ['▁Perlu', 'kah', '▁isola', 'si', '▁mandiri', '▁di', '▁rumah', '?']
# ['▁Apakah', '▁ber', 'olah', 'raga', '▁renang', '▁aman', '?']
# ['▁Pil', 'ek', ',', '▁sakit', '▁tenggorokan', '.', '▁Apa', '▁itu', '▁gejala', '▁corona', '?']
# ['▁Saya', '▁de', 'man', ',', '▁dan', '▁sakit', '▁tenggorokan', '.', '▁Apakah', '▁saya', '▁ter', 'tu', 'lar', '▁corona', '?']
# ['▁Apakah', '▁vaksin', '▁dapat', '▁mencegah', '▁corona', '?', '▁setidaknya', '▁mengurangi', '▁kerusakan', '▁akibat', '▁corona', '?']
# ['▁Perlu', 'kah', '▁isola', 'si', '▁mandiri', '▁di', '▁rumah', '?', '▁memperhatikan', '▁kebersihan', '▁diri', '?']
# ['▁Apakah', '▁ber', 'olah', 'raga', '▁renang', '▁aman', '?', '▁dengan', '▁penyebaran', '▁virus', '▁corona', '▁saat', '▁ini', '?']

OOV: penular, isolasi, tular, olah, pilek.

###indobenchmark/indobert-base-p2

In [ ]:
# ['gejala', 'cor', '##ona']
# ['penularan', 'cor', '##ona']
# ['pencegahan', 'cor', '##ona']
# ['isolasi', 'mandiri']
# ['olah', 'raga']
# ['apa', 'gejala', 'cor', '##ona', '?']
# ['apa', 'saya', 'tertular', 'cor', '##ona', '?']
# ['apa', 'yang', 'dapat', 'mencegah', 'cor', '##ona', '?']
# ['perlu', '##kah', 'isolasi', 'mandiri', 'di', 'rumah', '?']
# ['apakah', 'berolahraga', 'renang', 'aman', '?']
# ['pilek', ',', 'sakit', 'tenggorokan', '.', 'apa', 'itu', 'gejala', 'cor', '##ona', '?']
# ['saya', 'dem', '##an', ',', 'dan', 'sakit', 'tenggorokan', '.', 'apakah', 'saya', 'tertular', 'cor', '##ona', '?']
# ['apakah', 'vaksin', 'dapat', 'mencegah', 'cor', '##ona', '?', 'setidaknya', 'mengurangi', 'kerusakan', 'akibat', 'cor', '##ona', '?']
# ['perlu', '##kah', 'isolasi', 'mandiri', 'di', 'rumah', '?', 'memperhatikan', 'kebersihan', 'diri', '?']
# ['apakah', 'berolahraga', 'renang', 'aman', '?', 'dengan', 'penyebaran', 'virus', 'cor', '##ona', 'saat', 'ini', '?']

OOV: corona.

###indolem/indobert-base-uncased

In [ ]:
# ['gejala', 'cor', '##ona']
# ['penularan', 'cor', '##ona']
# ['pencegahan', 'cor', '##ona']
# ['isolasi', 'mandiri']
# ['olah', 'raga']
# ['apa', 'gejala', 'cor', '##ona', '?']
# ['apa', 'saya', 'tertular', 'cor', '##ona', '?']
# ['apa', 'yang', 'dapat', 'mencegah', 'cor', '##ona', '?']
# ['perlu', '##kah', 'isolasi', 'mandiri', 'di', 'rumah', '?']
# ['apakah', 'berolahraga', 'renang', 'aman', '?']
# ['pil', '##ek', ',', 'sakit', 'tenggorokan', '.', 'apa', 'itu', 'gejala', 'cor', '##ona', '?']
# ['saya', 'dem', '##an', ',', 'dan', 'sakit', 'tenggorokan', '.', 'apakah', 'saya', 'tertular', 'cor', '##ona', '?']
# ['apakah', 'vaksin', 'dapat', 'mencegah', 'cor', '##ona', '?', 'setidaknya', 'mengurangi', 'kerusakan', 'akibat', 'cor', '##ona', '?']
# ['perlu', '##kah', 'isolasi', 'mandiri', 'di', 'rumah', '?', 'memperhatikan', 'kebersihan', 'diri', '?']
# ['apakah', 'berolahraga', 'renang', 'aman', '?', 'dengan', 'penyebaran', 'virus', 'cor', '##ona', 'saat', 'ini', '?']


OOV: corona, pilek.

###bert-base-multilingual-cased

In [ ]:
# ['ge', '##jala', 'corona']
# ['pen', '##ular', '##an', 'corona']
# ['pen', '##ceg', '##ahan', 'corona']
# ['isola', '##si', 'mand', '##iri']
# ['ola', '##h', 'ra', '##ga']
# ['A', '##pa', 'ge', '##jala', 'corona', '?']
# ['A', '##pa', 'saya', 'ter', '##tula', '##r', 'corona', '?']
# ['A', '##pa', 'yang', 'dapat', 'men', '##ceg', '##ah', 'corona', '?']
# ['Perl', '##uka', '##h', 'isola', '##si', 'mand', '##iri', 'di', 'rumah', '?']
# ['A', '##paka', '##h', 'be', '##rol', '##ah', '##rag', '##a', 'ren', '##ang', 'ama', '##n', '?']
# ['Pi', '##lek', ',', 'sakit', 'ten', '##ggo', '##roka', '##n', '.', 'A', '##pa', 'itu', 'ge', '##jala', 'corona', '?']
# ['Say', '##a', 'dem', '##an', ',', 'dan', 'sakit', 'ten', '##ggo', '##roka', '##n', '.', 'A', '##paka', '##h', 'saya', 'ter', '##tula', '##r', 'corona', '?']
# ['A', '##paka', '##h', 'va', '##ksi', '##n', 'dapat', 'men', '##ceg', '##ah', 'corona', '?', 'set', '##ida', '##knya', 'men', '##gura', '##ngi', 'kerusakan', 'akibat', 'corona', '?']
# ['Perl', '##uka', '##h', 'isola', '##si', 'mand', '##iri', 'di', 'rumah', '?', 'me', '##mper', '##hat', '##ikan', 'ke', '##bers', '##ihan', 'diri', '?']
# ['A', '##paka', '##h', 'be', '##rol', '##ah', '##rag', '##a', 'ren', '##ang', 'ama', '##n', '?', 'dengan', 'pen', '##ye', '##baran', 'virus', 'corona', 'saat', 'ini', '?']

It's quite confusing why m-BERT tokenization produces subwords like that. Is it really out of vocabulary?

##4. Discussion

**Model Rangking:**
1. BERT bert-base-multilingual-cased
- Ground Truth by Assessor 1: MAP = 0.3902452427003428 (Best)
- Ground Truth by Assessor 2: MAP = 0.4150421632429007 (Best)
2. SBERT bert-base-multilingual-cased
- Ground Truth by Assessor 1: MAP = 0.3902452427003428 (Best)
- Ground Truth by Assessor 2: MAP = 0.4089237044069697
3. BERT indobenchmark/indobert-base-p2
- Ground Truth by Assessor 1: MAP = 0.3877980987064341
- Ground Truth by Assessor 2: MAP = 0.3967489637185163
4. SBERT indobenchmark/indobert-base-p2
- Ground Truth by Assessor 1: MAP = 0.38765058447945505
- Ground Truth by Assessor 2: MAP = 0.39620738236291886
5. ALBERT indobenchmark/indobert-lite-base-p2
- Ground Truth by Assessor 1: MAP = 0.3498230154820345
- Ground Truth by Assessor 2: MAP = 0.3732223998946945
6. SBERT xlm-r-100langs-bert-base-nli-stsb-mean-tokens.
- Ground Truth by Assessor 1: MAP = 0.2721887412104953
- Ground Truth by Assessor 2: MAP = 0.28898470921330954
7. BERT indolem/indobert-base-uncased
- Ground Truth by Assessor 1: MAP = 0.22113510924339766 (Worst)
- Ground Truth by Assessor 2: MAP = 0.2441372443167979
8. SBERT indolem/indobert-base-uncased
- Ground Truth by Assessor 1: MAP = 0.22120440995694593
- Ground Truth by Assessor 2: MAP = 0.24366042187549788 (Worst)

**Analyze the Models:**

Generally SBERT with bert-base-multilingual-cased and indobenchmark/indobert-base-p2 pretrained model win in combination of speed and precision. The best two options. There is a trade-off anyway.

SBERT and BERT using the same pretrained model have the same result precision.

ALBERT's average precission is lower than BERT's because ALBERT has far fewer parameters than BERT. However it's a good achievement of ALBERT having result close to BERT.

Generally the average precisions are bad. It is most likely because of:
1. The questions in our dataset are not broken down and separated by "[SEP]" token.
2. BERT's nature as word embedding, not sentence embedding.
3. Hidden Layer selection and the process of deducing sentence vector.
4. Out of vocabulary.
5. Bad testbed. It's hard to make a close-to-perfect testbed.

**Analyze the Testbed and the Testing Result:**

We make 3 types of query:
1. Phrase (2-3 words).
2. Question (1 sentence).
3. Long Question (2 sentences).

Besides words, the length of sentences affects the similarity. Our queries tend to be shorter than the questions in the dataset. So, short question will most likely appear in higher rank. It's harder to get correct semantic similarity match between query and sentences than just classify/cluster them into some classes.

**More variation we can do:**
1. Try evaluation using Euclidean Distance.
2. Try other Hidden Layer selections and processes of deducing sentence vector. There are many approaches.
3. Try manually splitting sentence and put "[SEP]" token between sentences.
4. Try other pretrained models, containing Indonesian language, available on sbert.net and Huggingface.

# E. Some Notes

Please take a look at this issue about Sentence Embedding on BERT.
- https://github.com/huggingface/transformers/issues/2298
- https://github.com/UKPLab/sentence-transformers/issues/80

# F. Future Works

After we learn BERT for the first time, the next things to do in next semester (in NLP and Advanced IR) are:
1. Explore Q&A System using SQuAD and Natural Question dataset.
2. Generate Indonesian health Q&A dataset for fine-tuning.